# Escape fractions <a class="tocSkip">

    
the aim of this notebook is to combine the HII-region and cluster catalogues.
   
This notebook useses multiple galaxies at once.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from astrotools.packages import *
from astrotools.constants import tab10, single_column, two_column, thesis_width

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logger = logging.getLogger('pymuse')
handler = logging.StreamHandler(stream=sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)-15s %(message)s",datefmt='%H:%M:%S')
handler.setFormatter(fmt)
logger.addHandler(handler)

# first we need to specify the path to the raw data
basedir = Path('..')
data_ext = Path('a:')/'Archive' #basedir / 'data' / 'raw' 

sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])
sample_table['power_index'] = 2.3
sample_table['power_index'][sample_table['AO'].mask]=2.8
sample_table['distance'] = Distance(distmod=sample_table['(m-M)'])

sample_table_v1p6 = Table.read(basedir/'data'/'external'/'phangs_sample_table_v1p6.fits')
sample_table_v1p6 = sample_table_v1p6[sample_table_v1p6['survey_muse_status']=='released'].copy()
sample_table_v1p6['gal_name'] = [x.upper() for x in sample_table_v1p6['name']]
sample_table_v1p6['dist_err'] = sample_table_v1p6['dist']*sample_table_v1p6['dist_unc']*np.log(10)
sample_table_v1p6.add_index('gal_name')
# we are interested in props_mstar, dist, dist_err, dist_unc

## Read in data

In [ ]:
# the association catalogue matched with the nebuale catalogue
version = 'v1p2'
for HSTband in ['nuv','v']:
    for scalepc in [8,16,32,64]:
        folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
        lst = []
        for file in folder.glob(f'*{scalepc}pc_associations.fits'):
            gal_name = file.stem.split('_')[0]
            tbl = Table(fits.getdata(file,ext=1))
            tbl.add_column(gal_name,name='gal_name',index=0)
            lst.append(tbl)
        assoc_tmp = vstack(lst)
        # missing = set(sample_table['name']) - set(np.unique(assoc_tmp['gal_name']))
        print(f'{HSTband:>3}, {scalepc:>2}pc: {len(lst)} galaxies, {len(assoc_tmp):>5} associations ({np.sum(assoc_tmp["1to1"]):>4} 1to1)')

In [ ]:
# choose which version of the association catalogue to use
version = 'v1p2'
HSTband = 'nuv'
scalepc = 32

### The nebulae catalogue

In [ ]:
# the original catalogue from Francesco
# Table.read treats the units correctly
nebulae = Table.read(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_v3.fits')
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra'],nebulae['cen_dec'],frame='icrs')
# we remove the old EW values
del nebulae[['EW_HA','EW_HA_ERR']]
nebulae.rename_columns(['cen_x','cen_y','cen_ra','cen_dec','region_area',
                          'EBV','EBV_ERR','SkyCoord'],
                         ['x_neb','y_neb','ra_neb','dec_neb','area_neb',
                          'EBV_balmer','EBV_balmer_err','SkyCoord_neb'])
    
# some additional properties 
for filename in ['dig','FUV_bkg','EW','density_refit','refitNII','in_frame','CO']:
    tbl = Table.read(basedir/'data'/'interim'/f'Nebulae_Catalogue_v2p1_{filename}.fits')
    nebulae = join(nebulae,tbl,keys=['gal_name','region_ID'],metadata_conflicts='silent')
nebulae['dig/hii'] = nebulae['dig_median'] / nebulae['hii_median']

# this will rais a few errors that we just ignore
with np.errstate(divide='ignore',invalid='ignore'):
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX']/nebulae['FUV_FLUX']
    nebulae['HA/FUV_err'] = nebulae['HA/FUV']*np.sqrt((nebulae['FUV_FLUX_ERR']/nebulae['FUV_FLUX'])**2+(nebulae['HA6562_FLUX_ERR']/nebulae['HA6562_FLUX'])**2)

# calculate luminosity based on distance from sample table
# we do not include the distance in the uncertainty as it should cancel out with the mass
nebulae = join(nebulae,sample_table_v1p6['gal_name','dist','dist_err'],keys='gal_name')
flux_to_lum = 1e-20 * 4*np.pi*(u.erg/u.cm**2/u.s * u.Mpc**2).to(u.erg/u.s)
nebulae['HA6562_LUM'] = nebulae['HA6562_FLUX']*flux_to_lum*nebulae['dist']**2
nebulae['HA6562_LUM_ERR'] = nebulae['HA6562_LUM'] * np.sqrt((nebulae['HA6562_FLUX_ERR']/nebulae['HA6562_FLUX'])**2)
nebulae['HA6562_LUM_CORR'] = nebulae['HA6562_FLUX_CORR']*flux_to_lum*nebulae['dist']**2
nebulae['HA6562_LUM_CORR_ERR'] = nebulae['HA6562_LUM_CORR'] * np.sqrt((nebulae['HA6562_FLUX_CORR_ERR']/nebulae['HA6562_FLUX_CORR'])**2)

# and the observed number of ionising photons
nebulae['Qobserved']     = 7.31e11*nebulae['HA6562_LUM_CORR']
nebulae['Qobserved_err'] = 7.31e11*nebulae['HA6562_LUM_CORR_ERR']
nebulae['Qobserved_uncorr']     = 7.31e11*nebulae['HA6562_LUM']
nebulae['Qobserved_uncorr_err'] = 7.31e11*nebulae['HA6562_LUM_ERR']

# the nebulae catalogue with additional information
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
lst = []
for file in folder.glob(f'*{scalepc}pc_nebulae.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    name=gal_name.lower()
    lst.append(tbl)
nebulae_tmp = vstack(lst)

nebulae = join(nebulae,nebulae_tmp,keys=['gal_name','region_ID'],join_type='outer')
nebulae = nebulae[nebulae['flag_star']==0]
nebulae = nebulae[nebulae['flag_edge']==0]
HIIregion_mask = (nebulae['BPT_NII']==0) & (nebulae['BPT_SII']==0) & (nebulae['BPT_OI']==0)

print(f'{len(nebulae)} nebulae in initial catalogue (all galaxies)')
nebulae = nebulae[HIIregion_mask]
print(f'we use {len(lst)} galaxies with {np.sum(~nebulae["overlap_neb"].mask & (nebulae["in_frame"]))} HII regions')
# only use HII regions and only the galaxies with associations
#nebulae = nebulae[HIIregion_mask & ~nebulae["overlap_neb"].mask]
print(f'{np.sum((nebulae["overlap_neb"]>0) & (nebulae["in_frame"]))} HII regions ({100*np.sum((nebulae["overlap_neb"]>0) & (nebulae["in_frame"]))/np.sum(nebulae["in_frame"]):.1f}%) overlap with association')

### The association catalogue

In [ ]:
# those files hold the merged association catalogues
with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
    associations = Table(hdul[1].data)
    
associations['SkyCoord'] = SkyCoord(associations['reg_ra']*u.degree,associations['reg_dec']*u.degree)
associations.rename_columns(['reg_ra','reg_dec','reg_x','reg_y',
                             'reg_dolflux_Age_MinChiSq','reg_dolflux_Mass_MinChiSq','reg_dolflux_Ebv_MinChiSq',
                             'reg_dolflux_Age_MinChiSq_err','reg_dolflux_Mass_MinChiSq_err','reg_dolflux_Ebv_MinChiSq_err',
                             'SkyCoord'],
                            ['ra_asc','dec_asc','x_asc','y_asc','age','mass',
                             'EBV_stars','age_err','mass_err','EBV_stars_err','SkyCoord_asc'])

with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}_in_frame.fits') as hdul:
    in_frame = Table(hdul[1].data)
associations = join(associations,in_frame,keys=['gal_name','assoc_ID'])

associations_SLUG = Table.read(basedir/'data'/'interim'/'associations_SLUG.fits')
associations = join(associations,associations_SLUG,keys=['gal_name','assoc_ID'])

# many associations have error=0. we set it to 1
associations['age_err'][associations['age_err']<1] = 1

# the association catalogue matched with the nebuale catalogue
folder = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'
lst = []
for file in folder.glob(f'*{scalepc}pc_associations.fits'):
    gal_name = file.stem.split('_')[0]
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
assoc_tmp = vstack(lst)
# combine both catalogues
associations = join(associations,assoc_tmp,keys=['gal_name','assoc_ID'])
print(f'{len(lst)} galaxies in sample ({np.sum(associations["in_frame"])} associations)')
print(f'{np.sum((associations["overlap_asc"]>0) & (associations["in_frame"]))} associations ({100*np.sum((associations["overlap_asc"]>0) & (associations["in_frame"]))/np.sum(associations["in_frame"]):.1f} %) overlap with an HII region')

#criteria = np.abs(associations['age']-associations['age_16'])>associations['age_err']
#criteria |= np.abs(associations['age']-associations['age_64'])>associations['age_err']
#associations['uniform_age'] = ~criteria

## The joint catalogues

### 1to1 sample

In [ ]:
# we can also recreate the table from the association an nebulae catalogues
catalogue = join(nebulae[~nebulae['assoc_ID'].mask],associations,keys=['gal_name','assoc_ID','region_ID'])

print(f'{len(catalogue)} objects in matched catalogue')

### The extended sample

some associations overlap with multiple HII regions. However we can only deal with those that overlap with only one HII regions (unless they are in an HII region complex, but this catalogue comes later).

In [ ]:
associations['one_to_multi'] = False

gal_name = None
for row in associations:
    if row['gal_name'] != gal_name:
        gal_name = row['gal_name']
        
        with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_associations.yml') as f:
            associations_dict = yaml.load(f,Loader=yaml.SafeLoader)

        with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_nebulae.yml') as f:
            nebulae_dict = yaml.load(f,Loader=yaml.SafeLoader)

        
    if len(associations_dict[row['assoc_ID']]) == 1:
        for assoc_ID in nebulae_dict[row['region_ID']]:
            if len(associations_dict[assoc_ID])!=1:
                break
        else:
            row['one_to_multi'] = True
    
extended = associations[associations['one_to_multi'] & ~associations['1to1']]

print(f'{len(extended)} associations in {len(np.unique(extended["gal_name","region_ID"]))} nebulae')

in the first version of the code, assocations that fall inside a nebulae with multiple assocations did not get assigned a `region_ID`. The following code fixes that (you only need to run it if you use a new resolution/band).

In [ ]:
for gal_name in sample_table_v1p6['name']:
    print(gal_name)
    with open(basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_associations.yml') as f:
        associations_dict = yaml.load(f,Loader=yaml.SafeLoader)
        
    filename = basedir/'data'/'map_nebulae_association'/version/HSTband/f'{scalepc}pc'/f'{gal_name}_{HSTband}_{scalepc}pc_associations.fits'
    assoc_tmp = Table.read(filename)
    print(f"{np.sum(~np.isnan(assoc_tmp['region_ID']))} region IDs in initial catalogue")
    
    assoc_tmp['region_ID'][assoc_tmp['Nnebulae']==1] = [associations_dict[k][0] for k in assoc_tmp[assoc_tmp['Nnebulae']==1]['assoc_ID']]
    print(f"{np.sum(~np.isnan(assoc_tmp['region_ID']))} region IDs in final catalogue")

    hdu = fits.BinTableHDU(assoc_tmp,name='joined catalogue')
    hdu.writeto(filename,overwrite=True)


### HII region complexes

The heavy lifting is done by the script `HII_region_complexes.py`. Here we just read in the six output files created by this script.

In [ ]:
with open(basedir/'data'/'interim'/f'nebulae_neighbors.yml') as f:
    nebulae_neighbors = yaml.load(f,Loader=yaml.SafeLoader)
with open(basedir/'data'/'interim'/f'complexes_nebulae.yml') as f:
    complexes_neb_dict = yaml.load(f,Loader=yaml.SafeLoader)
with open(basedir/'data'/'interim'/f'complexes_associations_{HSTband}_ws{scalepc}pc_{version}.yml') as f:
    complexes_assoc_dict = yaml.load(f,Loader=yaml.SafeLoader)
    
complexes = Table.read(basedir/'data'/'interim'/'complexes.fits')
complexes['SkyCoord'] = SkyCoord(complexes['ra']*u.deg,complexes['dec']*u.deg,frame='icrs')
# split string with region_IDs and assoc_IDs into lists
complexes['region_IDs'] = [list(map(int,row['region_IDs'].split(','))) for row in complexes]
complexes['assoc_IDs'] = [list(map(int,row['assoc_IDs'].split(','))) if row['assoc_IDs'] else [] for row in complexes]
complexes.add_column([len(row['region_IDs']) for row in complexes],index=4,name='N_neb')
complexes.add_column([len(row['assoc_IDs']) for row in complexes],index=5,name='N_assoc')

complexes_nebulae = Table.read(basedir/'data'/'interim'/'complexes_nebulae.fits')
complexes_associations = Table.read(basedir/'data'/'interim'/f'complexes_associations_{HSTband}_ws{scalepc}pc_{version}.fits')

print(f"{np.sum([len(x) for x in complexes['region_IDs']])} nebulae in {len(complexes)} complexes")

we filter the catalogues to only include clean HII region complexes and their associations

In [ ]:
complexes = complexes[complexes['isHII'] & complexes['multi_to_multi'] & (complexes['N_assoc']>0)]
complexes_associations = join(complexes_associations[~complexes_associations['complex_ID'].mask],associations,keys=['gal_name','assoc_ID'])
complexes_nebulae = join(complexes_nebulae[~complexes_nebulae['complex_ID'].mask],nebulae,keys=['gal_name','region_ID'])

print(f"{np.sum([len(x) for x in complexes['region_IDs']])} HII regions in {len(complexes)} complexes")

### The Cluster catalogue

read the existing Catalogue. Most HII regions overlap with multiple clusters. Therefore the matching between the two catalogues is done at a later stage.

In [ ]:
compact_clusters = Table.read(basedir/'data'/'interim'/f'compact_clusters_in_HIIregions.fits')
# we remove clusters that do not overlap
compact_clusters = compact_clusters[(compact_clusters['region_ID']>-1)]
compact_clusters['age_err'][compact_clusters['age_err']<1] = 1

print(f'{len(compact_clusters)} objects in cluster catalogue')

#### Code to match clusters to HII regions

if this file does not exist yet, create it with the following code

In [ ]:
from astrotools.regions import find_sky_region

def get_value(matrix, index, default_value=np.nan):
    '''
    The `to_pixel` method returns the x,y coordinates. However in the 
    image they correspond to img[y,x]
    '''
    result = np.zeros(len(index))+default_value
    mask = (index[:,1] < matrix.shape[0]) & (index[:,0] < matrix.shape[1])
    mask &= (index[:,1] >= 0) & (index[:,0] >=0)

    valid = index[mask]
    result[mask] = matrix[valid[:,1], valid[:,0]]
    return result

compact_clusters_list = []
for gal_name in np.unique(sample_table['name']):

    filename = data_ext / 'MUSE' / 'DR2.1' / 'copt' / 'MUSEDAP'
    filename = [x for x in filename.iterdir() if x.stem.startswith(gal_name)][0]
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))    

    reg_muse_pix, reg_muse_sky = find_sky_region(Halpha.mask.astype(int),wcs=Halpha.wcs)
    
    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
    with fits.open(filename) as hdul:
        nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))    
    
    # read the compact cluster catalogues
    filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'
    if not filename.is_file():
        print(f'no compact clusters for {gal_name}')
    else:     
        compact_clusters = Table.read(filename)
        compact_clusters['SkyCoord'] = SkyCoord(compact_clusters['PHANGS_RA']*u.deg,compact_clusters['PHANGS_DEC']*u.deg)
        compact_clusters.add_index('ID_PHANGS_CLUSTERS')
        compact_clusters['in_frame'] = reg_muse_sky.contains(compact_clusters['SkyCoord'],Halpha.wcs)
        compact_clusters['region_ID'] = get_value(nebulae_mask.data,np.array(compact_clusters['SkyCoord'].to_pixel(nebulae_mask.wcs)).T.astype(int))

        # we save all objects that fall inside one of our empty HII regions
        compact_clusters.add_column(gal_name,index=0,name='gal_name')
        del compact_clusters['SkyCoord']
        compact_clusters_list.append(compact_clusters)

compact_clusters = vstack(compact_clusters_list)
compact_clusters.rename_columns(['ID_PHANGS_CLUSTERS','PHANGS_CLUSTER_CLASS_HUMAN','PHANGS_CLUSTER_CLASS_ML_VGG','PHANGS_AGE_MINCHISQ','PHANGS_AGE_MINCHISQ_ERR','PHANGS_MASS_MINCHISQ','PHANGS_MASS_MINCHISQ_ERR','PHANGS_EBV_MINCHISQ','PHANGS_EBV_MINCHISQ_ERR'],
                                ['cluster_ID','class_human','class_ml','age','age_err','mass','mass_err','EBV_stellar','EBV_stellar_err'])
print(f'{len(compact_clusters)} compact clusters in final catalogue')

primary_hdu = fits.PrimaryHDU()
table_hdu   = fits.BinTableHDU(compact_clusters)
hdul = fits.HDUList([primary_hdu, table_hdu])
hdul.writeto(basedir/'data'/'interim'/f'compact_clusters_in_HIIregions.fits',overwrite=True)

In [ ]:
s = 0
for gal_name in np.unique(compact_clusters['gal_name']):
    sub = compact_clusters[compact_clusters['gal_name']==gal_name]
    s+=len(np.unique(sub['region_ID']))
print(s)

In [ ]:
# how many clusters and HII regions have a 1to1 relation
one_to_one = 0
for gal_name in sample_table_v1p6['name']:
    _,counts = np.unique(compact_clusters[compact_clusters['gal_name']==gal_name]['region_ID'],return_counts=True)
    one_to_one += np.sum(counts==1)
print(f'{one_to_one} clusters with 1to1 relation')

### Table to showcase the sample

In [ ]:
latexdict = {'tabletype': 'table',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'PHANGS sample',
'preamble': '\\centering',
'col_align':'lrlrrr',
'tablefoot': f'\\label{{tbl:sample}}'
            }

sample = sample_table_v1p6[['name','dist','dist_unc']].copy()

Nasc = []
Nneb = []
Nclu = []

for gal_name in sample['name']:

    Nneb.append(np.sum((nebulae["gal_name"]==gal_name) & (nebulae['in_frame'])))    
    Nasc.append(np.sum((associations["gal_name"]==gal_name) & (associations['in_frame'])))
    Nclu.append(np.sum((compact_clusters["gal_name"]==gal_name) & (compact_clusters['in_frame'])))

sample['Nneb'] = Nneb
sample['Nasc'] = Nasc
sample['Nclu'] = Nclu

sample['dist_unc'].info.format = '%.2f'

ascii.write(sample,sys.stdout, Writer = ascii.Latex,latexdict=latexdict)

In [ ]:
astrosat_sample = np.unique(nebulae[~np.isnan(nebulae['FUV_FLUX'])]['gal_name'])
muse_sample     = sample_table['name']
hst_sample      = associations['gal_name']
sitelle_sample  = ['NGC0628','NGC2835','NGC3351','NGC3627','NGC4535']

t = Table({
    'name':muse_sample,
    'MUSE':np.isin(muse_sample,muse_sample),
    'HST':np.isin(muse_sample,hst_sample),
    'Astrosat':np.isin(muse_sample,astrosat_sample),
    'Sitelle':np.isin(muse_sample,sitelle_sample)}
     )

for col in t.columns[1:]:
    t[col] = ['\checkmark' if x else '' for x in t[col] ]
ascii.write(t,sys.stdout, Writer = ascii.Latex)

sample table for paper

In [ ]:
latexdict = {'tabletype': 'table*',
'header_start': '\\toprule',
'header_end': '\\midrule',
'data_end': '\\bottomrule',
'caption': f'Galaxy sample',
'units': {'R.A.':'(J2000)','Dec.':'(J2000)','$i$':'deg','PA':'deg','Distance':'$\si{\mega\parsec}$',
          'r25':'arcmin'},
'preamble': '\\centering',
'tablefoot': f'\\label{{tbl:sample}}'
            }

tbl = sample_table[['name','Type','R.A.','Dec.','Inclination','posang','r25','mass','SFR','PSF','(m-M)']]

tbl['R.A.'] = [row.replace('h','x').replace('m','y').replace('.','z').replace('s','') for row in tbl['R.A.']]
tbl['R.A.'] = [row.replace('x','$^\mathrm{h}$').replace('y','$^\mathrm{m}$').replace('z','$^\mathrm{s}\kern -3pt.$') for row in tbl['R.A.']]

tbl['Dec.'] = [row.replace('d','x').replace('m','y').replace('.','z').replace('s','') for row in tbl['Dec.']]
tbl['Dec.'] = [row.replace('-','$-$').replace('+','$+$').replace('x','$^\mathrm{d}$').replace('y','$^\mathrm{m}$').replace('z','$^\mathrm{s}\kern -3pt.$') for row in tbl['Dec.']]
tbl.add_column(Distance(distmod=tbl['(m-M)']).to(u.Mpc),index=4,name='Distance')
tbl['Distance'].info.format = '%.2f' 
tbl['Nneb'] = [np.sum((nebulae['gal_name']==name) & (nebulae['in_frame'])) for name in tbl['name']]
tbl['Nasc'] = [np.sum((associations['gal_name']==name) & (associations['in_frame'])) for name in tbl['name']]

#tbl['name'] = [f'\\galaxyname{{{row["name"][:-4]}}}{{{row["name"][-4:]}}}' for row in tbl]
tbl.rename_columns(['Inclination','posang','r25'],['$i$','PA','$r_{25}$'])
#tbl = join(tbl,t,keys='name')

#with open(basedir / 'data' / 'interim' /'sample.tex','w',newline='\n') as f:
#    ascii.write(tbl,f,Writer=ascii.Latex, latexdict=latexdict,overwrite=True,exclude_names=['(m-M)','Sitelle'])
ascii.write(tbl,sys.stdout, Writer = ascii.Latex,latexdict=latexdict,exclude_names=[])

### Plot the sample (cutouts)

plot all objects in the merged catalogues

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from cluster.plot import single_cutout


criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
criteria &= (catalogue['overlap_neb']>0.1) 
criteria &= (catalogue['overlap_asc']==1) 
criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria]

print(f'{len(tmp)} objects in sample')

size=10*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'cutouts_{HSTband}_{scalepc}pc'
    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                # nebulae mask
                filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')

            
            ax = next(axes_iter)
            ax = single_cutout(ax,
                             position = row['SkyCoord_neb'],
                             image = F275,
                             mask1 = nebulae_mask,
                             mask2 = associations_mask,
                             label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}",
                             size  = 4*u.arcsecond)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

three color composit with CO emission

In [ ]:
from cluster.io import read_associations
from matplotlib.backends.backend_pdf import PdfPages
from reproject import reproject_interp
from skimage.measure import find_contours
from pnlf.plot import create_RGB


criteria = (catalogue['mass']>1e4) 
criteria &= (catalogue['overlap_asc'] == 1)
criteria &= (catalogue['age'] < 10)
#tmp = catalogue[criteria]

print(f'{len(tmp)} objects')

size=5*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/f'cutouts_rgb_{HSTband}_{scalepc}pc'

width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for j in range(Npages):
        print(f'working on page {j+1} of {Npages}')

        sub_sample = tmp[j*Npage:(j+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                print(f'reading files for {gal_name}')
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                filename = data_ext/'MUSE'/'DR2.1'/'MUSEDAP'/ f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
    
                # nebulae mask
                filename = data_ext/'Products'/ 'Nebulae catalogue'/'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'HST'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')
            
                with fits.open(data_ext/'ALMA'/'v4p0'/f'{gal_name.lower()}_12m+7m+tp_co21_broad_tpeak.fits') as hdul:
                    CO = NDData(data=hdul[0].data,
                                meta=hdul[0].header,
                                wcs=WCS(hdul[0].header))
            
            ax = next(axes_iter)
            
            position = row['SkyCoord_neb']
            
            label = f"{row['gal_name']}: {row['region_ID']:.0f}/{row['assoc_ID']:.0f}"

            cutout_F275 = Cutout2D(F275.data,position,size=size,wcs=F275.wcs)
            norm = simple_norm(cutout_F275.data,stretch='linear',clip=False,percent=99.9)

            cutout_CO, _  = reproject_interp(CO,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    
            cutout_Halpha, _  = reproject_interp(Halpha,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape)    

            rgb = create_RGB(cutout_CO,cutout_Halpha,cutout_F275.data,
                             percentile=[98,98,99.8],weights=[0.7,0.6,1])
            #ax.imshow(cutout_image.data,origin='lower',norm=norm,cmap=plt.cm.gray_r)
            ax.imshow(rgb,origin='lower')

            # plot the nebulae catalogue
            cutout_mask, _  = reproject_interp(nebulae_mask,output_projection=cutout_F275.wcs,shape_out=cutout_F275.shape,order='nearest-neighbor')    
            region_ID = np.unique(cutout_mask[~np.isnan(cutout_mask)])

            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_mask)
                blank_mask[cutout_mask==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='tab:green',lw=0.8,label='HII-region')

            # 32 pc
            cutout_32 = Cutout2D(associations_mask.data,position,size=size,wcs=associations_mask.wcs)
            region_ID = np.unique(cutout_32.data[~np.isnan(cutout_32.data)])
            contours = []
            for i in region_ID:
                blank_mask = np.zeros_like(cutout_32.data)
                blank_mask[cutout_32.data==i] = 1
                contours += find_contours(blank_mask, 0.5)
            for coords in contours:
                ax.plot(coords[:,1],coords[:,0],color='blue',lw=0.8,label='32pc assoc.')
            t = ax.text(0.06,0.87,label, transform=ax.transAxes,color='black',fontsize=8)
            t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
            ax.set_xticks([])
            ax.set_yticks([])
            
        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if j == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()

## Compute number of ionising photons

the available models are GENEVASTD, GENEVAHIGH, PADOVASTD, PADOVAAGB, GENEVAv00, GENEVAv40

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster = Cluster(stellar_model='GENEVAv00',metallicity=0.014)

# model age in Myr and flux in s-1 per Msun
model_age  = cluster.quanta['Time'].value/1e6    
model_flux = cluster.quanta['HI_rate'].value / cluster.mass

for each association, calculate the predicted number of ionizing photons based on the mass and the age of the association.

The observed number of ionizing photos is calulcated based on the conversion factor from Niederhofer+2016 (or Kennicutt+98)

$$
Q(\mathrm{H}^0) = 7.31\cdot 10^{11} L(\mathrm{H}\alpha)
$$

Starburst99 uses a factor of 7.354+-0.008 (no Idea why it changes over time) to convert between quanta (number of ionizing photons) and ewidth (Halpha luminosity).

we only use a subsample (high mass, contained, speparated etc.)

### Age to photon flux

In [ ]:
from cluster.escape_fractions import plot_ionising_photon_flux

HI_rate = cluster.quanta['HI_rate'].value
time = cluster.quanta['Time'].value/1e6
    
plot_ionising_photon_flux(2,1,time,HI_rate,sample_size=1000)


In [ ]:
from cluster.escape_fractions import ionising_photon_flux

# model age in Myr and flux in s-1 per Msun
model_age  = cluster.quanta['Time'].value/1e6    
model_flux = cluster.quanta['HI_rate'].value / cluster.mass


fig,ax=plt.subplots(figsize=(thesis_width,thesis_width/1.618))

for age_err in [1,2,4,8]:
    
    flux, flux_err = ionising_photon_flux(
                                     age = np.arange(1,11),
                                     age_err = age_err,
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=1000
                                     )
    ax.plot(np.arange(1,11),flux/flux_err,label=f'$\Delta \mathrm{{age}} = {age_err}$')

ax.legend()
ax.set(xlabel='age / Myr',yscale='linear')
plt.show()


In [ ]:
from cluster.escape_fractions import ionising_photon_flux

# model age in Myr and flux in s-1 per Msun
model_age  = cluster.quanta['Time'].value/1e6    
model_flux = cluster.quanta['HI_rate'].value / cluster.mass


fig,ax=plt.subplots(figsize=(thesis_width,thesis_width/1.618))


flux, flux_err = ionising_photon_flux(
                                 age = np.arange(1,11,0.1),
                                 age_err = np.arange(1,11,0.1),
                                 model_age = model_age,
                                 model_flux = model_flux,
                                 sample_size=1000
                                 )
ax.plot(np.arange(1,11,0.1),flux/flux_err,label=f'$\Delta \mathrm{{age}} = {age_err}$')

ax.set(xlabel='age / Myr',ylabel='S/N flux',yscale='linear')
plt.show()


the ages are often integers (and so are the errors). Therefore we compute the flux and uncertainties multiple times. To save time, we can compute them for all unique combinations of age and error and then just look up the result.

In [ ]:
sample = extended[extended['mass']>1e4]
unique_age = np.unique(sample[['age','age_err']].as_array(),axis=0)

### For the 1to1 sample

we compute the ionising photon flux and the escape fraction

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

flux, flux_err = ionising_photon_flux(
                                     age = catalogue['age'],
                                     age_err = catalogue['age_err'],
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=100
                                     )

Qpredicted     = flux*catalogue['mass']
Qpredicted_err = Qpredicted * np.sqrt((flux_err/flux)**2+(catalogue['mass_err']/catalogue['mass'])**2)

catalogue['Qpredicted']     = Qpredicted
catalogue['Qpredicted_err'] = Qpredicted_err

calculate escape fraction

In [ ]:
from cluster.escape_fractions import escape_fraction


fesc, fesc_err = escape_fraction(catalogue['Qpredicted'],catalogue['Qpredicted_err'],catalogue['Qobserved'],catalogue['Qobserved_err'],stats=True)

catalogue['fesc'] = fesc
catalogue['fesc_err'] = fesc_err
catalogue['robust'] = (catalogue['mass']>1e4) & (catalogue['overlap']=='contained') & (catalogue['age']<=8)

print('for the robust sample')
tmp = catalogue[catalogue['robust']]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)


look at different sub-samples

In [ ]:
from cluster.escape_fractions import plot_escape_fraction

print('contained')
tmp = catalogue[(catalogue['overlap']=='contained') & (catalogue['mass']>1e4) & (catalogue['age']<=8)]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)
plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'])
plt.show()

print('partial')
tmp = catalogue[(catalogue['overlap']=='partial') & (catalogue['mass']>1e4) & (catalogue['age']<=8)]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)
plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'])
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,0.8*single_column))

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

Qpredicted_line = np.logspace(48,52)
lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved_line = Qpredicted_line*(1-f)
    ax.plot(np.log10(Qpredicted_line),np.log10(Qobserved_line),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)
ax.fill_between(np.log10(Qpredicted_line),4*np.log10(Qpredicted_line),np.log10(Qpredicted_line),color='0.7',alpha=0.1)

for gal_name in sample_table_v1p6['gal_name']:
    
    tmp = catalogue[catalogue['robust'] & (catalogue['gal_name']==gal_name)]
    color = cmap(norm(np.log10(sample_table_v1p6.loc[gal_name]['props_mstar'])))
    
    sc=ax.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),s=5,
                  color=color,cmap=cmap,zorder=1,
                  rasterized=True)
    
    tmp = tmp[(tmp['Qpredicted']+tmp['Qpredicted_err']>tmp['Qobserved']) & (tmp['age']<=2)]
    ax.errorbar(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
             xerr=tmp['Qpredicted_err']/tmp['Qpredicted']/np.log(10),
             yerr=tmp['Qobserved_err']/tmp['Qobserved']/np.log(10),
             color=color,fmt='o',ms=0.1,zorder=0)

    
ax.set(xlabel=r'$\log_{10} (Q (\mathrm{H}^0)\,/\,\mathrm{s}^{-1})$ predicted',
       ylabel=r'$\log_{10} (Q_{\mathrm{H}\,\alpha}\,/\,\mathrm{s}^{-1})$ observed',
       xlim=[49,52],ylim=[49,52])

ax.set_xticks([49,50,51,52])
ax.set_yticks([49,50,51,52])


fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.8))


ax.legend(handlelength=2,loc=2)
ax.set_title(r'\texttt{robust sample}')
plt.show()

### For the Extended sample

thus far we ignored HII regions with multiple associations. Here we also include them

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

flux, flux_err = ionising_photon_flux(
                                     age = extended['age'],
                                     age_err = extended['age_err'],
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=100
                                     )

Qpredicted     = flux*extended['mass']
Qpredicted_err = Qpredicted * np.sqrt((flux_err/flux)**2+(extended['mass_err']/extended['mass'])**2)
extended['Qpredicted']     = Qpredicted
extended['Qpredicted_err'] = Qpredicted_err

# a few additional flags we need to clean the sample
extended['robust'] = (extended['mass']>1e4) & (extended['overlap']=='contained') & (extended['age']<=8)
extended['young_and_massive'] = (extended['mass']>1e4) & (extended['age']<=8)
extended['young_and_massive_contained'] = extended['young_and_massive'] & (extended['overlap']=='contained')
extended['young_and_massive_partial'] = extended['young_and_massive'] & (extended['overlap']=='partial')

we use the `group_by` function to combine multiple associations inside on HII region

In [ ]:
def error_prop(array):
    return np.sqrt(np.sum(array**2))
def mass_cut(array):
    return np.all(array>1e4)
def age_cut(array):
    return np.all(array<=8)
def overlap_cut(array):
    return np.all(array=='contained')

groups = extended.group_by(['gal_name','region_ID'])
temp = groups.groups.keys

# sum up predicted ionising photon flux
temp['Qpredicted'] = groups['Qpredicted'].groups.aggregate(np.sum)
temp['Qpredicted_err'] = groups['Qpredicted_err'].groups.aggregate(error_prop)

# a few additional properties
temp['Ncluster'] = groups['gal_name'].groups.aggregate(len)
temp['mass'] = groups['mass'].groups.aggregate(np.sum)
temp['mass_cut'] = groups['mass'].groups.aggregate(mass_cut)
temp['overlap_cut'] = groups['overlap'].groups.aggregate(overlap_cut)
temp['age_cut'] = groups['age'].groups.aggregate(age_cut)
temp['ymc'] = groups['young_and_massive_contained'].groups.aggregate(np.any)
temp['ymp'] = groups['young_and_massive_partial'].groups.aggregate(np.any)

# finally we merge with the nebulae catalogue
nebulae_extended = join(nebulae,temp,keys=['gal_name','region_ID'])
nebulae_extended['robust'] = nebulae_extended['ymc'] & ~nebulae_extended['ymp']

print(f'{len(nebulae_extended)} HII regions in extended catalogue')
N_extended = np.sum(nebulae_extended['robust']) 
print(f'{N_extended} objects in robust extended catalogue')

In [ ]:
from cluster.escape_fractions import plot_escape_fraction

fesc, fesc_err = escape_fraction(nebulae_extended['Qpredicted'],nebulae_extended['Qpredicted_err'],nebulae_extended['Qobserved'],nebulae_extended['Qobserved_err'],stats=True)
plot_escape_fraction(nebulae_extended['Qpredicted'],nebulae_extended['Qpredicted_err'],nebulae_extended['Qobserved'],nebulae_extended['Qobserved_err'])

nebulae_extended['fesc'] = fesc
nebulae_extended['fesc_err'] = fesc_err

print('for the robust sample')
tmp = nebulae_extended[nebulae_extended['robust']]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)


### For the HII region complexes

for the complexes things are a lot more difficult. 

1. We first compute the ionising photon flux of the individual associations
2. We then sum up all HII regions inside each complex
3. We match the associations to their complex

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

flux, flux_err = ionising_photon_flux(
                                     age = complexes_associations['age'],
                                     age_err = complexes_associations['age_err'],
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=100
                                     )
Qpredicted     = flux*complexes_associations['mass']
Qpredicted_err = Qpredicted * np.sqrt((flux_err/flux)**2+(complexes_associations['mass_err']/complexes_associations['mass'])**2)
complexes_associations['Qpredicted']     = Qpredicted
complexes_associations['Qpredicted_err'] = Qpredicted_err
complexes_associations['young_and_massive'] = (complexes_associations['mass']>1e4) & (complexes_associations['age']<=8)

first we group the nebulae by their `complex_ID`

In [ ]:
def error_prop(array):
    return np.sqrt(np.sum(array**2))
def most_common_value(array):
    v,count = np.unique(array,return_counts=True)
    return v[np.argmax(count)]

groups = complexes_nebulae.group_by(['gal_name','complex_ID'])
temp = groups.groups.keys
temp['N_HII']   = groups['gal_name'].groups.aggregate(len)
temp['env']   = groups['env_neb'].groups.aggregate(most_common_value)

for col in ['HA6562_LUM','HA6562_LUM_CORR','Qobserved','Qobserved_uncorr']:
    temp[col] = groups[col].groups.aggregate(np.sum) 
for col in ['HA6562_LUM_ERR','HA6562_LUM_CORR_ERR','Qobserved_err','Qobserved_uncorr_err']:
    temp[col] = groups[col].groups.aggregate(error_prop)    

complexes_sample = join(complexes,temp,keys=['gal_name','complex_ID'])

and next the associations

In [ ]:
complexes_associations['young_and_massive'] = (complexes_associations['mass']>1e4) & (complexes_associations['age']<=8)
complexes_associations['young_and_massive_contained'] = complexes_associations['young_and_massive'] & (complexes_associations['overlap']=='contained')
complexes_associations['young_and_massive_partial'] = complexes_associations['young_and_massive'] & (complexes_associations['overlap']=='partial')

groups = complexes_associations.group_by(['gal_name','complex_ID'])
temp = groups.groups.keys
temp['N_assoc']   = groups['gal_name'].groups.aggregate(len)
temp['ymc'] = groups['young_and_massive_contained'].groups.aggregate(np.any)
temp['ymp'] = groups['young_and_massive_partial'].groups.aggregate(np.any)

temp['Qpredicted'] = groups['Qpredicted'].groups.aggregate(np.sum)
temp['Qpredicted_err'] = groups['Qpredicted_err'].groups.aggregate(error_prop)
 
complexes_sample = join(complexes_sample,temp,keys=['gal_name','complex_ID'])
complexes_sample['robust'] = complexes_sample['ymc'] & ~complexes_sample['ymp']


In [ ]:
from cluster.escape_fractions import escape_fraction

fesc, fesc_err = escape_fraction(complexes_sample['Qpredicted'],complexes_sample['Qpredicted_err'],complexes_sample['Qobserved'],complexes_sample['Qobserved_err'],stats=True)

complexes_sample['fesc'] = fesc
complexes_sample['fesc_err'] = fesc_err

In [ ]:
for env in np.unique(complexes_sample['env']):
    print(f"{env}: {np.sum(complexes_sample['env']==env)}")

### For the compact clusters

for each cluster that overlaps with an HII region, we compute the ionising photon flux

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

flux, flux_err = ionising_photon_flux(
                                     age = compact_clusters['age'],
                                     age_err = compact_clusters['age_err'],
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=100
                                     )
Qpredicted     = flux*compact_clusters['mass']
Qpredicted_err = Qpredicted * np.sqrt((flux_err/flux)**2+(compact_clusters['mass_err']/compact_clusters['mass'])**2)
compact_clusters['Qpredicted']     = Qpredicted
compact_clusters['Qpredicted_err'] = Qpredicted_err
compact_clusters['young_and_massive'] = (compact_clusters['mass']>1e4) & (compact_clusters['age']<=8)


next we use `group_by` to sum the flux of all clusters inside the same HII region and join this catalogue with the HII region catalogue

In [ ]:
def error_prop(array):
    return np.sqrt(np.sum(array**2))
def mass_cut(array):
    return np.all(array>1e4)
def age_cut(array):
    return np.all(array<=8)

groups = compact_clusters.group_by(['gal_name','region_ID'])
temp = groups.groups.keys

temp['Qpredicted'] = groups['Qpredicted'].groups.aggregate(np.sum)
temp['Qpredicted_err'] = groups['Qpredicted_err'].groups.aggregate(error_prop)
temp['Ncluster'] = groups['gal_name'].groups.aggregate(len)
temp['mass'] = groups['mass'].groups.aggregate(np.sum)
temp['mass_cut'] = groups['mass'].groups.aggregate(mass_cut)
temp['age_cut'] = groups['age'].groups.aggregate(age_cut)
temp['young_and_massive'] = groups['young_and_massive'].groups.aggregate(np.any)

nebulae_cluster = join(nebulae,temp,keys=['gal_name','region_ID'])
nebulae_cluster['robust'] = nebulae_cluster['young_and_massive']

print(f'{len(nebulae_cluster)} HII regions in cluster catalogue')
N_cluster = np.sum(nebulae_cluster['robust']) 
print(f'{N_cluster} objects in robust cluster catalogue')

then we can compute the escape fraction in this new catalogue

In [ ]:
fesc, fesc_err = escape_fraction(nebulae_cluster['Qpredicted'],nebulae_cluster['Qpredicted_err'],nebulae_cluster['Qobserved'],nebulae_cluster['Qobserved_err'],stats=True)

nebulae_cluster['fesc'] = fesc
nebulae_cluster['fesc_err'] = fesc_err

print('for the robust sample')
tmp = nebulae_cluster[nebulae_cluster['robust']]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)

## Compare ionized photon flux

### For different samples

In [ ]:
from cluster.escape_fractions import compare_ionising_photons

fig,(ax1,ax2,ax4,ax3) = plt.subplots(figsize=(1.1*two_column,two_column/3.),ncols=4,nrows=1,sharex=True,sharey=True)

tmp = catalogue[~catalogue['robust']]
ax1 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax1,color='0.7',s=0.5,zorder=0)
tmp = catalogue[catalogue['robust']]
ax1 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax1,plot_lines=False,color=tab10[0],s=0.5)
ax1.legend(handlelength=2,loc=2,fontsize=6)
ax1.set_aspect('equal', adjustable='box')
ax1.set_title(r'\texttt{1to1 sample}')

tmp = nebulae_extended[~nebulae_extended['robust']]
ax2 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax2,color='0.7',s=0.5,zorder=0)
tmp = nebulae_extended[nebulae_extended['robust']]
ax2 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax2,plot_lines=False,color=tab10[0],s=0.5)
ax2.set(ylabel='')
ax2.set_aspect('equal', adjustable='box')
ax2.set_title(r'\texttt{extended sample}')

tmp = complexes_sample[~complexes_sample['robust']]
ax4 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax4,color='0.7',s=0.5,zorder=0)
tmp = complexes_sample[complexes_sample['robust']]
ax4 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax4,plot_lines=False,color=tab10[0],s=0.5)
ax4.set(ylabel='')
ax4.set_aspect('equal', adjustable='box')
ax4.set_title(r'\texttt{complexes sample}')


tmp = nebulae_cluster[~nebulae_cluster['robust']]
ax3 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax3,color='0.7',s=0.5,zorder=0)
tmp = nebulae_cluster[nebulae_cluster['robust']]
ax3 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],ax=ax3,plot_lines=False,color=tab10[0],s=0.5)
ax3.set(ylabel='')
ax3.set_aspect('equal', adjustable='box')
ax3.set_title(r'\texttt{cluster sample}')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'ionising_photons_samples.pdf',dpi=600)
plt.show()

### For different Population synthesis models

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models
from cluster.io import read_bpass

GENEVAv00 = Cluster(stellar_model='GENEVAv00',metallicity=0.014)
GENEVAv40 = Cluster(stellar_model='GENEVAv40',metallicity=0.014)
bpass = read_bpass(basedir/'..'/'BPASS',metallicity='z020')
bc03 = Table.read(basedir/'data'/'external'/'bc03_q.fits')

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

sample = catalogue

flux_v00, flux_err_v00 = ionising_photon_flux(
                                     age = sample['age'],
                                     age_err = sample['age_err'],
                                     model_age = GENEVAv00.quanta['Time'].value/1e6   ,
                                     model_flux = GENEVAv00.quanta['HI_rate'].value / GENEVAv00.mass,
                                     sample_size=100
                                     )

flux_v40, flux_err_v40 = ionising_photon_flux(
                                     age = sample['age'],
                                     age_err = sample['age_err'],
                                     model_age = GENEVAv40.quanta['Time'].value/1e6   ,
                                     model_flux = GENEVAv40.quanta['HI_rate'].value / GENEVAv40.mass,
                                     sample_size=100
                                     )

flux_BPASS, flux_err_BPASS = ionising_photon_flux(
                                     age = sample['age'],
                                     age_err = sample['age_err'],
                                     model_age = bpass['age'].value/1e6,
                                     model_flux = bpass['Q'].value / 1e6, # the BPASS cluster start with 1e6 Msun
                                     sample_size=100
                                     )

flux_bc03, flux_err_bc03 = ionising_photon_flux(
                                     age = sample['age'],
                                     age_err = sample['age_err'],
                                     model_age = bc03['sfh.age'],
                                     model_flux = bc03['stellar.n_ly_young']/bc03['stellar.m_star'],
                                     sample_size=100
                                     )

Qpredicted_v00     = flux_v00*sample['mass']
Qpredicted_v00_err = flux_err_v00*sample['mass']
Qpredicted_v40     = flux_v40*sample['mass']
Qpredicted_v40_err = flux_err_v40*sample['mass']
Qpredicted_BPASS     = flux_BPASS*sample['mass']
Qpredicted_BPASS_err = flux_err_BPASS*sample['mass']
Qpredicted_bc03     = flux_bc03*sample['mass']
Qpredicted_bc03_err = flux_err_bc03*sample['mass']
Qobserved = 7.31e11*sample['HA6562_LUM_CORR']
Qobserved_err = 7.31e11*sample['HA6562_LUM_CORR_ERR']

In [ ]:
fig,ax=plt.subplots()

ax.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value / GENEVAv00.mass,label='SB99')
ax.plot(bpass['age'].value/1e6,bpass['Q'].value / 1e6,label='BPASS')
ax.plot(bc03['sfh.age'],bc03['stellar.n_ly_young']/bc03['stellar.m_star'],label='CIGALE')
ax.legend()
ax.set(xlim=[0,10])

plt.show()

In [ ]:
from cluster.escape_fractions import compare_ionising_photons

fig,(ax1,ax2,ax3) = plt.subplots(figsize=(two_column,two_column/2.7),ncols=3,sharey=True)

ax1 = compare_ionising_photons(Qpredicted_v00,Qobserved,ax=ax1,color='0.7',s=0.5,zorder=0)
ax1.set_title(r'\texttt{GENEVAv00}')

ax2 = compare_ionising_photons(Qpredicted_v40,Qobserved,ax=ax2,color='0.7',s=0.5,zorder=0)
ax2.set(ylabel='')
ax2.set_title(r'\texttt{GENEVAv40}')

ax3 = compare_ionising_photons(Qpredicted_BPASS,Qobserved,ax=ax3,color='0.7',s=0.5,zorder=0)
ax3.set(ylabel='')
ax3.set_title(r'\texttt{BPASS}')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'escape_fraction_samples.pdf',dpi=600)
plt.show()

#### Corner

In [ ]:
tbl = Table({'v00':Qpredicted_v00,'v00_err':Qpredicted_v00_err,
             'v40':Qpredicted_v40,'v40_err':Qpredicted_v40_err,
             'BPASS':Qpredicted_BPASS,'BPASS_err':Qpredicted_BPASS_err,
             'CIGALE':Qpredicted_bc03,'CIGALE_err':Qpredicted_bc03_err,
             'SLUG':catalogue['q_SLUG'],'SLUG_err':catalogue['q_SLUG_err_plus'],
            })

for col in ['v00','v40','BPASS','CIGALE','SLUG']:
    tbl[col][tbl[col]/tbl[f'{col}_err']<1] = np.nan

tbl['mass'] = catalogue['mass']
tbl['age'] = catalogue['age']

#tbl['SLUG'][tbl['SLUG']/tbl['SLUG_err']<1] = np.nan

In [ ]:
from astrotools.plot.utils import fix_aspect_ratio
from scipy.interpolate import interpn

def corner(table,columns,mask,labels={},filename=None,figsize=10,aspect_ratio=1,**kwargs):
        
    lim = [1e46,1e52]
    # create a figure with the correct proportions
    nrows, ncols = len(columns)-1, len(columns)-1

    fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(figsize,figsize*aspect_ratio*ncols/nrows))
    for i,row in enumerate(columns[1:]):
        for j,col in enumerate(columns[:-1]):
            ax=axes[i,j]
            if j>i:
                ax.remove()
            else:
                ax.set(xscale='log',yscale='log',xlim=lim,ylim=lim)
                ax.set_xticks([1e47,1e49,1e51])
                ax.set_yticks([1e47,1e49,1e51])

                if j==0:
                    ax.set_ylabel(labels.get(row,row.replace("_","")))
                else:
                    ax.set_yticklabels([])
                if i==len(columns)-2:
                    ax.set_xlabel(labels.get(col,col.replace("_","")))
                else:
                    ax.set_xticklabels([])

                x,y = table[col][mask],table[row][mask]
                ax.scatter(x,y,s=0.5,color='0.7',rasterized=True,**kwargs)
                
                x,y = table[col][~mask],table[row][~mask]    
                ax.scatter(x,y,s=0.5,color=tab10[0],rasterized=True,**kwargs)
                #hist, x_e, y_e = np.histogram2d(x,y,bins=np.logspace(45,52,10),density=True)
                #z = interpn((0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ),hist,np.vstack([x,y]).T,method="linear",bounds_error=False)
                #vmin,vmax=np.nanpercentile(z,[15,95])
                #sc=ax.scatter(x,y,c=z,vmin=vmin,vmax=vmax,s=0.5,cmap=plt.cm.Reds)
                ax.plot(lim,lim,color='black')
                
            fix_aspect_ratio(ax,aspect_ratio=aspect_ratio)
    
    ax = fig.add_axes([0.6, 0.6, 0.2, 0.2])
    ax.axis('off')
    ax.text(0.,0.,r'$Q (\mathrm{H}^0)\,/\,\mathrm{s}^{-1}$')
    plt.subplots_adjust(wspace=0.12, hspace=0.12)
    
    if filename:
        plt.savefig(filename,dpi=600)

    return fig


columns  = ['v00','v40','BPASS','CIGALE','SLUG']
labels = {'v00': 'SB99 v00', 'v40': 'SB99 v40', 'BPASS': 'BPASS'}
filename = basedir/'reports'/f'corner_ionising_photons.pdf'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tbl,columns,labels=labels,mask=(tbl['mass']<0),
           filename=filename,figsize=1.2*single_column)
plt.show()

In [ ]:
mask = np.zeros(len(associations),dtype=bool)

for filt in ['NUV','U','B','I','V']:
    mask |= (associations[f'{filt}_dolmag_vega']==-999)

In [ ]:
associations[mask & (associations['q_SLUG']>associations['q_SLUG_err_plus'])]

#### SLUG

In [ ]:
fig,ax=plt.subplots()

ax.hist(catalogue['age'],bins=np.linspace(0.5,40.5,40),histtype='step',label='cigale')
ax.hist(catalogue['age_SLUG'],bins=np.linspace(0.5,40.5,40),histtype='step',label='SLUG')
ax.legend()
ax.set(xlabel='age / Myr')
plt.show()

In [ ]:
from scipy.stats import binned_statistic


fig,(ax1,ax2,ax3)=plt.subplots(figsize=(two_column,two_column/3),ncols=3)

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)


for gal_name in sample_table_v1p6['gal_name']:
    
    tmp = catalogue[(catalogue['gal_name']==gal_name)]
    
    color=cmap(norm(np.log10(sample_table_v1p6.loc[gal_name]['props_mstar'])))
    
    #ax1.errorbar(tmp['mass'],tmp['mass_SLUG'],xerr=tmp['mass_err'],yerr=tmp['mass_SLUG_err_plus'],fmt='o',color=color,ms=0,alpha=0.3)
    ax1.scatter(tmp['mass'],tmp['mass_SLUG'],color=color,s=0.5,alpha=0.5)

    #ax2.errorbar(tmp['age'],tmp['age_SLUG'],xerr=tmp['age_err'],yerr=tmp['age_SLUG_err_plus'],fmt='o',color=color,ms=0,alpha=0.3)
    ax2.scatter(tmp['age'],tmp['age_SLUG'],color=color,s=0.5,alpha=0.5)

    #ax3.errorbar(tmp['EBV_balmer'],tmp['av_SLUG'],xerr=tmp['EBV_balmer_err'],yerr=tmp['av_SLUG_err_plus'],fmt='o',color=color,ms=0,alpha=0.3)
    ax3.scatter(tmp['EBV_balmer'],tmp['av_SLUG'],color=color,s=0.5,alpha=0.5)

    
mean, edges, _ = binned_statistic(catalogue['mass'],catalogue['mass_SLUG'],statistic='median',bins=np.logspace(2,6,8))
x = (edges[1:]+edges[:-1])/2
ax1.plot(x,mean,color='0.2',label='SLUG median')
    
lim = [1e2,1e6]
ax1.plot(lim,lim,color='black')
ax1.set(xlim=lim,ylim=lim,xscale='log',yscale='log',xlabel='mass cigale / Msun',ylabel='mass SLUG / Msun')

mean, edges, _ = binned_statistic(catalogue['age'],catalogue['age_SLUG'],statistic='median',bins=np.linspace(0,50,10))
x = (edges[1:]+edges[:-1])/2
ax2.plot(x,mean,color='0.2',label='SLUG median')

lim = [0,50]
ax2.plot(lim,lim,color='black')
ax2.set(xlim=lim,ylim=lim,xlabel='age cigale / Myr',ylabel='age SLUG / Myr')

mean, edges, _ = binned_statistic(catalogue['EBV_balmer'],catalogue['av_SLUG'],statistic='median',bins=np.linspace(0,1,8))
x = (edges[1:]+edges[:-1])/2
ax3.plot(x,mean,color='0.2',label='SLUG median')

ax3.plot([0,1],[0,3.1],color='black')
ax3.set(xlim=[0,1],ylim=[0,10],xlabel='E(B-V) Balmer',ylabel='Av')
    
plt.tight_layout()
plt.subplots_adjust(hspace=0.05,right=0.92)
cbar_ax = fig.add_axes([0.95, 0.18, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='vertical',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))
    
plt.savefig(basedir/'reports'/'SLUG_vs_CIGALE.pdf',dpi=300)
plt.show()

#### impact of models on the escape fraction

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(figsize=(two_column,two_column/2.7),ncols=3,sharey=True)

bins = np.linspace(0,1,20)

fesc_v00 = (Qpredicted_v00-Qobserved)/Qpredicted_v00
#fesc_v00_err = Qobserved/nebulae_cluster['Qpredicted']*np.sqrt((Qobserved_err/nebulae_cluster['Qobserved'])**2+(nebulae_cluster['Qpredicted_err']/nebulae_cluster['Qpredicted'])**2)

ax1.hist(fesc_v00,bins=bins)
label = f'$f_\mathrm{{esc}}={np.nanmedian(fesc_v00):.2f}$'
ax1.text(0.05,0.85,label, transform=ax1.transAxes,color='black',fontsize=7)
ax1.set_title(r'\texttt{GENEVAv00}')

fesc_v40 = (Qpredicted_v40-Qobserved)/Qpredicted_v40
ax2.hist(fesc_v40,bins=bins)
label = f'$f_\mathrm{{esc}}={np.nanmedian(fesc_v40):.2f}$'
ax2.text(0.05,0.85,label, transform=ax2.transAxes,color='black',fontsize=7)
ax2.set(ylabel='')
ax2.set_title(r'\texttt{GENEVAv40}')

fesc_BPASS = (Qpredicted_BPASS-Qobserved)/Qpredicted_BPASS
ax3.hist(fesc_BPASS,bins=bins)
label = f'$f_\mathrm{{esc}}={np.nanmedian(fesc_BPASS):.2f}$'
#ax3.text(0.05,0.85,label, transform=ax3.transAxes,color='black',fontsize=10)
ax3.set(ylabel='')
ax3.set_title(r'\texttt{BPASS}')

for ax in (ax1,ax2,ax3):
    ax.set(xlim=[0,1],xlabel=r'$f_\mathrm{esc}$')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'escape_fraction_samples.pdf',dpi=600)
plt.show()

In [ ]:
from cluster.escape_fractions import plot_escape_fraction

fig,(ax1,ax2,ax3) = plt.subplots(figsize=(two_column,two_column/2.7),ncols=3)

ax1 = plot_escape_fraction(Qpredicted_v00,Qpredicted_v00_err,Qobserved,Qobserved_err,ax=ax1)
ax1.set_title(r'\texttt{GENEVAv00}')

ax2 = plot_escape_fraction(Qpredicted_v40,Qpredicted_v40_err,Qobserved,Qobserved_err,ax=ax2)
ax2.set(ylabel='')
ax2.set_title(r'\texttt{GENEVAv40}')

ax3 = plot_escape_fraction(Qpredicted_BPASS,Qpredicted_BPASS_err,Qobserved,Qobserved_err,ax=ax3)
ax3.set(ylabel='')
ax3.set_title(r'\texttt{BPASS}')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'escape_fraction_samples.pdf',dpi=600)
plt.show()

## Escape fractions

In [ ]:
from cluster.escape_fractions import plot_escape_fraction

fig,(ax1,ax2,ax3,ax4) = plt.subplots(figsize=(single_column,single_column*4/1.618),nrows=4,sharex=True,sharey=True)

tmp = catalogue[catalogue['robust']]
ax1 = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax1,type='other')
ax1.set(xlabel='')
ax1.set_title(r'\texttt{1to1 sample}')
#ax1.set_aspect('equal', adjustable='box')

tmp = nebulae_extended[nebulae_extended['robust']]
ax2 = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax2,type='other')
ax2.set(xlabel='')
ax2.set_title(r'\texttt{extended sample}')
#ax2.set_aspect('equal', adjustable='box')

tmp = complexes_sample[complexes_sample['robust']]
ax3 = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax3,type='other')
ax3.set(xlabel='')
ax3.set_title(r'\texttt{complexes sample}')
#ax3.set_aspect('equal', adjustable='box')

tmp = nebulae_cluster[nebulae_cluster['robust']]
ax4 = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax4,type='other')
ax4.set_title(r'\texttt{cluster sample}')
#ax4.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig(basedir/'reports'/f'escape_fraction_samples.pdf',dpi=600)
plt.show()

escape fraction over the entire galaxy

In [ ]:

for gal_name in sample_table_v1p6['gal_name']:
    Qobserved = nebulae[nebulae['in_frame'] & (nebulae['gal_name']==gal_name)]['Qobserved']
    Qpredicted = associations[associations['in_frame'] & (associations['gal_name']==gal_name)]['Qpredicted']
    print(f'{gal_name}: {100*(np.nansum(Qpredicted)-np.nansum(Qobserved))/np.nansum(Qpredicted):.1f}')

Qobserved = nebulae[nebulae['in_frame']]['Qobserved']
Qpredicted = associations[associations['in_frame']]['Qpredicted']
print(f'total: {100*(np.nansum(Qpredicted)-np.nansum(Qobserved))/np.nansum(Qpredicted):.1f}')


In [ ]:

for gal_name in sample_table_v1p6['gal_name']:
    Qobserved = catalogue[(catalogue['gal_name']==gal_name)]['Qobserved']
    Qpredicted = catalogue[(catalogue['gal_name']==gal_name)]['Qpredicted']
    print(f'{gal_name}: {100*(np.nansum(Qpredicted)-np.nansum(Qobserved))/np.nansum(Qpredicted):.1f}')

Qobserved = catalogue['Qobserved']
Qpredicted = catalogue['Qpredicted']
print(f'total: {100*(np.nansum(Qpredicted)-np.nansum(Qobserved))/np.nansum(Qpredicted):.1f}')


In [ ]:
from cluster.escape_fractions import compare_ionising_photons

tmp = catalogue[catalogue['robust'] & (catalogue['Qobserved']+catalogue['Qobserved_err']>catalogue['Qpredicted']-catalogue['Qpredicted_err'])]
tmp = tmp[tmp['age']<=2]
fesc, fesc_err = escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],stats=True)

ax1 = compare_ionising_photons(tmp['Qpredicted'],tmp['Qobserved'],color=tab10[0],s=0.5)
ax1.errorbar(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
             xerr=tmp['Qpredicted_err']/tmp['Qpredicted']/np.log(10),
             yerr=tmp['Qobserved_err']/tmp['Qobserved']/np.log(10),
             color=tab10[0],fmt='o',ms=0.1,zorder=0)


ax1.legend(handlelength=2,loc=2)
ax1.set_title(r'\texttt{robust sample}')
plt.show()

In [ ]:
compare = join(catalogue,temp,keys=['gal_name','region_ID'])
#compare = compare[compare['mass1']>1e4]
print(len(compare))

In [ ]:
fig,ax=plt.subplots(figsize=(thesis_width,thesis_width))

lim=[48,53]
ax.scatter(np.log10(compare['Qpredicted_1']),np.log10(compare['Qpredicted_2']))
ax.plot(lim,lim,color='black')
ax.set(xlim=lim,ylim=lim,xlabel='Q form stellar associations',ylabel='Q from compact cluster')
plt.show()

In [ ]:
from cluster.escape_fractions import plot_escape_fraction

fig,ax = plt.subplots(figsize=(single_column,single_column/1.618))

tmp = catalogue[(catalogue['robust']) & (catalogue['age']<=2)]
ax = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax,type='other',label='young')

tmp = catalogue[(catalogue['robust']) & (catalogue['age']>2)]
ax = plot_escape_fraction(tmp['Qpredicted'],tmp['Qpredicted_err'],tmp['Qobserved'],tmp['Qobserved_err'],ax=ax,type='other',label='old')

ax.legend(loc=6)

plt.show()

### Histogram of fesc

Many regions have invalid escape fractions (<0). It is difficult to decide how to include them in the median and histogram. 

In [ ]:
def weighted_median(values, weights):
    i = np.argsort(values)
    c = np.cumsum(weights[i])
    return values[i[np.searchsorted(c, 0.5 * c[-1])]]

fesc = catalogue[catalogue['robust']]['fesc']
fesc_err = catalogue[catalogue['robust']]['fesc_err']

weighted_median(fesc[fesc>0],weights=1/fesc_err[fesc>0])

In [ ]:
from scipy.stats import norm

def hist_with_uncertainty(mu,std,bins=np.linspace(0,100,21)):
    '''Plot hisogram with uncertaintes
    
    For each bin in the histogram, the
    
    '''
    
    n = np.zeros(len(bins)-1)
    for loc,scale in zip(mu,std):
        cdf = norm.cdf(bins,100*loc,100*scale)
        if not np.any(np.isnan(cdf)):
            n+=cdf[1:]-cdf[:-1]
    # normalize the result
    n = n/np.sum(n)
    
    edges = (bins[1:]+bins[:-1])/2
    
    fig,ax=plt.subplots()
    
    ax.step(bins,np.append(n,0),where='post')
    ax.set(xlim=[-2,102],xlabel=r'$f_\mathrm{esc}$ / per\,cent')
    plt.show()
    
tmp = catalogue[catalogue['robust']].copy()
#tmp = complexes_sample[complexes_sample['robust']]
hist_with_uncertainty(tmp['fesc'],tmp['fesc_err'])

In [ ]:
from scipy.stats import gaussian_kde


fesc = tmp['fesc']
fesc_err = tmp['fesc_err']

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

# first the normal histogram
ax.hist(100*fesc,bins=np.linspace(0,100,21),label='hist',histtype='step')

# next the histogram with the uncertainties
bins=np.linspace(0,100,21)
n = np.zeros(len(bins)-1)
for loc,scale in zip(fesc,fesc_err):
    cdf = norm.cdf(bins,100*loc,100*scale)
    if not np.any(np.isnan(cdf)):
        n+=cdf[1:]-cdf[:-1]
n = n/np.sum(n)*np.sum(fesc>0)
ax.step(bins,np.append(n,0),where='post',label='hist inc. error')

# and finally the gaussian plot
x = np.linspace(0,100,1000)
y = np.zeros_like(x)
for loc,scale in zip(fesc,fesc_err):
    y += norm.pdf(x,loc=100*loc,scale=100*scale)
ax.plot(x,y*np.max(n)/np.max(y),label='summed Gaussian')

# or gaussian KDE
kde = gaussian_kde(100*fesc,weights=1/(100*fesc_err))
y_kde = kde(x)
ax.plot(x,y_kde*np.max(n)/np.max(y_kde),label='Gaussian KDE')


ax.legend()
ax.set(xlim=[-2,102],xlabel=r'$f_\mathrm{esc}$ / per\,cent')
plt.savefig(basedir/'reports'/'fesc_hist.pdf')
plt.show()
    

In [ ]:
N_negative = 0
Ntotal = 0
for loc,scale in zip(fesc,fesc_err):
    N_negative += norm.cdf(0,100*loc,100*scale)
    Ntotal += (norm.cdf(100,100*loc,100*scale) - norm.cdf(0,100*loc,100*scale))
print(f'{Ntotal/len(fesc)*100:.2f} % have valid escape fractions')

depending on what domain we compute the main, we get different results

In [ ]:
x = np.linspace(0,1,1000)
y = np.zeros_like(x)
for loc,scale in zip(fesc,fesc_err):
    y += norm.pdf(x,loc=loc,scale=scale)

mid = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.5))
high = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.8415))
low = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.1585))

median, errp,errm = x[mid], x[high]-x[mid], x[mid]-x[low]

print(f'fesc = {median:.2f} + {errp:.2f} - {errm:.2f}')

In [ ]:
from scipy.stats import norm

tmp = catalogue[catalogue['robust']].copy()
tmp['fesc'][(tmp['fesc']<0) & (tmp['fesc']+tmp['fesc_err']>0)] = 0
#tmp = tmp[tmp['fesc']<0.5]

def combine_measurments(mu,std,lim=[-0.5,1.5]):
    '''
    
    Assuming that individual measurments are normal distributed, 
    we combine them by summing
    
    Parameters
    ----------
    
    mu : array
        the measured value
    std : array
        the standard deviation
    
    '''
    
    x = np.linspace(*lim,1000)
    y = np.zeros_like(x)
    for loc,scale in zip(mu,std):
        y += norm.pdf(x,loc=loc,scale=scale)
    
    return x,y

x,y = combine_measurments(tmp['fesc'],tmp['fesc_err'])

mid = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.5))
high = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.8415))
low = np.argmin(np.abs(np.cumsum(y)/np.sum(y)-0.1585))

median, errp,errm = x[mid], x[high]-x[mid], x[mid]-x[low]

In [ ]:
fig,(ax1,ax2)=plt.subplots(nrows=2,figsize=(single_column,2*single_column/1.618),sharex=True)

n,bins,_ = ax1.hist(tmp['fesc'],bins=np.linspace(0,1,20),histtype='step')
ax1.plot(x,y*np.max(n[bins[1:]>0.5])/np.max(y))

#for row in tmp:
#    ax1.plot(x,0.002*norm.pdf(x,loc=row['fesc'],scale=row['fesc_err']),color='gray',zorder=0)
    
ax1.axvline(median,ls='--',c='k',lw=0.5)
ax1.axvspan(1,1.5,color='gray',alpha=0.3,zorder=0,ec=None)
ax1.axvspan(-0.5,0,color='gray',alpha=0.3,zorder=0,ec=None)
ax1.set(xlim=[-0.5,1.5],ylabel='PDF')
label = f'$f_\mathrm{{esc}}={median:.2f}^{{+{errp:.2f}}}_{{-{errm:.2f}}}$'
ax1.text(0.3,0.85,label, transform=ax1.transAxes,color='black',fontsize=8)

ax2.plot(x,np.cumsum(y)/np.sum(y))

ax2.axhline(0.5,ls='--',c='k',lw=0.5)
ax2.axhline(0.5+0.683/2,ls='--',c='k',lw=0.5)
ax2.axhline(0.5-0.683/2,ls='--',c='k',lw=0.5)
ax2.axvline(median,ls='--',c='k',lw=0.5)
ax2.axvline(median-errm,ls='--',c='k',lw=0.5)
ax2.axvline(median+errp,ls='--',c='k',lw=0.5)
ax2.axvspan(1,1.5,color='gray',alpha=0.3,zorder=0,ec=None)
ax2.axvspan(-0.5,0,color='gray',alpha=0.3,zorder=0,ec=None)

ax2.set(xlim=[-0.5,1.5],ylim=[0,1],xlabel=r'$f_\mathrm{esc}$',ylabel='CDF')
plt.tight_layout()
#plt.savefig(basedir/'reports'/'fesc_pdf.pdf',dpi=300)
plt.show()

In [ ]:
mid = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.5))
high = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.8415))
low = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.1585))
median, errp,errm = x1[mid], x1[high]-x1[mid], x1[mid]-x1[low]

median


In [ ]:
from scipy.stats import gaussian_kde

x1 = np.linspace(-0.5,1.5,1000)

kde = gaussian_kde(fesc,weights=1/fesc_err)
y1 = kde(x1)

mid = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.5))
high = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.8415))
low = np.argmin(np.abs(np.cumsum(y1)/np.sum(y1)-0.1585))
median, errp,errm = x1[mid], x1[high]-x1[mid], x1[mid]-x1[low]

fig,ax=plt.subplots()

ax.plot(x1,y1,label='kde')
ax.axvline(median,label='kde',color='black',ls=':')

x2,y2 = combine_measurments(tmp['fesc'],tmp['fesc_err'])
mid = np.argmin(np.abs(np.cumsum(y2)/np.sum(y2)-0.5))
high = np.argmin(np.abs(np.cumsum(y2)/np.sum(y2)-0.8415))
low = np.argmin(np.abs(np.cumsum(y2)/np.sum(y2)-0.1585))
median, errp,errm = x2[mid], x2[high]-x2[mid], x2[mid]-x2[low]

ax.plot(x2,y2*np.max(y1)/np.max(y2),label='summed gaussians')
ax.axvline(median,label='summed gaussians',color='black',ls='-')

ax.legend()
plt.show()

### Plot escape fraction

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,0.9*single_column))

Qpredicted = np.logspace(48,52)
cmap = plt.cm.get_cmap('cool',6)
lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved = Qpredicted*(1-f)
    ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)
Qobserved = Qpredicted*(1-0.76)
ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls='-',color='gray',label=f'$f_\mathrm{{esc}}={0.76}$',zorder=1)

#ax.errorbar(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
#            xerr=tmp['Qpredicted_err']/tmp['Qpredicted'],yerr=tmp['Qobserved_err']/tmp['Qobserved'],fmt='',color='gray',zorder=0)
sc=ax.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['density'],cmap=cmap,vmin=0.,vmax=60,s=2,zorder=2)
ax.legend(handlelength=2)
fig.colorbar(sc,label='density / cm$^{-3}$')

ax.set(xlabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ predicted (SB99)',
       ylabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ observed (MUSE)',
       xlim=[49,52],ylim=[49,52])

ax.set_xticks([49,50,51,52])
ax.set_yticks([49,50,51,52])
#ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 
#ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 

plt.savefig(basedir/'reports'/f'escape_fraction.png',dpi=600)
plt.show()

In [ ]:
labels = ['bar', 'bar ends', 'center', 'interarm', 'interbar', 'spiral arms','others']
l_to_n = {l:i for i,l in zip(range(len(labels)),labels)}

#fig,((ax1,ax2),(ax3,ax4),(ax5,ax6)) =plt.subplots(ncols=2,nrows=3,figsize=(two_column,1.2*two_column))
fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) =plt.subplots(ncols=3,nrows=2,figsize=(1.2*two_column,two_column/1.5))


criteria  = (catalogue['overlap_asc']==1) 
criteria  &= (catalogue['mass']>1e4) 
#criteria &= (catalogue['age']<=10) 
tmp = catalogue[criteria].copy()

print(f'{len(tmp)} objects with fesc={np.mean(tmp[tmp["fesc"]>0]["fesc"]):.2f}')

Qpredicted = np.logspace(47,52)
lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved = Qpredicted*(1-f)
    for ax in (ax1,ax2,ax3,ax4,ax5,ax6):
        ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)

cmap = plt.cm.get_cmap('plasma',5)
sc1=ax1.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['logq_D91'],cmap=cmap,vmin=6,vmax=8,s=2,zorder=2)
cmap = plt.cm.get_cmap('cividis',5)
sc2=ax2.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['dig/hii'],cmap=cmap,vmin=0,vmax=1,s=2,zorder=2)
cmap = plt.cm.get_cmap('viridis',4)
sc3=ax3.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['EBV_balmer'],cmap=cmap,vmin=0,vmax=0.8,s=2,zorder=2)
cmap = plt.cm.get_cmap('copper',5)
sc4=ax4.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
                c=tmp['age'],cmap=cmap,vmin=0,vmax=10,s=2,zorder=2)
#cmap = plt.cm.get_cmap('ocean',5)
#https://stackoverflow.com/questions/18926031/how-to-extract-a-subset-of-a-colormap-as-a-new-colormap-in-matplotlib
#cmap = mpl.colors.LinearSegmentedColormap.from_list('new_ocean',cmap(np.linspace(0, 0.79, 4)),8)
#sc5=ax5.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
#                c=np.log10(tmp['HA6562_LUM_CORR']),cmap=cmap,vmin=37,vmax=39,s=2,zorder=2)
cmap = plt.cm.get_cmap('cool',6)
sc5=ax5.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
                c=tmp['density'],cmap=cmap,vmin=0.,vmax=60,s=2,zorder=2)

tmp.sort('env_neb',reverse=True)
cmap = plt.cm.get_cmap('Spectral',6)
env = [l_to_n.get(x,6) for x in tmp['env_neb']]
sc6=ax6.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=env,cmap=cmap,vmin=-0.5,vmax=5.5,s=1,zorder=2)

''' # check if the colorbar is correctly labeld
for col in labels:
    sub = tmp[tmp['env_neb']==col]
    env = [l_to_n.get(x,6) for x in sub['env_neb']]
    sc6=ax6.scatter(np.log10(sub['Qpredicted']),np.log10(sub['Qobserved']),
                  c=env,cmap=cmap,vmin=-0.5,vmax=5.5,s=2,zorder=2,label=col)
ax6.legend()
'''

ax1.legend(handlelength=2)
fig.colorbar(sc1,ax=ax1,label=r'$\log q$ (from Diaz 91)',ticks=[6,6.4,6.8,7.2,7.6,8])
fig.colorbar(sc2,ax=ax2,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
fig.colorbar(sc3,ax=ax3,label=r'$E(B-V)_\mathrm{Balmer}$',ticks=[0,0.2,0.4,0.6,0.8])
fig.colorbar(sc4,ax=ax4,label=r'age / Myr')
fig.colorbar(sc5,ax=ax5,label=r'density / cm$^{-3}$')
cbar=fig.colorbar(sc6,ax=ax6,ticks=np.arange(0.,5.5,1))
cbar.ax.set_yticklabels(labels[:-1])  # vertically oriented colorbar

for ax in (ax4,ax5,ax6):
    ax.set(xlabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ predicted')
for ax in (ax1,ax4):
    ax.set(ylabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ observed')

for ax in (ax1,ax2,ax3,ax4,ax5,ax6):
    ax.set(xlim=[49,52],ylim=[49,52])
    ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(3)) 
    ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(3)) 
plt.tight_layout()
#plt.savefig(basedir/'reports'/f'escape_fraction_{HSTband}_{scalepc}pc.png',dpi=600)
plt.show()

In [ ]:
fig,ax=plt.subplots()


for env in ['interarm','interbar','spiral arms']:
    sub = tmp[tmp['env_neb']==env]
    print(f'{env}: {100*np.nanmean(sub["fesc"]):.1f}+-{100*np.nanstd(sub["fesc"]):.1f} % ({len(sub)})')
    
    ax.hist(100*sub['fesc'],bins=np.linspace(0,100,20),
            histtype='step',alpha=0.9,label=env)
    
ax.legend(loc=2)
ax.set(xlabel=r'$f_\mathrm{esc}$ / percent')
plt.show()

In [ ]:
from scipy.stats import spearmanr

fig,((ax1,ax2,ax3),(ax4,ax5,ax6)) =plt.subplots(ncols=3,nrows=2,figsize=(two_column,two_column/1.5))


for col in ['logq_D91','dig/hii','EBV_balmer','age','mass','density']:
    rho,p = spearmanr(tmp[col],tmp['fesc'],nan_policy='omit')

    print(f'{col:>10}: rho={rho:>5.2f}, p-value={p:>7.2g}')

ax1.scatter(tmp['logq_D91'],tmp['fesc'])
ax1.set(xlabel=r'$\log q$ (from Diaz 91)')

ax2.scatter(tmp['dig/hii'],tmp['fesc'])
ax2.set(xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')

ax3.scatter(tmp['EBV_balmer'],tmp['fesc'])
ax3.set(xlabel=r'$E(B-V)_\mathrm{stars}$')

ax4.scatter(tmp['age'],tmp['fesc'])
ax4.set(xlabel=r'age / Myr',xlim=[0,10])

ax5.scatter(tmp['mass'],tmp['fesc'])
ax5.set(xlabel=r'$\log_{10} M/\mathrm{M}_\odot$',xscale='log')

ax6.scatter(tmp['density'],tmp['fesc'])
ax6.set(xlabel=r'density')



for ax in [ax1,ax2,ax3,ax4,ax5,ax6]:
    ax.set(ylim=[0,1.1])
for ax in [ax1,ax4]:
    ax.set(ylabel='fesc')

plt.tight_layout()
plt.show()


### External sources

we need to ionizing photon flux of most associations, so we compute them first

In [ ]:
with open(basedir/'data'/'interim'/f'neighboring_associations.yml') as f:
    neighboring_associations = yaml.load(f,Loader=yaml.SafeLoader)

In [ ]:
from cluster.escape_fractions import ionising_photon_flux

flux, flux_err = ionising_photon_flux(
                                     age = associations['age'],
                                     age_err = associations['age_err'],
                                     model_age = model_age,
                                     model_flux = model_flux,
                                     sample_size=100
                                     )

Qpredicted     = flux*associations['mass']
Qpredicted_err = Qpredicted * np.sqrt((flux_err/flux)**2+(associations['mass_err']/associations['mass'])**2)

associations['Qpredicted']     = Qpredicted
associations['Qpredicted_err'] = Qpredicted_err

In [ ]:
from tqdm import tqdm

subsample = catalogue.copy()
subsample['Qexternal'] = 0.0
subsample['Nexternal'] = 0
subsample['d_max'] = 0.

gal_name = None 
for row in tqdm(subsample):
    
    if row['gal_name']!=gal_name:
        gal_name = row['gal_name']
        assoc_sub = associations[associations['gal_name']==gal_name]
        assoc_sub['distance'] = np.nan
        assoc_sub.add_index('assoc_ID')
        
        # these properties also change with gal_name
        distance = row['dist']
        arcsec_to_parsec = distance*1e6*u.arcsec.to(u.rad)
    
    dic = neighboring_associations[gal_name][row['region_ID']]
    assoc_IDs = np.atleast_1d(list(dic.keys()))
    distances = np.atleast_1d(list(dic.values())) 
    
    distances = distances[np.isin(assoc_IDs,assoc_sub['assoc_ID'])]
    assoc_IDs = assoc_IDs[np.isin(assoc_IDs,assoc_sub['assoc_ID'])]
    
    row['d_max'] = np.max(distances) * arcsec_to_parsec
    row['Nexternal'] = len(distances)
    
    # area in arcsec2
    area_neb = row['area_neb'] / 25
        
    if len(dic)>1:
        tmp = assoc_sub.loc[assoc_IDs]
        tmp['distance'] = distances
        tmp = tmp[tmp['distance']>0]

        Qexternal = tmp['Qpredicted'] / (4*np.pi*tmp['distance']**2) * area_neb 

        # we exlude the associations that overlap and those that are contained within another 
        # nebula. only associations within 500 pc are considered
        row['Qexternal'] += np.sum(Qexternal[(tmp['distance']>0) & (tmp['overlap_asc']<1) & (tmp['distance']*arcsec_to_parsec<=1000)])
        #row['Nexternal'] += 1
    
    else:
        tmp = assoc_sub.loc[list(dic.keys())]
        tmp['distance'] = np.atleast_1d(list(dic.values()))
        if (tmp['distance']==0) or (tmp['overlap_asc']==1) or (tmp['distance']*arcsec_to_parsec>1000):
            continue

        Qexternal = tmp['Qpredicted'] / (4*np.pi*tmp['distance']**2) * area_neb 
        row['Qexternal'] += np.sum(Qexternal)
       

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=5,vmax=20)
bins = np.linspace(0,500)

for gal_name in sample_table_v1p6['gal_name']:
    tmp = subsample[subsample['gal_name']==gal_name]
    dist = sample_table_v1p6.loc[gal_name]['dist']
    ax.hist(tmp['d_max'],bins=bins,color=cmap(norm(dist)),alpha=0.6)
    
ax.set(xlim=[10,100],xlabel=r'$r_\mathrm{max}$ / pc')
plt.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.95, 0.12, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,
             label='distance / Mpc',ticks=np.arange(5,20,6))

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=5,vmax=20)
bins = np.linspace(0,100)

nebulae['r_pc'] = np.sqrt(nebulae['area_neb']/np.pi)/5 * nebulae['dist']*1e6*u.arcsec.to(u.rad)

for gal_name in sample_table_v1p6['gal_name']:
    tmp = nebulae[nebulae['gal_name']==gal_name]
    dist = sample_table_v1p6.loc[gal_name]['dist']
    ax.hist(tmp['r_pc'],bins=bins,color=cmap(norm(dist)),alpha=0.6)
    
ax.set(xlim=[10,100],xlabel=r'$r_\mathrm{nebula}$ / pc')
plt.subplots_adjust(right=0.92)
cbar_ax = fig.add_axes([0.95, 0.12, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,
             label='distance / Mpc',ticks=np.arange(5,20,6))

plt.show()

In [ ]:
for row in sample_table_v1p6:
    distance = row['dist']
    arcsec_to_parsec = distance*1e6*u.arcsec.to(u.rad)
    print(f'{row["gal_name"]}: {arcsec_to_parsec*20:.1f}')

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(-3,3,21)

tmp = subsample[(subsample['Qpredicted']>subsample['Qobserved'])]
ax.hist(tmp['Qexternal']/tmp['Qpredicted'],bins=bins,label=r'positive $f_\mathrm{esc}$',alpha=0.5)
tmp = tmp[np.isfinite(tmp['Qexternal'])]
print(f"fesc>0: Qe/Qp={np.nanmedian(tmp['Qexternal']/tmp['Qpredicted']):.5f}")

tmp = subsample[(subsample['Qpredicted']<subsample['Qobserved'])]
ax.hist(tmp['Qexternal']/tmp['Qpredicted'],bins=bins,label=r'negative $f_\mathrm{esc}$',alpha=0.5)
tmp = tmp[np.isfinite(tmp['Qexternal'])]
print(f"fesc<0: Qe/Qp={np.nanmedian(tmp['Qexternal']/tmp['Qpredicted']):.5f}")

ax.axvline(1,color='gray')
ax.legend()
ax.set(xlim=[1e-3,1e3],xscale='log',xlabel=r'$Q (\mathrm{H}^0)_\mathrm{external}\,/\,Q (\mathrm{H}^0)_\mathrm{internal}$')

#plt.savefig(basedir/'reports'/'external_ionising_sources.pdf')
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(-3,3,21)

tmp = subsample[(subsample['Qpredicted']>subsample['Qobserved'])]
ax.scatter(tmp['Qexternal']/tmp['Qpredicted'],tmp['fesc'],s=1)
ax.axvline(1,color='gray')

ax.set(ylim=[0,1],xlim=[1e-3,1e3],xscale='log',ylabel=r'$f_\mathrm{esc}$',
       xlabel=r'$Q (\mathrm{H}^0)_\mathrm{external}\,/\,Q (\mathrm{H}^0)_\mathrm{internal}$')

#plt.savefig(basedir/'reports'/'external_ionising_sources.pdf')
plt.show()

### Trends with Fesc

In [ ]:
# define limits and labels for corner plots
limits   = {
            'Delta_met_scal':(-0.1,0.1),
            'density':(0,100),
            'dig/hii':(0.2,0.9),
            'fesc':(0,1),
            'HA/FUV_corr':(0.8,60),
            'HA6562_LUM_CORR':(5e35,5e39),
            'EBV_balmer':(0,1),
            'logq_D91':(5.7,8),
            'EW_HA':(6,600),
            'EW_HA_CORR':(60,6000),
            'temperature':(6000,8e3),
            'age' : (0,8),
            'mass' : (1e4,2e5)
            }


labels   = {
            'Delta_met_scal':r'$\Delta$(O/H)',
            'density':r'density / cm$^{-3}$','temperature':'T / K',
            'dig/hii':r'$I_\mathrm{DIG}\,/\,I_{\mathrm{H}\,\tiny{\textsc{ii}}}$',
            'fesc':r'$f_\mathrm{esc}$',
            'HA/FUV_corr':r'H$\alpha$ / FUV',
            'HA6562_LUM_CORR':r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
            'EBV_balmer':r'$E(B-V)_\mathrm{Balmer}$',
            'EW_HA' : r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$',
            'EW_HA_CORR' : r'$\mathrm{EW}(\mathrm{H}\alpha)_{\mathrm{corr}}/\mathrm{\AA}$',
            'logq_D91':r'$\log q$',
            'age' : 'age / Myr',
            'mass' : r'mass / $\mathrm{M}_\odot$'
           }

with fesc on x axis

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile, \
                                   corner_scatter
from scipy.stats import spearmanr,binned_statistic
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

columns = ['logq_D91','density','EBV_balmer','dig/hii']
tmp = catalogue[catalogue['robust']]

def plot_function(x,y,group_by,ax,bins,**kwargs):

    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_scatter(x_sub,y_sub,ax,s=4,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')

fig,axes=plt.subplots(nrows=len(columns),figsize=(single_column,single_column*len(columns)/2),sharex='col')
cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

for ax,col in zip(axes,columns):
    
    bins = np.linspace(0,1,11)
    plot_function(tmp['fesc'],tmp[col],tmp['gal_name'],ax,bins=bins)
    ax.set(ylabel=labels[col],ylim=limits[col])
    
    if col == 'HA6562_LUM_CORR':
        ax.set(yscale='log')
    
    if col == 'logq_D91':
        ymin,ymax=limits['logq_D91']
        ax_right = ax.twinx()
        ax_right.set(ylim=[logq_D91_reverse(ymin),logq_D91_reverse(ymax)],yscale='log')
        ax_right.set_yticklabels([])
        ax_right.set_ylabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
        ax_right.set_yticks([0.1,1])
        ax_right.set_yticklabels(['0.1','1'])
    
ax.set(xlim=[0,1],xscale='linear',xlabel=r'$f_\mathrm{esc}$ / per\,cent')       
#axes[0].set(ylabel=labels['fesc'])

plt.tight_layout()
plt.subplots_adjust(hspace=0.05,top=0.92)
cbar_ax = fig.add_axes([0.17, 0.95, 0.67, 0.02])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='horizontal',ticklocation='top',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

#plt.savefig(basedir/'reports'/'trends_with_fesc.pdf',dpi=300)

plt.show()

or with fesc on y-axis

In [ ]:
from astrotools.plot.corner import corner_binned_stat, corner_binned_percentile, \
                                   corner_scatter
from scipy.stats import spearmanr,binned_statistic
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

tmp = catalogue[catalogue['robust']]

def plot_function(x,y,group_by,ax,bins,**kwargs):

    finite = np.isfinite(x) & np.isfinite(y)
    x,y,group_by = x[finite], y[finite],group_by[finite]
    
    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.2)
    
    for gal_name in np.unique(group_by):
        x_sub,y_sub=x[group_by==gal_name],y[group_by==gal_name]
        corner_scatter(x_sub,y_sub,ax,s=4,color=cmap(norm(sample_table.loc[gal_name]['mass'])))
        
    r,p = spearmanr(x,y,nan_policy='omit')
    label = r'$\rho'+f'={r:.2f}$'
    #t = ax.text(0.08,0.9,label,transform=ax.transAxes,ha='left',va='top',fontsize=7)
    #t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))
    corner_binned_stat(x,y,ax,bins=bins,color='black')

fig,(ax1,ax2,ax3,ax4)=plt.subplots(ncols=len(columns),figsize=(two_column,two_column/len(columns)*1.2),sharey='row')
cmap = mpl.cm.get_cmap('spring')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

    
bins = np.linspace(0.5,8.5,9)
plot_function(tmp['age'],tmp['fesc'],tmp['gal_name'],ax1,bins=bins)
ax1.set(xlabel=labels['age'],xlim=limits['age'])

bins = np.logspace(4,5.3,6)
plot_function(tmp['mass'],tmp['fesc'],tmp['gal_name'],ax2,bins=bins)
ax2.set(xlabel=labels['mass'],xlim=limits['mass'],xscale='log')

bins = np.linspace(0,100,6)
plot_function(tmp['density'],tmp['fesc'],tmp['gal_name'],ax3,bins=bins)
ax3.set(xlabel=labels['density'],xlim=limits['density'])

bins = np.linspace(0,1,6)
plot_function(tmp['EBV_balmer'],tmp['fesc'],tmp['gal_name'],ax4,bins=bins)
ax4.set(xlabel=labels['EBV_balmer'],xlim=limits['EBV_balmer'])


if col == 'logq_D91':
    xmin,xmax=limits['logq_D91']
    ax_top = ax.twiny()
    ax_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
    ax_top.set_xticklabels([])
    ax_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
    ax_top.set_xticks([0.1,1])
    ax_top.set_xticklabels(['0.1','1'])
        
ax1.set(ylabel=labels['fesc'],ylim=[0,1],yscale='linear')

plt.tight_layout()
plt.subplots_adjust(wspace=0.12,right=0.95)
cbar_ax = fig.add_axes([0.97, 0.22, 0.02, 0.7])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,orientation='vertical',
             label=r'$\log (M_\star\,/\,\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

#plt.savefig(basedir/'reports'/'trends_with_fesc.pdf',dpi=300)

plt.show()

## Other Results

### Ha luminosity function

In [ ]:
bins = np.logspace(35,40,20)

fig,(ax1,ax2,ax3,ax4) = plt.subplots(figsize=(two_column,two_column/3.5),ncols=4,sharey=True)

tmp = catalogue
ax1.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='1to1',zorder=0,histtype='step')
ax1.set_title(r'\texttt{1to1 sample}')
#ax1.set_aspect('equal', adjustable='box')
print(f"1to1: {np.nanmedian(tmp['HA6562_LUM_CORR']):.2g} erg s-1")

tmp = nebulae_extended
ax2.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='1to1',zorder=0,histtype='step')
ax2.set(ylabel='')
ax2.set_title(r'\texttt{extended sample}')
#ax2.set_aspect('equal', adjustable='box')
print(f"Extended: {np.nanmedian(tmp['HA6562_LUM_CORR']):.2g} erg s-1")

tmp = complexes_sample
ax3.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='1to1',zorder=0,histtype='step')
ax3.set(ylabel='')
ax3.set_title(r'\texttt{complexes sample}')
#ax3.set_aspect('equal', adjustable='box')
print(f"Complexes: {np.nanmedian(tmp['HA6562_LUM_CORR']):.2g} erg s-1")

tmp = nebulae_cluster
ax4.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='1to1',zorder=0,histtype='step')
ax4.set(ylabel='')
ax4.set_title(r'\texttt{cluster sample}')
#ax4.set_aspect('equal', adjustable='box')
print(f"Cluster: {np.nanmedian(tmp['HA6562_LUM_CORR']):.2g} erg s-1")

for ax in [ax1,ax2,ax3,ax4]:
    ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='linear')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'Ha_luminosity_subsamples.pdf',dpi=600)
plt.show()

In [ ]:
bins = np.logspace(35,40,20)

fig,ax= plt.subplots(figsize=(thesis_width,thesis_width/1.618),ncols=1)

tmp = catalogue
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label=r'\texttt{1to1 sample}',zorder=0,histtype='step')

tmp = nebulae_extended
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label=r'\texttt{extended sample}',zorder=0,histtype='step')

tmp = complexes_sample
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label=r'\texttt{complexes sample}',zorder=0,histtype='step')

tmp = nebulae_cluster
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label=r'\texttt{cluster sample}',zorder=0,histtype='step')

ax.legend()
ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
   yscale='linear')

plt.tight_layout()
#plt.savefig(basedir/'reports'/f'Ha_luminosity_subsamples.pdf',dpi=600)
plt.show()

### mass vs observed Halpha

In [ ]:
# calculate Ha on a grid of different ages/masses with Starburst99
age_grid = np.array([0,6,12,18])*u.Myr/2
mass_grid = np.logspace(2,7,10)
ionizing_photons = np.zeros((len(age_grid),len(mass_grid)))
for i,mass in enumerate(mass_grid):
    scaled_cluster = cluster.scale(mass)
    for j,age in enumerate(age_grid):
        idx = np.argmin(np.abs(scaled_cluster.quanta['Time']-age))
        ionizing_photons[j,i] = scaled_cluster.quanta['HI_rate'][idx].value
distance = 10*u.Mpc
LHa = ionizing_photons / 7.31e11 * u.erg 
FHa = LHa / (1e-20*u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)

In [ ]:
nrows, ncols = 4,5

aspect_ratio = 0.9
cmap = plt.cm.get_cmap('copper',6)
fig=plt.figure(figsize=(two_column,aspect_ratio*nrows/ncols*two_column))
axes = []

criteria  = (catalogue['overlap_asc']==1) 
criteria  &= (catalogue['mass']>1e2) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.1) 
#criteria &= fesc>0
tmp = catalogue[criteria].copy()


sample = list(np.unique(tmp['gal_name']))
#sample.remove('IC5332')

for i,gal_name in enumerate(sample):
    ax = fig.add_subplot(nrows,ncols,i+1)
    axes.append(ax)
    sub = tmp[tmp['gal_name']==gal_name]
    
    distance = sub['distance'][0]*u.Mpc
    LHa = ionizing_photons / 7.31e11 * u.erg 
    FHa = LHa / (u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)
    
    sc=ax.scatter(sub['mass'],1e-20*sub['HA6562_FLUX_CORR'],s=3,
                  c=sub['age'],cmap=cmap,vmin=0,vmax=9,zorder=2)
    
    # plot the theoretical liens
    for j,age in enumerate(age_grid):
        color = cmap(age/9/u.Myr)
        ax.plot(mass_grid,FHa[j,:],label=age,color=color)
    
    ax.set(xlim=[2e2,2e5],ylim=[1e-17,1e-12],xscale='log',yscale='log')
    ax.text(0.05,0.85,f'{gal_name}', transform=ax.transAxes,fontsize=7)

    #ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(1e4))
    #ax.xaxis.set_minor_locator(mpl.ticker.MultipleLocator(1e5))
    
    if i%ncols==0:
        #ax.set(ylabel=r'$F(\mathrm{H}\alpha)$ / erg s$^{-1}$ cm$^{-2}$')
        pass
    else:
        ax.set_yticklabels([])
    if i//ncols==nrows-1:
        #ax.set(xlabel=r'mass / M$_\odot$')
        pass
    else:
        ax.set_xticklabels([])
    #print(np.diff(ax.get_xlim())/np.diff(ax.get_ylim()))
    #ax.set_aspect(0.5)

fig.subplots_adjust(wspace=0.05,hspace=0.05,right=0.9,left=0.1,bottom=0.1)
fig.text(0.5, 0.04, r'mass / M$_\odot$', ha='center')
fig.text(0.0, 0.5, r'$F(\mathrm{H}\alpha)$ / erg s$^{-1}$ cm$^{-2}$', va='center', rotation='vertical')
    
# Create the legend
h,l = axes[0].get_legend_handles_labels()
fig.legend(h,l,
           ncol=5,
           #loc="upper center",   # Position of legend
           #borderaxespad=-0.1,    # Small spacing around legend box
           bbox_to_anchor=(0.25, 0.9, 0.5, 0.05) 
          )

cbar_ax = fig.add_axes([0.93, 0.08, 0.02, 0.8])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr',ticks=age_grid)

#plt.savefig(basedir/'reports'/f'mass_vs_Halpha_{HSTband}_{scalepc}pc.png',dpi=600)
plt.show()

## Inspect sample

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(35,40,20)

tmp = catalogue
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='1to1',zorder=0)
print(f"1to1: {np.mean(tmp['HA6562_LUM_CORR']):.2g}")

tmp = nebulae_extended
ax.hist(tmp['HA6562_LUM_CORR'],bins=bins,alpha=0.6,label='extened',zorder=0)
print(f"extened: {np.mean(tmp['HA6562_LUM_CORR']):.2g}")

ax.legend()

ax.set(xscale='log',xlim=[2e35,9e39],xlabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$',
       yscale='linear',ylim=[1,800],ylabel='N')
#plt.savefig(basedir/'reports'/'Halpha_luminosity_function.pdf',dpi=300)
plt.show()


### For the stellar associations

In [ ]:
sub = extended 
groups = sub.group_by(['gal_name','region_ID'])

i=0
j=0
k=0
for group in groups.groups:
    if np.all(group['mass']>1e4):
        i+=1
    if np.all(group['overlap']=='contained'):
        j+=1
    if np.all(group['mass']>1e4) & np.all(group['overlap']=='contained') & np.all(group['age']<=8):
        k+=1
        
print(f'in {i} of {len(groups.groups)} groups, all associations are massive')
print(f'in {j} of {len(groups.groups)} groups, all associations are contained')
print(f'in {k} fullfill both criteria')

### Photon flux of individual objects

In [ ]:
sub = extended 
groups = sub.group_by(['gal_name','region_ID'])

# aggregate additional properties to the group
temp = groups.groups.keys
temp['Qpredicted'] = groups['Qpredicted'].groups.aggregate(np.sum)
temp['mass'] = groups['mass'].groups.aggregate(np.sum)
temp.sort('mass',reverse=False)

In [ ]:
fig,ax=plt.subplots(figsize=(two_column,two_column/3))
cmap = plt.cm.get_cmap('viridis', 10)
norm = mpl.colors.Normalize(vmin=0.5,vmax=10.5)

y1,y2=5,50
x1,x2=46,52
a = (y2-y1) / (x2-x1)
b = y1-a*x1

i=0
#for row in temp[~np.isnan(temp['mass'])][200::16]:
for row in temp[temp['gal_name']=='NGC2835']:
    mask  = (extended['region_ID'] == row['region_ID']) & (extended['gal_name']==row['gal_name'])
    group = extended[mask]
    
    i+=1
    sub = group[group['overlap']=='contained']
    s = a*np.log10(sub['Qpredicted'])+b
    s[s<5]  = 5
    s[s>50] = 50
    sc= ax.scatter(len(sub)*[i],sub['mass'],c=sub['age'],s=s,marker='o',vmin=0,vmax=10,cmap=cmap)
    sub = group[group['overlap']=='partial']
    s = a*np.log10(sub['Qpredicted'])+b
    s[s<5]  = 5
    s[s>50] = 50
    sc= ax.scatter(len(sub)*[i],sub['mass'],c=sub['age'],s=s,marker='o',vmin=0.5,vmax=10.5,cmap=cmap)
    sc= ax.scatter(len(sub)*[i],sub['mass'],c='white',s=s/8,marker='o',vmin=0.5,vmax=10.5,cmap=cmap)

    # we want at least one young and massive association that is fully contained
    # and no youg and massive objects with partial overlap
    if not np.any((group['mass']>1e4) & (group['age']<=8) & (group['overlap']=='contained')) or np.any((group['mass']>1e4) & (group['overlap']=='partial') & (group['age']<=8)):
        ax.axvspan(i-0.4,i+0.4,color='gray',alpha=0.3,zorder=0,ec=None)
    
# adjustments to the figure
for s in [52,49,46]:
    ax.scatter([-1],[-1],s=a*s+b,color='gray',label=f'$10^{{{s}}}$')
ax.legend(title=r'$Q(\mathrm{H}^0)\,/\,\mathrm{s}^{-1}$',title_fontsize='xx-small',loc=2)

ax.set(xlim=[0.5,i+0.5],yscale='log',ylim=[2e2,3e5],
       xlabel=r'H\,\textsc{ii} regions',ylabel=r'mass / $\mathrm{M}_\odot$')
ax.axhline(1e4,color='gray',lw=0.5,zorder=0)
ax.set_xticks([])
#ax.grid()

fig.subplots_adjust(right=0.95,wspace=0.3)
cbar_ax = fig.add_axes([0.96, 0.126, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,label=r'age / Myr')
plt.savefig(basedir/'reports'/'association_groups.pdf')
plt.show()

same plot but mass and Q are changed

In [ ]:
fig,ax=plt.subplots(figsize=(two_column,two_column/3))
cmap = plt.cm.get_cmap('viridis', 10)
norm = mpl.colors.Normalize(vmin=0.5,vmax=10.5)

y1,y2=5,50
x1,x2=2,6
a = (y2-y1) / (x2-x1)
b = y1-a*x1

i=0
#for row in temp[~np.isnan(temp['mass'])][200::16]:
for row in temp[temp['gal_name']=='NGC2835']:
    mask  = (extended['region_ID'] == row['region_ID']) & (extended['gal_name']==row['gal_name'])
    group = extended[mask]
    
    i+=1
    sub = group[group['overlap']=='contained']
    s = a*np.log10(sub['mass'])+b
    s[s<5]  = 5
    s[s>50] = 50
    sc= ax.scatter(len(sub)*[i],sub['Qpredicted'],c=sub['age'],s=s,marker='o',vmin=0,vmax=10,cmap=cmap)
    sub = group[group['overlap']=='partial']
    s = a*np.log10(sub['mass'])+b
    s[s<5]  = 5
    s[s>50] = 50
    sc= ax.scatter(len(sub)*[i],sub['Qpredicted'],c=sub['age'],s=s,marker='o',vmin=0.5,vmax=10.5,cmap=cmap)
    sc= ax.scatter(len(sub)*[i],sub['Qpredicted'],c='white',s=s/8,marker='o',vmin=0.5,vmax=10.5,cmap=cmap)

    # we want at least one young and massive association that is fully contained
    # and no youg and massive objects with partial overlap
    if not np.any((group['mass']>1e4) & (group['age']<=8) & (group['overlap']=='contained')) or np.any((group['mass']>1e4) & (group['overlap']=='partial') & (group['age']<=8)):
        ax.axvspan(i-0.4,i+0.4,color='gray',alpha=0.3,zorder=0,ec=None)
    
# adjustments to the figure
for s in [6,4,2]:
    ax.scatter([-1],[-1],s=a*s+b,color='gray',label=f'$10^{{{s}}}$')
ax.legend(title=r'mass / M$_\odot$',title_fontsize='xx-small',loc=2)

ax.set(xlim=[0.5,i+0.5],yscale='log',ylim=[1e47,1e53],
       xlabel=r'H\,\textsc{ii} regions',ylabel=r'$Q(\mathrm{H}^0)\,/\,\mathrm{s}^{-1}$')
#ax.axhline(1e4,color='gray',lw=0.5,zorder=0)
ax.set_xticks([])
#ax.grid()

fig.subplots_adjust(right=0.95,wspace=0.3)
cbar_ax = fig.add_axes([0.96, 0.126, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,label=r'age / Myr')
plt.savefig(basedir/'reports'/'association_groups_Q.pdf')
plt.show()

### Plot cutouts

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.lines as mlines

from cluster.io import read_associations
from cluster.plot import single_cutout

#tmp = catalogue[catalogue['Qpredicted']+3*catalogue['Qpredicted_err']<catalogue['Qobserved']-3*catalogue['Qpredicted_err']]
#tmp = catalogue[(catalogue['mass']>1e4) & (catalogue['age']<=8) & (catalogue['overlap']=='contained')]
#tmp = nebulae_extended[nebulae_extended['mass']>1e4]
#tmp = nebulae_extended[nebulae_extended['robust']]
#tmp = nebulae[(nebulae['gal_name']=='NGC2835') & (nebulae['Nassoc']>=1) & (nebulae['neighbors']==0)]
#tmp = catalogue[(catalogue['gal_name']=='NGC2835') & (catalogue['overlap']=='partial')]
tmp = catalogue[catalogue['robust'] & (catalogue['fesc']<0.) ]
tmp = catalogue[catalogue['robust'] & (catalogue['fesc']+catalogue['fesc_err']<0.) ]
#tmp = tmp[:95]

print(f'{len(tmp)} objects in sample')

size=8*u.arcsec
nrows=5
ncols=4
filename = basedir/'reports'/'cutouts'/f'cutouts_negative_fesc_error'
    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                
                filename = data_ext / 'MUSE' / 'DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
            
                # nebulae mask
                filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')
                
                # finally the compact clusters
                filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'
                if not filename.is_file():
                    print(f'no compact clusters for {gal_name}')
                    clusters = None
                else:     
                    clusters = Table.read(filename)
                    clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.deg,clusters['PHANGS_DEC']*u.deg)
                    clusters.add_index('ID_PHANGS_CLUSTERS')
 
            
            
            ax = next(axes_iter)
            ax = single_cutout(ax,
                             position = row['SkyCoord_neb'],
                             image = F275,
                             mask1 = nebulae_mask,
                             mask2 = associations_mask,
                             points = clusters,
                             label = f"{row['gal_name']}: {row['region_ID']:.0f}",
                             size  = size)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        #h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        #ax.legend(h[::len(h)-3],l[::(len(l)-3)],fontsize=7,loc='center',frameon=False)
        h1 = mlines.Line2D([], [], color=tab10[0],label='HII region')
        h2 = mlines.Line2D([], [], color=tab10[1],label='association')
        h3 = mlines.Line2D([], [], lw=0,mfc='white',mec=tab10[4],marker='o',label='compact cluster')
        ax.legend(handles=[h1,h2,h3],fontsize=7,loc='center',frameon=False)        
        t = ax.text(0.07,0.87,'name: region ID/assoc ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


#### The HII region complexes

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.lines as mlines

from cluster.io import read_associations
from cluster.plot import single_cutout_complex

tmp = complexes[complexes['gal_name']=='NGC2835']

print(f'{len(tmp)} objects in sample')

size=16*u.arcsec
nrows=4
ncols=5
filename = basedir/'reports'/'cutouts'/f'HII_region_complexes'
    
width = 8.27
N = len(tmp) 
Npage = nrows*ncols-1
if N%Npage==0:
    print('sample size % subplots = 0: no subplot for legend')
Npages = int(np.ceil(N/Npage))
gal_name = None

with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = tmp[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            
            # for a new galaxy we need to read in the masks/images
            if row['gal_name'] != gal_name:
                
                gal_name = row['gal_name']
                
                # HST image for the background
                filename = data_ext / 'HST' / 'filterImages' / f'hlsp_phangs-hst_hst_wfc3-uvis_{gal_name.lower()}_f275w_v1_exp-drc-sci.fits'
                with fits.open(filename) as hdul:
                    F275 = NDData(hdul[0].data,
                                  mask=hdul[0].data==0,
                                  meta=hdul[0].header,
                                  wcs=WCS(hdul[0].header))
                
                
                filename = data_ext / 'MUSE' / 'DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
                with fits.open(filename) as hdul:
                    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                                    meta=hdul['HA6562_FLUX'].header,
                                    wcs=WCS(hdul['HA6562_FLUX'].header))
            
                # nebulae mask
                filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
                with fits.open(filename) as hdul:
                    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
                    nebulae_mask.data[nebulae_mask.data==-1] = np.nan
                
                # association mask
                associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                                      target=gal_name.lower(),
                                                      scalepc=scalepc,
                                                      data='mask')
                
                # finally the compact clusters
                filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'
                if not filename.is_file():
                    print(f'no compact clusters for {gal_name}')
                    clusters = None
                else:     
                    clusters = Table.read(filename)
                    clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.deg,clusters['PHANGS_DEC']*u.deg)
                    clusters.add_index('ID_PHANGS_CLUSTERS')
            
            ax = next(axes_iter)
            ax = single_cutout_complex(ax,
                             position = row['SkyCoord'],
                             image = F275,
                             nebulae_mask = nebulae_mask,
                             associations_mask = associations_mask,
                             points = clusters,
                             region_IDs = row['region_IDs'],
                             label = f"{row['gal_name']}: {row['complex_ID']:.0f}",
                             size  = size)

        plt.subplots_adjust(wspace=-0.01, hspace=0.05)

        # only the last page has subplots that need to be removed
        #h,l = fig.axes[0].get_legend_handles_labels()
        ax = next(axes_iter)
        ax.axis('off')
        #ax.legend(h[::len(h)-3],l[::(len(l)-3)],fontsize=7,loc='center',frameon=False)
        h1 = mlines.Line2D([], [], color=tab10[0],label='HII region')
        h2 = mlines.Line2D([], [], color=tab10[1],label='association')
        h3 = mlines.Line2D([], [], lw=0,mfc='white',mec=tab10[4],marker='o',label='compact cluster')
        ax.legend(handles=[h1,h2,h3],fontsize=7,loc='center',frameon=False)        
        t = ax.text(0.07,0.87,'name: complex ID', transform=ax.transAxes,color='black',fontsize=8)

        if i == int(np.ceil(N/Npage))-1:

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


#### the nice plot for the paper

In [ ]:
from cluster.io import read_associations, ReadHST

gal_name = 'NGC2835'

filename = data_ext / 'MUSE' / 'DR2.1' / 'MUSEDAP' / f'{gal_name}_MAPS.fits'
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))

# nebulae mask
filename = data_ext / 'Products' / 'Nebulae_catalogs' / 'Nebulae_catalogue_v2' /'spatial_masks'/f'{gal_name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

hst_images = ReadHST(gal_name,data_ext / 'HST' / 'filterImages' )
hst_images.Halpha = Halpha

# association mask
associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',
                                      target=gal_name.lower(),
                                      scalepc=scalepc,
                                      data='mask')

# finally the compact clusters
filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{gal_name.lower()}_phangs-hst_v1p1_ml_class12.fits'    
clusters = Table.read(filename)
clusters['SkyCoord'] = SkyCoord(clusters['PHANGS_RA']*u.deg,clusters['PHANGS_DEC']*u.deg)
clusters.add_index('ID_PHANGS_CLUSTERS')

In [ ]:
from cluster.plot.cutouts import multi_cutout_hst

# also not bad: 318, 301
# 27 an HII region with compact cluster but no association
positions = nebulae[(nebulae['gal_name']=='NGC2835') & np.isin(nebulae['region_ID'],[47,53,102,375])]['SkyCoord_neb']

zipped_pairs = zip([2,1,4,3,6], positions)
positions = [x for _, x in sorted(zipped_pairs)]
# [83, 723]
positions += list(complexes[(complexes['gal_name']==gal_name) & np.isin(complexes['complex_ID'],[114])]['SkyCoord'])

filename = basedir/'reports'/'ionising_sources.pdf'

multi_cutout_hst(positions = positions,
             images=hst_images,
             nebulae_mask = nebulae_mask,
             associations_mask = associations_mask,
             points= clusters,
             complexes=[[],[],[],[],[521, 114, 846]],
             labels= list(map(str,range(1,len(positions)+1))),
             scalebar=('100 pc',0.2109),
             size = 8*u.arcsecond,
             width = single_column,
             filename=filename,
             ncols=3)

In [ ]:
# calculate the length of the scalebar
cutout_size = 8*u.arcsec
distance = 12.22*u.Mpc
scalebar_length = 100 # in parsec
scalebar_length / (distance*(cutout_size).to(u.rad).value).to(u.pc).value

## Population Synthesis models

### Plot model predictions

In [ ]:
from cluster.io import read_bpass

#GENEVASTD = Cluster(stellar_model='GENEVASTD',metallicity=0.014)
#GENEVAHIGH = Cluster(stellar_model='GENEVAHIGH',metallicity=0.014)
#PADOVASTD = Cluster(stellar_model='PADOVASTD',metallicity=0.014)
#PADOVAAGB = Cluster(stellar_model='PADOVAAGB',metallicity=0.014)
GENEVAv00 = Cluster(stellar_model='GENEVAv00',metallicity=0.014)
GENEVAv40 = Cluster(stellar_model='GENEVAv40',metallicity=0.014)

bc03 = Table.read(basedir/'data'/'external'/'bc03_q.fits')
bpass = read_bpass(metallicity='z020')

In [ ]:
GENEVAv00_04  = Cluster(stellar_model='GENEVAv00',metallicity=0.04)
GENEVAv00_014 = Cluster(stellar_model='GENEVAv00',metallicity=0.014)
GENEVAv00_008 = Cluster(stellar_model='GENEVAv00',metallicity=0.008)
GENEVAv00_002 = Cluster(stellar_model='GENEVAv00',metallicity=0.002)
GENEVAv00_001 = Cluster(stellar_model='GENEVAv00',metallicity=0.001)

In [ ]:
fig,(ax1,ax2)=plt.subplots(figsize=(single_column,single_column*2/1.618),nrows=2,sharex=True)

cmap = mpl.cm.get_cmap('cividis_r',5)
bins = np.array([0,0.0015,0.005,0.011,0.032,0.048])
norm = mpl.colors.BoundaryNorm(boundaries=bins,ncolors=5)

#ax1.plot(GENEVASTD.quanta['Time'].value/1e6,GENEVASTD.quanta['HI_rate'].value,label='GENEVASTD')
#ax1.plot(GENEVAHIGH.quanta['Time'].value/1e6,GENEVAHIGH.quanta['HI_rate'].value,label='GENEVAHIGH')
#ax1.plot(PADOVASTD.quanta['Time'].value/1e6,PADOVASTD.quanta['HI_rate'].value,label='PADOVASTD')
#ax1.plot(PADOVAAGB.quanta['Time'].value/1e6,PADOVAAGB.quanta['HI_rate'].value,label='PADOVAAGB')#ax.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value,label='GENEVAv00')
ax1.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value,label='GENEVAv00',ls='-',color=cmap(norm((0.014))))
ax1.plot(GENEVAv40.quanta['Time'].value/1e6,GENEVAv40.quanta['HI_rate'].value,label='GENEVAv40',ls='--',color=cmap(norm((0.014))))
ax1.plot(bpass['age'].value/1e6,bpass['Q'].value,label='BPASS',ls=':',color=cmap(norm((0.014))))
ax1.plot(bc03['sfh.age'],bc03['stellar.n_ly_young']*1e6,label='CIGALE',ls='-.',color=cmap(norm((0.014))))

ax1.legend(handlelength=2)
ax1.set(xlim=[0.5,10],xscale='linear',
       ylim=[1e50,5e53],yscale='log',ylabel='$Q(\mathrm{H}^0)$ / s$^{-1}$')
 
ax2.plot(GENEVAv00_04.quanta['Time'].value/1e6,GENEVAv00_04.quanta['HI_rate'].value,color=cmap(norm((0.04))))
ax2.plot(GENEVAv00_014.quanta['Time'].value/1e6,GENEVAv00_014.quanta['HI_rate'].value,color=cmap(norm((0.014))))
ax2.plot(GENEVAv00_008.quanta['Time'].value/1e6,GENEVAv00_008.quanta['HI_rate'].value,color=cmap(norm((0.008))))
ax2.plot(GENEVAv00_002.quanta['Time'].value/1e6,GENEVAv00_002.quanta['HI_rate'].value,color=cmap(norm((0.002))))
ax2.plot(GENEVAv00_001.quanta['Time'].value/1e6,GENEVAv00_001.quanta['HI_rate'].value,color=cmap(norm((0.001))))


ax2.set(xlim=[0.5,10.5],xscale='linear',ylim=[1e50,5e53],yscale='log',
        ylabel='$Q(\mathrm{H}^0)$ / s$^{-1}$',xlabel=r'age / Myr')
    
plt.subplots_adjust(hspace=0.05)
    
fig.subplots_adjust(top=0.85)
cbar_ax = fig.add_axes([0.13, 0.88, 0.76, 0.02])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),spacing='uniform',
                    cax=cbar_ax,orientation='horizontal',ticklocation='top',label=r'$Z$',
                    ticks=(bins[1:]+bins[:-1])/2)
cbar.ax.set_xticklabels(np.array([0.001,0.002,0.008,0.014,0.04]))    

#plt.savefig(basedir/'reports'/'thesis'/'impact_of_models.pdf',dpi=300)
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(figsize=(two_column,two_column/2),ncols=2,sharey=True)

cmap = mpl.cm.get_cmap('cividis_r',5)
bins = np.array([0,0.0015,0.005,0.011,0.032,0.048])
norm = mpl.colors.BoundaryNorm(boundaries=bins,ncolors=5)

#ax1.plot(GENEVASTD.quanta['Time'].value/1e6,GENEVASTD.quanta['HI_rate'].value,label='GENEVASTD')
#ax1.plot(GENEVAHIGH.quanta['Time'].value/1e6,GENEVAHIGH.quanta['HI_rate'].value,label='GENEVAHIGH')
#ax1.plot(PADOVASTD.quanta['Time'].value/1e6,PADOVASTD.quanta['HI_rate'].value,label='PADOVASTD')
#ax1.plot(PADOVAAGB.quanta['Time'].value/1e6,PADOVAAGB.quanta['HI_rate'].value,label='PADOVAAGB')#ax.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value,label='GENEVAv00')
ax1.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value,label='SB99 v00',ls='-',color=cmap(norm((0.014))))
ax1.plot(GENEVAv40.quanta['Time'].value/1e6,GENEVAv40.quanta['HI_rate'].value,label='SB99 v40',ls='--',color=cmap(norm((0.014))))
ax1.plot(bpass['age'].value/1e6,bpass['Q'].value,label='BPASS',ls=':',color=cmap(norm((0.014))))
ax1.plot(bc03['sfh.age'],bc03['stellar.n_ly_young']*1e6,label='CIGALE',ls='-.',color=cmap(norm((0.014))))

ax1.legend(handlelength=2)
ax1.set(xlim=[0.5,10],xscale='linear',xlabel=r'age / Myr',
       ylim=[1e50,5e53],yscale='log',ylabel='$Q(\mathrm{H}^0)$ / s$^{-1}$')
 
ax2.plot(GENEVAv00_04.quanta['Time'].value/1e6,GENEVAv00_04.quanta['HI_rate'].value,color=cmap(norm((0.04))))
ax2.plot(GENEVAv00_014.quanta['Time'].value/1e6,GENEVAv00_014.quanta['HI_rate'].value,color=cmap(norm((0.014))))
ax2.plot(GENEVAv00_008.quanta['Time'].value/1e6,GENEVAv00_008.quanta['HI_rate'].value,color=cmap(norm((0.008))))
ax2.plot(GENEVAv00_002.quanta['Time'].value/1e6,GENEVAv00_002.quanta['HI_rate'].value,color=cmap(norm((0.002))))
ax2.plot(GENEVAv00_001.quanta['Time'].value/1e6,GENEVAv00_001.quanta['HI_rate'].value,color=cmap(norm((0.001))))


ax2.set(xlim=[0.5,10.5],xscale='linear',ylim=[1e50,5e53],yscale='log',xlabel=r'age / Myr')
    
fig.subplots_adjust(wspace=0.05,right=0.9)
cbar_ax = fig.add_axes([0.92, 0.13, 0.02, 0.75])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),spacing='uniform',
                    cax=cbar_ax,orientation='vertical',label=r'$Z$',
                    ticks=(bins[1:]+bins[:-1])/2)
cbar.ax.set_yticklabels(np.array([0.001,0.002,0.008,0.014,0.04]))    

plt.savefig(basedir/'reports'/'slides'/'impact_of_models.png',dpi=300)
plt.show()

### BPASS

https://flexiblelearning.auckland.ac.nz/bpass/8/files/bpassv2_1_manual.pdf

In [ ]:
from cluster.io import read_bpass

BPASS_folder = basedir/'..'/'BPASS'

bpass = read_bpass(BPASS_folder,metallicity='z020')

In [ ]:
fig,ax=plt.subplots()

ax.plot(bpass['age'],bpass['Halpha']/bpass['FUV'])
ax.set(xlim=[1e6,1e7],xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')
plt.show()

In [ ]:
from tqdm import tqdm

catalogue['Qpredicted'] = np.nan
HI_rate = bpass['Q'].value
time = bpass['age']
for row in tqdm(catalogue):
    idx = np.argmin(np.abs(time-row['age']*u.Myr))
    row['Qpredicted'] = HI_rate[idx] * row['mass'] / 1e6
    
catalogue['distance'] = np.nan
for gal_name in catalogue['gal_name']:
    distance = Distance(distmod=sample_table.loc[gal_name]['(m-M)'])
    catalogue['distance'][catalogue['gal_name']==gal_name] = distance
    
catalogue['L(Ha)'] = (catalogue['HA6562_FLUX_CORR']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*(catalogue['distance']*u.Mpc)**2).to(u.erg/u.s)
catalogue['Qobserved'] = 7.31e11*catalogue['L(Ha)']/u.erg
fesc_classic = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc_classic
catalogue['HA/FUV_corr'] = catalogue['HA/FUV'] / (1-fesc_classic)
catalogue['eq_width_corr'] = catalogue['eq_width'] / (1-fesc_classic)

print(f'fesc={np.nanmean(fesc_classic[fesc_classic>0]):.2f}+-{np.nanstd(fesc_classic[fesc_classic>0]):.2f} (from {np.sum(fesc_classic>0)} objects)')
print(f"{np.sum(fesc_classic<0)} of {len(catalogue)} ({np.sum(fesc_classic<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
# for objects with negative fesc we redo the analysis with age-age_err
Qpredict_new = []
for row in tqdm(catalogue):
    if row['fesc']<0:
        idx = np.argmin(np.abs(time-(row['age']-3*row['age_err'])*u.Myr))
        row['Qpredicted'] = ( HI_rate[idx] * row['mass'] / 1e6 )

fesc_classic = (catalogue['Qpredicted']-catalogue['Qobserved'])/catalogue['Qpredicted']
catalogue['fesc'] = fesc_classic
catalogue['HA/FUV_corr'] = catalogue['HA/FUV'] / (1-fesc_classic)
catalogue['eq_width_corr'] = catalogue['eq_width'] / (1-fesc_classic)

print(f'fesc={np.nanmean(fesc_classic[fesc_classic>0]):.2f}+-{np.nanstd(fesc_classic[fesc_classic>0]):.2f} (from {np.sum(fesc_classic>0)} objects)')
print(f"{np.sum(fesc_classic<0)} of {len(catalogue)} ({np.sum(fesc_classic<0)/len(catalogue)*100:.1f}%) regions have negative fesc")

In [ ]:
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.1) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= fesc>0
tmp = catalogue[criteria].copy()

print(f'fesc={np.nanmean(tmp[tmp["fesc"]>0]["fesc"]):.2f} (from {np.sum(criteria)} objects)')
print(f"{np.sum(tmp['fesc']<0)} of {len(tmp)} ({np.sum(tmp['fesc']<0)/len(tmp)*100:.1f}%) regions have negative fesc")

### SLUG

The SLUG output contains the full PDF of each property. Here we compute the median and uncertainty and save it a new and smaller file.

In [ ]:
# we need the raw association catalogue that Jia Wei used in SLUG
#with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
#    associations = Table(hdul[1].data)
    
mtab = np.genfromtxt(basedir/'data'/'external'/'SLUG'/'mtab_PHANGS_FS.tab', skip_header = 1)
atab = np.genfromtxt(basedir/'data'/'external'/'SLUG'/'agetab_PHANGS_FS.tab', skip_header = 1)
qtab = np.genfromtxt(basedir/'data'/'external'/'SLUG'/'qtab_PHANGS_FS.tab', skip_header = 1)
avtab = np.genfromtxt(basedir/'data'/'external'/'SLUG'/'avtab_PHANGS_FS.tab', skip_header = 1)

In [ ]:
cum = np.cumsum(mtab[1:],axis=1)/np.sum(mtab[1:],axis=1)[:,np.newaxis]
m_min = 10**mtab[0][np.argmin(np.abs(cum-0.16),axis=1)]
m_med = 10**mtab[0][np.argmin(np.abs(cum-0.5),axis=1)]
m_max = 10**mtab[0][np.argmin(np.abs(cum-0.84),axis=1)]

cum = np.cumsum(atab[1:],axis=1)/np.sum(atab[1:],axis=1)[:,np.newaxis]
a_min = 10**atab[0][np.argmin(np.abs(cum-0.16),axis=1)] / 1e6
a_med = 10**atab[0][np.argmin(np.abs(cum-0.5),axis=1)] / 1e6
a_max = 10**atab[0][np.argmin(np.abs(cum-0.84),axis=1)] / 1e6

cum = np.cumsum(avtab[1:],axis=1)/np.sum(avtab[1:],axis=1)[:,np.newaxis]
av_min = 10**avtab[0][np.argmin(np.abs(cum-0.16),axis=1)]
av_med = 10**avtab[0][np.argmin(np.abs(cum-0.5),axis=1)]
av_max = 10**avtab[0][np.argmin(np.abs(cum-0.84),axis=1)]

cum = np.cumsum(qtab[1:],axis=1)/np.sum(qtab[1:],axis=1)[:,np.newaxis]
q_min = 10**qtab[0][np.argmin(np.abs(cum-0.16),axis=1)]
q_med = 10**qtab[0][np.argmin(np.abs(cum-0.5),axis=1)]
q_max = 10**qtab[0][np.argmin(np.abs(cum-0.84),axis=1)]

In [ ]:
with fits.open(basedir/'data'/'interim'/f'phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits') as hdul:
    SLUG_table= Table(hdul[1].data)['gal_name','assoc_ID']

SLUG_table['mass_SLUG'] = m_med
SLUG_table['mass_SLUG_err_plus'] = m_max - m_med
SLUG_table['mass_SLUG_err_minus'] = m_med - m_min

SLUG_table['age_SLUG'] = a_med
SLUG_table['age_SLUG_err_plus'] = a_max - a_med
SLUG_table['age_SLUG_err_minus'] = a_med - a_min

SLUG_table['av_SLUG'] = av_med
SLUG_table['av_SLUG_err_plus'] = av_max - av_med
SLUG_table['av_SLUG_err_minus'] = av_med - av_min

SLUG_table['q_SLUG'] = q_med
SLUG_table['q_SLUG_err_plus'] = q_max - q_med
SLUG_table['q_SLUG_err_minus'] = q_med - q_min

mask = []
for pdf in qtab[1:]:
    if np.all(np.ediff1d(pdf)==0):
        mask.append(True)
    else:
        mask.append(False)
for col in SLUG_table.columns[2:]:
    SLUG_table[col][mask] = np.nan

SLUG_table.write(basedir/'data'/'interim'/'associations_SLUG.fits',overwrite=True)

In [ ]:
from scipy.stats import binned_statistic

fig,ax=plt.subplots()

ax.scatter(SLUG_table['age_SLUG'],SLUG_table['q_SLUG']/SLUG_table['mass_SLUG']*1e6,label='SLUG')

mean, edges, _ = binned_statistic(SLUG_table['age_SLUG'],SLUG_table['q_SLUG']/SLUG_table['mass_SLUG']*1e6,statistic='median',bins=np.linspace(0.5,20.5,40))
x = (edges[1:]+edges[:-1])/2
ax.plot(x,mean,color='gray',label='SLUG median')

ax.plot(GENEVAv00.quanta['Time'].value/1e6,GENEVAv00.quanta['HI_rate'].value,label='GENEVAv00',ls='-',color='black')

ax.legend()
ax.set(xlim=[0,20],ylim=[1e47,1e54],yscale='log',xlabel='age / Myr',ylabel='q / s-1')

plt.show()

### Compare different stellar models/population synthesis

In [ ]:
bpass_z014 = read_bpass(metallicity='z014')
bpass_z008 = read_bpass(metallicity='z008')

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(nrows=3,figsize=(two_column,two_column),sharex=True)

ax1.plot(bpass['age']/1e6,bpass['Halpha'],label='BPASSv014')
#ax1.plot(bpass_z008['age']/1e6,bpass_z008['Halpha'],label='BPASSv008')
ax1.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A'],label='GENEVAv40')
ax1.legend()
ax1.set(xlim=[0,10],ylabel=r'$\mathrm{H}\alpha \,/\, \mathrm{erg}\ \mathrm{s}^{-1}$',
        yscale='log',ylim=[1e39,2e41])

ax2.plot(bpass['age']/1e6,bpass['FUV'],label='BPASSv014')
#ax2.plot(bpass_z008['age']/1e6,bpass_z008['FUV'],label='BPASSv008')
ax2.plot(cluster.ewidth['Time']/1e6,cluster.FUV['FUV'],label='GENEVAv40')
#ax2.legend()
ax2.set(xlim=[0,10],ylabel=r'$\mathrm{FUV}\,/\, \mathrm{erg}\ \mathrm{s}^{-1}\ \mathrm{\AA}^{-1}$',
       yscale='log',ylim=[8e37,3e39])

ax3.plot(bpass['age']/1e6,bpass['Halpha']/bpass['FUV'],label='BPASSv014')
#ax3.plot(bpass_z008['age']/1e6,bpass_z008['Halpha']/bpass_z008['FUV'],label='BPASSv008')
ax3.plot(cluster.ewidth['Time']/1e6,cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV'],label='GENEVAv40')
#ax3.legend()
ax3.set(xlim=[0,10],xlabel='age / Myr',ylabel=r'$\mathrm{H}\alpha/\mathrm{FUV}$')

plt.subplots_adjust(hspace=0.)
plt.show()

## Dust from JWST

how can we disentangle the photons that are lost due to dust and those that escape the cloud.

In [ ]:
jwst_sample = ['IC5332','NGC0628','NGC1365','NGC7496']

jwst_catalogue = catalogue[np.isin(catalogue['gal_name'],jwst_sample)]

In [ ]:
lst = []
for gal_name in jwst_sample:
    tbl = Table.read(basedir/'data'/'external'/'MIRI'/f'{gal_name}_MIRI.fits')
    lst.append(tbl[['gal_name','region_ID']+[x for x in tbl.columns if x.startswith('F')]])
MIRI = vstack(lst)
jwst_catalogue = join(jwst_catalogue,MIRI,keys=['gal_name','region_ID'])

In [ ]:
tmp = jwst_catalogue

vmin,vmax = -4.5,-2.5
fig,ax=plt.subplots(figsize=(single_column,single_column))
cmap = plt.cm.get_cmap('cool')
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)

ax.scatter(tmp['fesc'],tmp['dig/hii'],c=np.log10(tmp['F2100W']),cmap=cmap,vmin=vmin,vmax=vmax,s=8,rasterized=True)

ax.set(xlim=[0.4,1],ylim=[0,1],xlabel=r'$f_\mathrm{esc}$',ylabel=r'$I_\mathrm{DIG}\,/\,I_{\mathrm{H}\,\tiny{\textsc{ii}}}$')
fig.subplots_adjust(right=0.95,wspace=0.3)
cbar_ax = fig.add_axes([0.96, 0.126, 0.02, 0.75])
#r'$21\,\mu\mathrm{m}$' r'$E(B-V)_\mathrm{Balmer}$'
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,label=r'$21\,\mu\mathrm{m}$')

plt.savefig(basedir/'reports'/'fesc_dust.pdf',dpi=300)
plt.show()

In [ ]:
from astrotools.plot.corner import corner_binned_stat

jwst_catalogue['dust_Ha'] = np.log10(jwst_catalogue['F2100W']/jwst_catalogue['HA6562_FLUX_CORR'])

tmp = jwst_catalogue #[jwst_catalogue['mass']>5e3]

vmin,vmax = -10,-7
fig,ax=plt.subplots(figsize=(single_column,single_column))
cmap = plt.cm.get_cmap('cool')
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
bins = np.linspace(0.5,1.05,5)

ax.scatter(tmp['fesc'],tmp['dig/hii'],c=tmp['dust_Ha'],cmap=cmap,vmin=vmin,vmax=vmax,s=8,rasterized=True)
#corner_binned_stat(tmp['fesc'],tmp['dig/hii'],ax=ax,bins=bins)

sub = tmp[(tmp['dust_Ha']<-9)]
corner_binned_stat(sub['fesc'],sub['dig/hii'],ax=ax,bins=bins,color=cmap(norm(-9.5)),lw=4)

sub = tmp[(tmp['dust_Ha']>=-9) & (tmp['dust_Ha']<-8)]
corner_binned_stat(sub['fesc'],sub['dig/hii'],ax=ax,bins=bins,color=cmap(norm(-8.5)),lw=4)

sub = tmp[(tmp['dust_Ha']>=-8)]
corner_binned_stat(sub['fesc'],sub['dig/hii'],ax=ax,bins=bins,color=cmap(norm(-7.5)),lw=4)

ax.set(xlim=[0.4,1],ylim=[0,1],xlabel=r'$f_\mathrm{esc}$',ylabel=r'$I_\mathrm{DIG}\,/\,I_{\mathrm{H}\,\tiny{\textsc{ii}}}$')
fig.subplots_adjust(right=0.95,wspace=0.3)
cbar_ax = fig.add_axes([0.96, 0.126, 0.02, 0.75])
#r'$21\,\mu\mathrm{m}$' r'$E(B-V)_\mathrm{Balmer}$'
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),cax=cbar_ax,label=r'$21\,\mu\mathrm{m}\,/\,\mathrm{H}\alpha$')

plt.savefig(basedir/'reports'/'fesc_dust.pdf',dpi=300)
plt.show()

## are the age trends due to fesc

In [ ]:
from cluster.auxiliary import bin_stat

fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2.2))

sc=ax1.scatter(tmp['HA/FUV'],tmp['logq_D91'],c=tmp['fesc'],vmin=0,vmax=1)
x,mean,std = bin_stat(tmp['HA/FUV'],tmp['logq_D91'],[0,80],nbins=5,statistic='median')
ax1.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax1.set(xlabel=r'H$\alpha$ / FUV',ylabel=r'$\log q$',xlim=[0,80],ylim=[6,8])

ax2.scatter(tmp['HA/FUV'],tmp['Delta_met_scal'],c=tmp['fesc'],vmin=0,vmax=1)
x,mean,std = bin_stat(tmp['HA/FUV'],tmp['Delta_met_scal'],[0,80],nbins=5,statistic='median')
ax2.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax2.set(xlabel=r'H$\alpha$ / FUV',ylabel=r'$\Delta$(O/H)',xlim=[0,80],ylim=[-0.1,0.1])

plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.3)
cbar_ax = fig.add_axes([0.87, 0.145, 0.02, 0.8])
fig.colorbar(sc,cax=cbar_ax,label=r'$f_\mathrm{esc}$')

plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2.2))

sc=ax1.scatter(tmp['HA/FUV'],tmp['logq_D91'],c=tmp['age'],vmin=0,vmax=5)
x,mean,std = bin_stat(tmp['HA/FUV'],tmp['logq_D91'],[0,80],nbins=5,statistic='median')
ax1.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax1.set(xlabel=r'H$\alpha$ / FUV',ylabel=r'$\log q$',xlim=[0,80],ylim=[6,8])

ax2.scatter(tmp['HA/FUV'],tmp['Delta_met_scal'],c=tmp['age'],vmin=0,vmax=5)
x,mean,std = bin_stat(tmp['HA/FUV'],tmp['Delta_met_scal'],[0,80],nbins=5,statistic='median')
ax2.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax2.set(xlabel=r'H$\alpha$ / FUV',ylabel=r'$\Delta$(O/H)',xlim=[0,80],ylim=[-0.1,0.1])

plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.3)
cbar_ax = fig.add_axes([0.87, 0.145, 0.02, 0.8])
fig.colorbar(sc,cax=cbar_ax,label=r'age / Myr')

plt.show()

In [ ]:
# https://stackoverflow.com/questions/20105364/how-can-i-make-a-scatter-plot-colored-by-density-in-matplotlib
from scipy.stats import gaussian_kde

fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

x,y = tmp['HA/FUV'],tmp['fesc']

x,y = x[~np.isnan(x) & ~np.isnan(y)], y[~np.isnan(x) & ~np.isnan(y)]
xy = np.vstack([x,y])
density = gaussian_kde(xy)(xy)

ax.scatter(x,y,cmap=plt.cm.Reds)
bins,mean,std = bin_stat(tmp['HA/FUV'],tmp['fesc'],[0,80],nbins=8,statistic='median')
ax.errorbar(bins,mean,fmt='o-',color='black')
ax.set(xlim=[0,80],ylim=[0,1.2],
       xlabel=r'H$\alpha$ / FUV',
       ylabel=r'$f_\mathrm{esc}$')
plt.show()

In [ ]:
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

ax1.scatter(tmp['age'],tmp['fesc'])
bins,mean,std = bin_stat(tmp['age'],tmp['fesc'],[0.5,10.5],nbins=10,statistic='median')
ax1.errorbar(bins,mean,fmt='o-',color='black')
ax1.set(xlim=[0,10],ylim=[0,1],
       xlabel=r'age / Myr',
       ylabel=r'$f_\mathrm{esc}$')

x,y = tmp['HA/FUV'],tmp['fesc']
ax2.scatter(x,y,cmap=plt.cm.Reds)
bins,mean,std = bin_stat(tmp['HA/FUV'],tmp['fesc'],[0,80],nbins=8,statistic='median')
ax2.errorbar(bins,mean,fmt='o-',color='black')
ax2.set(xlim=[0,80],ylim=[0,1],
       xlabel=r'H$\alpha$ / FUV',
       ylabel=r'$f_\mathrm{esc}$')

plt.tight_layout()

plt.savefig(basedir/'reports'/'trends_due_to_fesc.png',dpi=400)

plt.show()

## Search for correlations

### Ha/FUV vs ionization

with log q from Diaz+91

$$
\log u = (-1.684±0.076)\cdot \log([SII]/[SIII])-(2.986 ±0.027)
$$

In [ ]:
from scipy.stats import binned_statistic, spearmanr
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat
from astrotools.metallicity.ionization import logq_D91, logq_D91_reverse

bins = np.linspace(*np.nanpercentile(catalogue['logq_D91'],[1,99]),10)
xlim = [5.7,7.8]

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = 'logq_D91','eq_width','HA/FUV_corr','Delta_met_scal'

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3))

table = nebulae.copy()
table = table[(table['HA/FUV_corr']>3*table['HA/FUV_corr_err']) | np.isnan(table['FUV_FLUX'])]
#table = table[table['[SIII]/[SII]']>3*table['[SIII]/[SII]_err']]
print(f'{len(table)} objects in sample')

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)
sample_table.sort('mass')

rho1,rho2,rho3 = [], [], []
for i,gal_name in enumerate(sample_table['name']):
    #print(gal_name)
    
    tmp = table[table['gal_name']==gal_name]

    x,mean,std = bin_stat(tmp[x_name],tmp[y1_name],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
    rho1.append(spearmanr(tmp[x_name],tmp[y1_name],nan_policy='omit')[0])
    
    x,mean,std = bin_stat(tmp[x_name],tmp[y2_name],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
    try:
        rho2.append(spearmanr(tmp[x_name],tmp[y2_name],nan_policy='omit')[0])
    except:
        pass
    
    x,mean,std = bin_stat(tmp[x_name],tmp[y3_name],[None,None],nbins=bins,statistic='median')
    ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
    rho3.append(spearmanr(tmp[x_name],tmp[y3_name],nan_policy='omit')[0])

# plot contours

for ax,y_name in zip([ax1,ax2,ax3],[y1_name,y2_name,y3_name]):
    
    x,y = table[x_name],table[y_name]
    #ax1.scatter(x,y,s=0.5,color='black')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]

    corner_binned_percentile(x,y,ax,nbins=bins,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=bins,n=98,color='gray',alpha=0.3)
    

    x,mean,std = bin_stat(table[x_name],table[y_name],[None,None],nbins=bins,statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=9.4,vmax=11)

ax1.set(xlim=xlim, yscale='log',ylim=[3,5e2],
        xlabel=r'$\log q$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')
ax2.set(xlim=xlim, yscale='log',ylim=[2,2e2],
        xlabel=r'$\log q$',ylabel=r'H$\alpha$/FUV')
ax3.set(xlim=xlim,ylim=[-0.15,0.15],
        xlabel=r'$\log q$',ylabel=r'$\Delta$(O/H)')

ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax3.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))

# get a second x axis on top with [SIII]/[SII]
xmin,xmax=ax1.get_xlim()
ax1_top = ax1.twiny()
ax1_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax1_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax1_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

ax2_top = ax3.twiny()
ax2_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax2_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax2_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

ax3_top = ax2.twiny()
ax3_top.set(xlim=[logq_D91_reverse(xmin),logq_D91_reverse(xmax)],xscale='log')
ax3_top.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
ax3_top.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

for i, rho in enumerate([rho1,rho2,rho3]):
    print(f'plot{i}: rho={np.nanmean(rho):.2f}')

plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.6])
fig.colorbar(sc,cax=cbar_ax,label=r'$\log (m/\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

plt.savefig(basedir/'reports'/'nebulae_correlations.pdf',dpi=600)
#plt.savefig(basedir/'reports'/'nebulae_correlations.png',dpi=600)

plt.show()

histogram/binned stat with log bins

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = '[SIII]/[SII]', 'Delta_met_scal','HA/FUV_corr','eq_width'
xlim,ylim1,ylim2,ylim3 = [1e-2,1],[-0.15,0.15],[8e-1,8e1],[3,3e2]

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3.2))

table = nebulae.copy()
table = table[(table['HA/FUV_corr']>3*table['HA/FUV_corr_err']) | np.isnan(table['FUV_FLUX'])]
table = table[table['[SIII]/[SII]']>3*table['[SIII]/[SII]_err']]

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)
sample_table.sort('mass')

# the density histogram

nbins=20

x,y = table[x_name],table[y1_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.linspace(*ylim1,nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax1.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y2_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim2),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax2.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y3_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim3),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax3.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)


# the bins for the binned mean
#bins = np.logspace(-1.9,-0.2,10)
bins = np.logspace(-2.4,-0.4,12)

for i,gal_name in enumerate(sample_table['name']):
    #print(gal_name)
    
    tmp = table[table['gal_name']==gal_name]

    x,mean,std = bin_stat(tmp[x_name],tmp[y1_name],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(tmp[x_name],tmp[y2_name],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(tmp[x_name],tmp[y3_name],[None,None],nbins=bins,statistic='median')
    ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

# plot contours

for ax,y_name in zip([ax1,ax2,ax3],[y1_name,y2_name,y3_name]):
    
    x,y = table[x_name],table[y_name]
    #ax1.scatter(x,y,s=0.5,color='black')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]
    
    x,mean,std = bin_stat(table[x_name],table[y_name],[None,None],nbins=bins,statistic='median')
    ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=9.4,vmax=11)

ax1.set(xscale='log',xlim=xlim,ylim=ylim1,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
ax2.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim2,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'H$\alpha$/FUV')

ax3.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim3,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')


ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   


plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.73])
fig.colorbar(sc,cax=cbar_ax,label=r'$\log (m/\mathrm{M}_\odot)$',ticks=np.arange(9.4,11.4,0.4))

#plt.savefig(basedir/'reports'/'nebulae_correlations.pdf',dpi=600)

plt.show()

the same plot but now binned by age

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.corner import corner_binned_percentile
from astrotools.plot.utils import bin_stat

q1 = 68
q2 = 98

x_name,y1_name,y2_name,y3_name = '[SIII]/[SII]', 'Delta_met_scal','HA/FUV','eq_width'
xlim,ylim1,ylim2,ylim3 = [1e-2,1],[-0.15,0.15],[8e-1,8e1],[3,3e2]

fig, (ax1,ax2,ax3) = plt.subplots(ncols=3,figsize=(two_column,two_column/3.2))

table = catalogue.copy()
#table = table[(table['FUV_FLUX_CORR']>3*table['FUV_FLUX_CORR_ERR']) | np.isnan(table['FUV_FLUX_CORR'])]
table = table[table['[SIII]/[SII]']>3*table['[SIII]/[SII]_err']]

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=0,vmax=20)


nbins=20

x,y = table[x_name],table[y1_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.linspace(*ylim1,nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax1.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y2_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim2),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax2.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)

x,y = table[x_name],table[y3_name]
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim3),nbins)]
hist, x_e, y_e = np.histogram2d(x,y,bins=bins,density=True)
ax3.pcolormesh(x_e,y_e,hist.T,cmap=plt.cm.gray_r)


# the bins for the binned mean
#bins = np.logspace(-1.9,-0.2,10)
bins = np.logspace(-2.4,-0.4,12)


tmp = catalogue #[catalogue['mass']>1e4]

for age in [(0,2),(2,5),(5,10),(10,20)]:
    age_mean = np.mean(age)
    tmp = table[(table['age']>age[0]) & (table['age']<age[1])]
    print(f'{age_mean}: {len(tmp)} objects')
    x,mean,std = bin_stat(tmp[x_name],tmp[y1_name],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

    x,mean,std = bin_stat(tmp[x_name],tmp[y2_name],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

    x,mean,std = bin_stat(tmp[x_name],tmp[y3_name],[None,None],nbins=bins,statistic='median')
    ax3.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_mean)),label=age_mean)

# plot contours

for ax,y_name in zip([ax1,ax2,ax3],[y1_name,y2_name,y3_name]):
    
    x,y = table[x_name],table[y_name]
    #ax1.scatter(x,y,s=0.5,color='black')

    # just ignore nan values
    x = x[~np.isnan(y) & np.isfinite(y)]
    y = y[~np.isnan(y) & np.isfinite(y)]
    
    x,mean,std = bin_stat(table[x_name],table[y_name],[None,None],nbins=bins,statistic='median')
    #ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')

    
sc = ax1.scatter(19*[1],19*[1],c=sample_table['mass'],cmap=cmap,vmin=0,vmax=12)

ax1.set(xscale='log',xlim=xlim,ylim=ylim1,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
ax2.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim2,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'H$\alpha$/FUV')

ax3.set(xscale='log',xlim=xlim, yscale='log',ylim=ylim3,
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$')


ax1.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax1.yaxis.set_major_locator(mpl.ticker.MultipleLocator(0.1))
ax1.yaxis.set_minor_locator(mpl.ticker.MultipleLocator(0.01))
ax2.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax2.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   
ax3.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
ax3.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))   


plt.tight_layout()
fig.subplots_adjust(right=0.85,wspace=0.4)
cbar_ax = fig.add_axes([0.87, 0.21, 0.02, 0.73])
fig.colorbar(sc,cax=cbar_ax,label=r'age / Myr',ticks=np.arange(0,20,4))

#plt.savefig(basedir/'reports'/'nebulae_correlations_age_bin.pdf',dpi=600)

plt.show()

a dedicated subplot for each galaxie

In [ ]:
from scipy.stats import binned_statistic, pearsonr, spearmanr

sample = set(astrosat_sample) & set(muse_sample)
filename = basedir/'reports'/'all_objects_HaFUV_over_SII'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 4
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
    
#vmin,vmax = np.min(catalogue['HA6562_FLUX_CORR']),np.max(catalogue['HA6562_FLUX_CORR'])
vmin,vmax = 1e-16,1e-14
# loop over the galaxies we want to plot
for name in sorted(sample):  
    
    tmp = nebulae[(nebulae['gal_name']==name)]
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    tmp = tmp[(tmp['HA/FUV_corr']>3*tmp['HA/FUV_corr_err']) | np.isnan(tmp['FUV_FLUX'])]
    tmp = tmp[tmp['[SIII]/[SII]']>3*tmp['[SIII]/[SII]_err']]

    r,p = spearmanr(tmp['[SIII]/[SII]'],tmp['HA/FUV_corr'])
    print(f'{name}: rho={r:.2f}, {len(tmp)} objects')

    sc = ax.scatter(tmp['[SIII]/[SII]'],tmp['HA/FUV_corr'],
               c=1e-20*tmp['HA6562_FLUX_CORR'],vmin=vmin,vmax=vmax,
               cmap=plt.cm.plasma,
               norm=mpl.colors.LogNorm(),
               s=1,marker='.')
    
    #q = 98
    #bins = np.logspace(*np.log10(np.percentile(tmp['[SIII]/[SII]'],[100-q,q])),10)
    #x,mean,std = bin_stat(tmp['[SIII]/[SII]'],tmp['HA/FUV'],[None,None],nbins=bins)
    #ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label=gal_name)
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.75,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.62,0.05,f'{len(tmp):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,4e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    
    if i==0:
        #ax.remove()
        ax.axis('off')
        cbar = fig.colorbar(sc, ax=ax,
                            label=r'$\mathrm{H}\alpha$ / (erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$)',
                            orientation='horizontal',
                           )
    else:
        ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$')


#plt.savefig(filename.with_suffix('.png'),dpi=600)
#plt.savefig(filename.with_suffix('.pdf'),dpi=600)

plt.show()


In [ ]:
tmp = nebulae[(nebulae['gal_name']=='NGC4254')]
#tmp = tmp[(tmp['HA/FUV_corr']>3*tmp['HA/FUV_corr_err']) | np.isnan(tmp['FUV_FLUX'])]
tmp = tmp[tmp['[SIII]/[SII]']>3*tmp['[SIII]/[SII]_err']]

tmp[['HA/FUV_corr','HA/FUV_corr_err']]

### age histograms

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0.5,10.5,1)

tmp = associations[(associations['mass']>1e4) & (associations['age']<=10)]
print(len(tmp))
ages_con = tmp[tmp['overlap']=='contained']['age']
ages_par = tmp[tmp['overlap']=='partial']['age']
ages_iso = tmp[(tmp['overlap']=='isolated') & (tmp['in_frame'])]['age']

print(f'ages: con={np.mean(ages_con):.2f}, par={np.mean(ages_par):.2f}, iso={np.mean(ages_iso):.2f}')

n1,_,_=ax1.hist(ages_con,bins=bins,histtype='step',label='contained')
n2,_,_=ax2.hist(ages_par,bins=bins,histtype='step',label='partially')
n3,_,_=ax3.hist(ages_iso,bins=bins,histtype='step',label='isolated')

ax1.set_title(f'contained ({np.nanmean(ages_con):.2f} Myr)')
ax2.set_title(f'partial ({np.nanmean(ages_par):.2f} Myr)')
ax3.set_title(f'isolated ({np.nanmean(ages_iso):.2f} Myr)')

ymax = np.max([n1,n2,n3])
ymax = np.round(ymax,-int(np.log10(ymax))+1)
for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,ymax],xlim=[0,10],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'tmp_age_hist_contained.png',dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky

tmp = associations[(associations['mass']>1e4)]
idx,sep,_=match_coordinates_sky(tmp['SkyCoord_asc'],nebulae['SkyCoord_neb'])

ages1 = tmp[(sep<0.4*u.arcsec)]['age']
ages2 = tmp[(sep>0.4*u.arcsec) & (sep<0.8*u.arcsec)]['age']
ages3 = tmp[(sep>0.8*u.arcsec)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'$s<0.4"$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'$0.4"<s<0.8"$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'$0.8"<s$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,1100],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_sep.png',dpi=600)

plt.show()

In [ ]:
tmp = catalogue[(catalogue['mass']>1e4) & (catalogue['age']<10)]

p1,p2=np.nanpercentile(tmp['eq_width'],[33,66])

ages1 = tmp[tmp['eq_width']<p1]['age']
ages2 = tmp[(tmp['eq_width']>p1) & (tmp['eq_width']<p2)]['age']
ages3 = tmp[(tmp['eq_width']>p2)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'1$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'2$^\mathrm{nd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'3$^\mathrm{rd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,160],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_eq_width.png',dpi=600)

plt.show()

In [ ]:
tmp = catalogue[(catalogue['mass']>1e4)]
#catalogue[(catalogue['mass']>1e4) & (catalogue['age']<10)]

p1,p2=np.nanpercentile(tmp['eq_width'],[33,66])

ages1 = tmp[tmp['eq_width']<p1]['age']
ages2 = tmp[(tmp['eq_width']>p1) & (tmp['eq_width']<p2)]['age']
ages3 = tmp[(tmp['eq_width']>p2)]['age']

print(f'mean age: 1={np.mean(ages1):.2f}, 2={np.mean(ages2):.2f}, 3={np.mean(ages3):.2f} Myr')

fig,(ax1,ax2,ax3)=plt.subplots(ncols=3,figsize=(10,3))
bins = np.arange(0,10,1)

ax1.hist(ages1,bins=bins,histtype='step',label='isolated')
ax2.hist(ages2,bins=bins,histtype='step',label='partially')
ax3.hist(ages3,bins=bins,histtype='step',label='contained')
ax1.set_title(r'1$^\mathrm{st}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.mean(ages1):.2f} Myr)')
ax2.set_title(r'2$^\mathrm{nd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$' +f' ({np.mean(ages2):.2f} Myr)')
ax3.set_title(r'3$^\mathrm{rd}$ percentile in $\mathrm{EW}(\mathrm{H}\alpha)$'+f' ({np.mean(ages3):.2f} Myr)')

for ax in [ax1,ax2,ax3]:
    ax.set(ylim=[0,150],xlabel='age / Myr')
plt.savefig(basedir/'reports'/f'age_hist_eq_width.png',dpi=600)

plt.show()

### Mass-to-Halpha

In [ ]:
from astrotools.plot.utils import bin_stat
from scipy.stats import spearmanr, gaussian_kde, binned_statistic, binned_statistic_2d

criteria  = (catalogue['mass']>1e2) 
#criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
criteria &= (catalogue['overlap_asc']==1) 

tmp = catalogue[criteria].copy()

fig,ax=plt.subplots(figsize=(single_column,single_column/1.2))

x,y,z = tmp['mass'],tmp['HA6562_LUM_CORR'],tmp['age']


xlim =[1e2,1e5]
ylim = [5e35,8e39]
nbins = 25
bins = [np.logspace(*np.log10(xlim),nbins),np.logspace(*np.log10(ylim),nbins)]

hist, x_e, y_e = np.histogram2d(x,y,bins=bins)
stat, x_e, y_e,_ = binned_statistic_2d(x,y,z,bins=bins)
#stat[hist<5] = np.nan
img = ax.pcolormesh(x_e,y_e,stat.T,cmap=plt.cm.viridis,vmin=0,vmax=10)

#sc = ax.scatter(x,y,c=z,vmin=0,vmax=10)
x_stat,mean,std = bin_stat(x,y,xlim,nbins=bins[0],statistic='median')
ax.errorbar(x_stat,mean,fmt='o-',ms=1.5,color='white')

rho = spearmanr(x,y,nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax.text(0.05,0.93,label,transform=ax.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

fig.colorbar(img,label='age / Myr')

ax.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

plt.savefig(basedir/'reports'/'tmp_mass_HA_age.png',dpi=600)

plt.show()

In [ ]:
from astrotools.plot.corner import corner_binned_percentile
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax1=plt.subplots(nrows=1,figsize=(single_column,single_column))

# ----------------------- ax1 ------------------------------------------
criteria  = (catalogue['mass']>1e2) 
#criteria &= (catalogue['age']<=8) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()

q1 = 68
q2 = 98

xlim =[8e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
vmin,vmax = 0,12
nbins = 8
bins = np.logspace(2.7,5.2,nbins)

cmap = mpl.cm.get_cmap('plasma',6)
norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)
rho = []
age_bins = [0,2,4,6,8,10,12]
for i in range(len(age_bins)-1):

    sub = tmp[(tmp['age']>=age_bins[i]) & (tmp['age']<age_bins[i+1])]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    x,mean,std = bin_stat(x,y,xlim,nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(age_bins[i]+1)))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']
x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)

rho = spearmanr(x,y,nan_policy='omit')[0]
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=19*[1],cmap=cmap,vmin=vmin,vmax=vmax)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('top', size="10%", pad=0.3)
cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal')
cbar.ax.xaxis.set_ticks_position('top')

plt.savefig(basedir/'reports'/'tmp_mass_HA.png',dpi=600)
plt.show()

In [ ]:
from astrotools.plot.utils import bin_stat
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import spearmanr, binned_statistic, binned_statistic_2d

fig,(ax1,ax2)=plt.subplots(nrows=2,figsize=(single_column,single_column*2))

# ----------------------- ax1 ------------------------------------------
criteria  = (catalogue['mass']>1e2) 
criteria &= (catalogue['age']<=8) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()

q1 = 68
q2 = 98

xlim =[8e2,1e5]
ylim = [1e4,5e7]
ylim = [1e36,8e39]
nbins = 8
bins = np.logspace(2.7,5.2,nbins)

cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)
rho = []
for k,gal_name in enumerate(np.unique(sample_table['name'])):

    sub = tmp[tmp['gal_name']==gal_name]
    x,y = sub['mass'],sub['HA6562_LUM_CORR']
    if len(x)>5:
        rho.append(spearmanr(x,y,nan_policy='omit')[0]) 
    x,mean,std = bin_stat(x,y,xlim,nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
label = r'$\rho'+f'={np.nanmean(rho):.2f}$'
t = ax1.text(0.05,0.93,label,transform=ax1.transAxes,ha='left',va='top')
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

x,y = tmp['mass'],tmp['HA6562_LUM_CORR']

x,y = x[~np.isnan(y) & np.isfinite(y)],y[~np.isnan(y) & np.isfinite(y)]
corner_binned_percentile(x,y,ax1,nbins=bins,n=68,color='gray',alpha=0.6)
corner_binned_percentile(x,y,ax1,nbins=bins,n=98,color='gray',alpha=0.3)
#x,mean,std = bin_stat(x,y,[None,None],nbins=bins,statistic='median')
#ax.errorbar(x,mean,fmt='o-',ms=2.5,color='black')    

ax1.set(xscale='log',yscale='log',xlim=xlim,ylim=ylim,
        xlabel=r'mass / M$_\odot$',ylabel=r'$L(\mathrm{H}\alpha)$ / erg s$^{-1}$')

# we need a scatter plot instance for the color bar
sc = ax1.scatter(19*[1],19*[1],c=sample_table['distance'],cmap=cmap,vmin=9.4,vmax=11)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('top', size="10%", pad=0.3)
cbar = fig.colorbar(sc,label='log M / M$_\odot$',cax=cax,orientation='horizontal')
cbar.ax.xaxis.set_ticks_position('top')

# ----------------------- ax2 ------------------------------------------
criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'ax1: {len(tmp)} objects')

cmap = plt.cm.get_cmap('viridis',10)


xlim = [0,10]
sc=ax2.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=cmap)
EBV_balmer_err = np.mean(tmp['EBV_balmer_err'])
EBV_stars_err = np.mean(tmp['EBV_stars_err'])
ax2.errorbar(0.5,0.1,xerr=EBV_stars_err,yerr=EBV_balmer_err,fmt='ko',ms=0)

ax2.plot([0,1],[0,2],color='black')
ax2.plot([0,2],[0,2],color='black')
ax2.set(xlim=[0,0.7],ylim=[0,0.7],xlabel=r'$E(B-V)$ stars',ylabel=r'$E(B-V)$ Balmer')

divider = make_axes_locatable(ax2)
cax = divider.append_axes('top', size="10%", pad=0.3)
cbar = fig.colorbar(sc,label='age / Myr',cax=cax,orientation='horizontal')
cbar.ax.xaxis.set_ticks_position('top')

cbar.set_ticks([0,2,4,6,8,10])
cbar.set_ticklabels([0,2,4,6,8,'10+'])

plt.tight_layout()
plt.savefig(basedir/'reports'/'mass_HA_EBV.pdf',dpi=600)
plt.show()

### Corner Plot

In [ ]:
# define limits and labels for corner plots
limits   = {'age':(0.5,8.5),
            'log_age':(5.9,7),
            'age_mw':(0,20),
            'met_scal':(8.3,8.7),
            'Delta_met_scal':(-0.07,0.07),
            'density':(0,150),
            'dig/hii':(0.2,0.9),
            'fesc':(0,1),
            'HA/FUV':(0,80),
            'HA/NUV':(0,50),
            'HA/FUV_corr':(0,80),
            'log_HA':(4.5,7.4),
            'EBV_balmer':(0,1),
            'sb_HA_pc':(0,200),
            'sb_HA_arcsec':(0,1e5),
            'log[SIII]/[SII]':(-1.5,0),
            'OIII/HB':(0,1),
            'logq_D91':(6.2,7.8),
            'eq_width':(10,150),
            'log_eq_width':(1.2,2.6),
            'temperature':(6000,8e3),
            'T_N2_REFIT':(6000,8e3),
            'log_mass':(4,5.5)}

labels   = {'age':'age / Myr',
            'log_age' : r'$\log (\mathrm{age / Myr})$',
            'age_mw':'age (stellar pops) / Myr',
            'met_scal':'12+logO/H',
            'Delta_met_scal':r'$\Delta$(O/H)',
            'density':r'density / cm$^{-3}$','temperature':'T / K',
            'fesc':r'$f_\mathrm{esc}$',
            'HA/FUV':r'H$\alpha$ / FUV',
            'HA/FUV_corr':r'H$\alpha$ / FUV',
            'OIII/HB':r'$[\mathrm{O}\,\textsc{iii}]/\mathrm{H}\beta$',
            'log_HA':r'$\log \mathrm{H}\alpha$ ',
            'EBV_balmer':r'$E(B-V)_\mathrm{Balmer}$',
            'sb_HA':r'$SB_{\mathrm{H}\alpha}$',
            'eq_width' : r'$\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA}$',
            'log_eq_width':r'$\log (\mathrm{EW}(\mathrm{H}\alpha)/\mathrm{\AA})$',
            'logq_D91':r'$\log q$',
            'log[SIII]/[SII]':r'$\log([\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}])$',
            'T_N2_REFIT':'temperature / K',
            'log_mass':r'$\log M/\mathrm{M}_\odot$'}
scale = {'eq_width':'log','HA/FUV':'log'}

# calculate a few additional columns
with np.errstate(divide='ignore',invalid='ignore'):
    catalogue['log_age'] = 6+np.log10(catalogue['age'])
    catalogue['log_eq_width'] = np.log10(catalogue['eq_width'])
    catalogue['log_HA'] = np.log10(catalogue['HA6562_FLUX_CORR'])
    area_per_pixel = ((0.2*u.arcsec).to(u.rad).value*catalogue['distance']*u.Mpc).to(u.pc)**2
    catalogue['sb_HA_pc']=catalogue['HA6562_FLUX_CORR']/(catalogue['area_neb']*area_per_pixel.value)
    catalogue['sb_HA_arcsec']=catalogue['HA6562_FLUX_CORR']/(catalogue['area_neb']*0.2**2)
    catalogue['log[SIII]/[SII]'] = np.log10(catalogue['[SIII]/[SII]'])
    catalogue['log_mass'] = np.log10(catalogue['mass'])
    catalogue['OIII/HB'] = catalogue['OIII5006_FLUX']/catalogue['HB4861_FLUX_CORR']

correlations with age

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,nbins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

criteria  = (catalogue['mass']>1e4) 
#criteria &= (catalogue['HA/FUV']>3*catalogue['HA/FUV_err']) | np.isnan(catalogue['FUV_FLUX'])
#criteria &= catalogue['[SIII]/[SII]']>3*catalogue['[SIII]/[SII]_err']
criteria &= (catalogue['age']<=8)
# young objects should be associated with a GMC
#criteria &= ((catalogue['GMC_sep']<4) | (catalogue['age']>2))
#criteria &= catalogue['U_dolmag_vega']-catalogue['B_dolmag_vega']<-1.
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
tmp['HA/FUV_corr'][tmp['FUV_FLUX_CORR']/tmp['FUV_FLUX_CORR_ERR']<3] = np.nan
print(f'sample contains {len(tmp)} objects')


columns  = ['age','eq_width','HA/FUV_corr','logq_D91','Delta_met_scal']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=filename,histogram=False,
           figsize=two_column,aspect_ratio=1)
plt.show()

correlations in the nebulae catalogue

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,nbins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds,rasterized=True)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    

tmp = nebulae.copy()
print(f'sample contains {len(tmp)} objects')
tmp['log_eq_width'] = np.log10(tmp['eq_width'])
tmp['log_HA'] = np.log10(tmp['HA6562_FLUX_CORR'])

columns  = ['eq_width','HA/FUV_corr','log_HA','T_N2_REFIT','density','EBV_balmer']
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=basedir/'reports'/f'corner_large.pdf',
           figsize=2*two_column,aspect_ratio=1)
plt.show()

In [ ]:
from astrotools.plot.corner import corner_by_galaxy

columns  = ['age','log_eq_width','HA/FUV_corr','logq_D91','Delta_met_scal']
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner_by_galaxy(tmp[tmp['gal_name']!='NGC1365'],sample_table=sample_table,columns=columns,
           limits=limits,labels=labels,nbins=5,filename=None,
           figsize=two_column,aspect_ratio=1)

histograms binned by age

In [ ]:
criteria  = (catalogue['mass']>=1e4) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= ((catalogue['GMC_sep']<2) | (catalogue['age']>2))

tmp = catalogue[criteria].copy()

fig,(ax1,ax2,ax3,ax4) = plt.subplots(ncols=4,figsize=(1.5*two_column,1.5*two_column/3))

style = {'alpha':0.5}
nbins = 10
age_bins = [0,2,8]
for i in range(len(age_bins)-1):
    sub = tmp[(tmp['age']>age_bins[i]) & (tmp['age']<=age_bins[i+1])]
    label = f'{age_bins[i]}\> age \>= {age_bins[i+1]} Myr: '
    
    ax1.hist(sub['eq_width'],bins=np.linspace(*limits['eq_width'],nbins),**style,
             label=label+f"{np.nanmean(sub['eq_width']):.1f}")
    ax2.hist(sub['HA/FUV_corr'],bins=np.linspace(*limits['HA/FUV_corr'],nbins),**style,
            label=label+f"{np.nanmedian(sub['HA/FUV_corr']):.1f}")
    ax3.hist(sub['Delta_met_scal'],bins=np.linspace(*limits['Delta_met_scal'],nbins),**style,
            label=label+f"{np.nanmean(sub['Delta_met_scal']):.2f}")
    ax4.hist(sub['logq_D91'],bins=np.linspace(*limits['logq_D91'],nbins),**style,
             label=label+f"{np.nanmean(sub['logq_D91']):.1f}")
    
ax1.set(xlabel=labels['eq_width'],xlim=limits['eq_width'])
ax2.set(xlabel=labels['HA/FUV_corr'],xlim=limits['HA/FUV_corr'])
ax3.set(xlabel=labels['Delta_met_scal'],xlim=limits['Delta_met_scal'])
ax4.set(xlabel=labels['logq_D91'],xlim=limits['logq_D91'])
ax1.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax2.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax3.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)
ax4.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=1)

plt.tight_layout()

plt.show()

### compare with starburst99

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster_CSF = Cluster(basedir/'..'/'starburst'/'data'/'others'/'continuous_SF')
cluster_SSF = Cluster(basedir/'..'/'starburst'/'data'/'others'/'single_burst_SF')
cluster_SSF.measure_FUV()
cluster_CSF.measure_FUV()

In [ ]:
# Halpha/FUV
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(two_column,two_column/2))

tmp = catalogue[(catalogue['mass']>1e4) & (catalogue['overlap_asc']==1)]
ax1.scatter(tmp['age'],tmp['HA/FUV_corr'],color='gray')

time = cluster_CSF.ewidth['Time']/1e6
Ha  = cluster_CSF.ewidth['Luminosity_H_A'].copy()
FUV = np.interp(cluster_CSF.ewidth['Time'],cluster_CSF.FUV['Time'],cluster_CSF.FUV['FUV'])
ax1.plot(time,Ha/FUV,label='continuous')

time = cluster_SSF.ewidth['Time']/1e6
Ha  = cluster_SSF.ewidth['Luminosity_H_A'].copy()
FUV = cluster_SSF.FUV['FUV'].copy()
ax1.plot(time,Ha/FUV,label='single burst')
ax1.legend()
ax1.set(xlim=[0,10],ylim=[0,100],xlabel='age / Myr',ylabel=r'H$\alpha$/FUV')


ax2.scatter(tmp['age'],tmp['eq_width'],color='gray')
time = cluster_CSF.ewidth['Time']/1e6
eq_width  = cluster_CSF.ewidth['eq_width_H_A']
ax2.plot(time,0.1*eq_width,label='continuous')

time = cluster_SSF.ewidth['Time']/1e6
eq_width  = cluster_SSF.ewidth['eq_width_H_A']
ax2.plot(time,0.1*eq_width,label='singel burst')
ax2.legend()
ax2.set(xlim=[0,10],ylim=[0,400],yscale='linear',xlabel='age / Myr',ylabel=r'EW(H$\alpha$)')

plt.tight_layout()

plt.show()

the distribution of EW observed vs model

In [ ]:
from starburst import Cluster, find_model, make_folder, list_available_models

cluster_solar = Cluster(stellar_model='GENEVAv40',metallicity=0.014)
cluster_subsolar = Cluster(stellar_model='GENEVAv40',metallicity=0.004)


In [ ]:
tmp = catalogue.copy()

eq_width_model_solar = cluster_solar.ewidth['eq_width_H_A']
eq_width_model_subsolar = cluster_subsolar.ewidth['eq_width_H_A']
age_model = cluster_solar.ewidth['Time']

tmp['eq_width_model_solar'] = np.nan
tmp['eq_width_model_subsolar'] = np.nan
for row in tmp:
    idx = np.argmin(np.abs(age_model-row['age']*u.Myr))
    row['eq_width_model_solar'] = eq_width_model_solar[idx].value
    row['eq_width_model_subsolar'] = eq_width_model_subsolar[idx].value

In [ ]:
fig,ax=plt.subplots(figsize=(single_column,single_column/1.618))

bins = np.logspace(0,4,15)

ax.hist(tmp['eq_width'],bins=bins,label='observed',alpha=0.6)
ax.hist(tmp['eq_width_model_solar'],bins=bins,label='model solar',alpha=0.6)
#ax.hist(tmp['eq_width_model_subsolar'],bins=bins,label='model subsolar',alpha=0.6)
ax.legend()
ax.set(xscale='log',xlim=[1,1e4],xlabel=r'$\mathrm{EW}(\mathrm{H}\alpha)$')

plt.show()

### other things like violine plots

In [ ]:
import warnings
from astrotools.plot import corner
from astrotools.plot.corner import corner_scatter, corner_binned_stat, corner_density_scatter,\
                                corner_density_histogram, corner_gaussian_kde_scatter,\
                                corner_binned_stat2d, corner_binned_stat2d_histogram,\
                                corner_binned_percentile,corner_violin, corner_spearmanr
                             
def plot_function(x,y,ax,xlim=None,**kwargs):
    nbins = 7
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=68,color='gray',alpha=0.6)
    corner_binned_percentile(x,y,ax,nbins=nbins,range=xlim,n=98,color='gray',alpha=0.2)
    corner_binned_stat(x,y,ax,nbins=nbins,color='0.3')
    #corner_density_histogram(x,y,ax,nbins=10,cmap=plt.cm.gray_r)
    #corner_scatter(x,y,ax,s=0.5)
    #corner_gaussian_kde_scatter(x,y,ax,s=0.5,cmap=plt.cm.Reds)
    corner_density_scatter(x,y,ax,nbins=20,s=0.5,cmap=plt.cm.Reds)
    corner_spearmanr(x,y,ax,position=(0.93,0.93),pvalue=False,fontsize=7)
    
filename = basedir/'reports'/f'corner_{HSTband}_{scalepc}pc.pdf'

criteria  = (catalogue['mass']>1e4) 
#criteria &= (catalogue['HA/FUV']>3*catalogue['HA/FUV_err']) | np.isnan(catalogue['FUV_FLUX'])
#criteria &= catalogue['[SIII]/[SII]']>3*catalogue['[SIII]/[SII]_err']
criteria &= (catalogue['age']<=8) #& (catalogue['age']>2)
# young objects should be associated with a GMC
#criteria &= ((catalogue['GMC_sep']<4) | (catalogue['age']>2))
#criteria &= catalogue['U_dolmag_vega']-catalogue['B_dolmag_vega']<-1.
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['log_age'] = 6+np.log10(tmp['age'])
    tmp['log_eq_width'] = np.log10(tmp['eq_width'])
    tmp['log[SIII]/[SII]'] = np.log10(tmp['[SIII]/[SII]'])

columns  = ['logq_D91','Delta_met_scal','HA/FUV_corr','log_eq_width']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=plot_function,
           limits=limits,labels=labels,
           filename=None,
           figsize=two_column,aspect_ratio=1)
plt.show()

In [ ]:
fig,ax=plt.subplots()


x = tmp['r_R25']
y = tmp['log_eq_width']

ax.scatter(x,y,c='gray')
corner_binned_stat(x,y,ax)
corner_spearmanr(x,y,ax,pvalue=True)

ax.set(xlim=[0,1],ylim=[0,3])
plt.show()

use a violin plot to show the distribution of points in each age bin

In [ ]:
fig,ax = plt.subplots()

x_label = 'age'
y_label = 'eq_width'

x,y = tmp[x_label], tmp[y_label]
x,y = x[~np.isnan(y)],y[~np.isnan(y)]
x,y = x[(x>=limits[x_label][0]) & (x<=limits[x_label][1]) & (y>=limits[y_label][0]) & (y<=limits[y_label][1])], \
      y[(x>=limits[x_label][0]) & (x<=limits[x_label][1]) & (y>=limits[y_label][0]) & (y<=limits[y_label][1])]

positions = np.arange(0,10)

#binned_data = corner_violin(x,y,ax,positions,showmedians=True)

bins = (positions[1:]+positions[:-1])/2
binned_data = [y[(bins[i]<x) & (x<bins[i+1])] for i in range(len(bins)-1)]
ax.violinplot(binned_data,positions=positions[1:-1],showmeans=True)

ax.set(xlabel=x_label,ylabel=y_label.replace('_',''))

plt.show()

filter age with color-color-diagram

In [ ]:
points = np.array([[-0.33 , -1.662],[-0.307, -1.577],[-0.25 , -1.468],[-0.218, -1.394],[-0.186, -1.321],
                   [-0.138, -1.235],[-0.076, -1.213],[-0.007, -1.206],[ 0.065, -1.197],[ 0.135, -1.187],
                   [ 0.203, -1.198],[ 0.276, -1.201],[ 0.347, -1.209],[ 0.418, -1.219],[ 0.488, -1.233],
                   [ 0.559, -1.251],[ 0.63 , -1.27 ],[ 0.706, -1.264],[ 0.771, -1.287],[ 0.648, -1.119],
                   [ 0.626, -1.026],[ 0.565, -0.989],[ 0.559, -0.87 ],[ 0.502, -0.807],[ 0.481, -0.706],
                   [ 0.457, -0.546],[ 0.505, -0.449],[ 0.487, -0.384],[ 0.488, -0.328],[ 0.468, -0.239],
                   [ 0.517, -0.109],[ 0.555, -0.015],[ 0.6  ,  0.041],[ 0.65 ,  0.132],[ 0.724,  0.174],
                   [ 0.864,  0.169],[ 0.948,  0.166],[ 1.02 ,  0.152],[ 1.138,  0.182],[ 1.203,  0.194],
                   [ 1.253,  0.282],[ 1.331,  0.378],[ 1.354,  0.469]])

x,y=points.T
fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(2.3*single_column,1*single_column))

tmp = associations
#ax.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],s=0.5)
tmp = catalogue
sc=ax1.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],
           c=tmp['GMC_sep'],vmin=0,vmax=10,
           s=0.5)
ax1.plot(x,y,color='red')
ax1.set(xlabel=r'$V-I$',ylabel=r'$U-B$',xlim=[-1.5,2],ylim=[-2.5,1])
ax1.invert_yaxis()
plt.colorbar(sc,label='GMC sep / arcsec',ax=ax1)

sc=ax2.scatter(tmp['V_dolmag_vega']-tmp['I_dolmag_vega'],tmp['U_dolmag_vega']-tmp['B_dolmag_vega'],
           c=tmp['age'],vmin=0,vmax=20,cmap=plt.cm.plasma,
           s=0.5)
ax2.plot(x,y,color='red')
ax2.set(xlabel=r'$V-I$',ylabel=r'$U-B$',xlim=[-1.5,2],ylim=[-2.5,1])
ax2.invert_yaxis()
fig.colorbar(sc,label='age / Myr',ax=ax2)

plt.tight_layout()

plt.savefig('UBVI_matched_associations.png',dpi=600)
plt.show()

In [ ]:
fig,ax=plt.subplots()

bins=np.arange(0,150,10)
tmp = catalogue[(catalogue['age']==1) & (catalogue['overlap_asc']==1)]
mask = tmp['GMC_sep']<4

ax.hist(tmp['eq_width'][mask],bins=bins,label=f'nearby GMC: {np.mean(tmp["eq_width"][mask]):.2f} AA',histtype='step',alpha=0.8)
ax.hist(tmp['eq_width'][~mask],bins=bins,label=f'no GMC {np.mean(tmp["eq_width"][~mask]):.2f} AA',histtype='step',alpha=0.8)
ax.set(xlabel=r'EW(H$\alpha$)')
ax.set_title('young associations (age 1 Myr)')
ax.legend()
plt.show()

In [ ]:
import warnings
from astrotools.plot import corner

# define limits and labels for corner plots
limits   = {'age':(0,20),
            'UB':(-2.0,0),
            'CO':(0,200),
            'GMC_sep':(0,10)}

labels   = {'age':'age / Myr',
            'UB':r'$U-B$',
            'CO':r'CO flux',
            'GMC_sep':r'GMC sep / arcsec'}

                             
tmp = catalogue.copy()
#tmp = tmp[tmp['GMC_sep']<5]
print(f'sample contains {len(tmp)} objects')
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['UB'] = tmp['U_dolmag_vega']-tmp['B_dolmag_vega']

columns  = ['age','GMC_sep','CO','UB']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    corner(tmp,columns,function=None,
           limits=limits,labels=labels,
           filename=None,
           figsize=two_column,aspect_ratio=1,s=0.5)
plt.show()

the old nebulae with high EW might come from Wolf Rayet Stars

In [ ]:
# the old nebulae with high EW might come from Wolf Rayet Stars
WR_candidates = catalogue[(catalogue['age']>6) & (catalogue['age']<10) & (catalogue['eq_width']>80) & (catalogue['mass']>5e3)]
print(f'{len(WR_candidates)} WR candidates')
groups = WR_candidates.group_by('gal_name')
spectra = []
for key, group in zip(groups.groups.keys, groups.groups):
    filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spectra'/f'{key["gal_name"]}_VorSpectra.fits'
    with fits.open(filename) as hdul:
        tbl = Table(hdul[1].data)
        spectral_axis = np.exp(Table(hdul[2].data)['LOGLAM'])*u.Angstrom
    tbl['region_ID'] = np.arange(len(tbl))
    tbl.add_index('region_ID')
    for region_ID in group['region_ID']:
        spectra.append((key['gal_name'],region_ID,spectral_axis,tbl.loc[region_ID]['SPEC']))
        
        hdu = fits.BinTableHDU(WR_candidates[['gal_name','region_ID','assoc_ID','x_neb','y_neb','age','mass','eq_width']],
                       name='WR_candidates')
hdu.writeto(basedir/'data'/'WR_candidates.fits',overwrite=True)

from matplotlib.backends.backend_pdf import PdfPages

filename = basedir/'reports'/'WR_candidates_spectra'
nrows=6
ncols=1
width = two_column
N = len(WR_candidates)
Npage = nrows*ncols
Npages = int(np.ceil(N/Npage))
with PdfPages(filename.with_suffix('.pdf')) as pdf:

    for i in range(Npages):
        print(f'working on page {i+1} of {Npages}')

        sub_sample = spectra[i*Npage:(i+1)*Npage]

        fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width*1.41))
        axes_iter = iter(axes.flatten())

        for row in sub_sample:  
            gal_name,region_ID,spectral_axis,spectrum = row
            ax = next(axes_iter)
            ax.plot(spectral_axis,spectrum,lw=0.5)
            ax.set(xlim=[4860,6800],yscale='log',ylim=[1e2,1e6])
            t = ax.text(0.01,0.87,f'{gal_name}: {region_ID:.0f}', transform=ax.transAxes,color='black',fontsize=8)
        plt.subplots_adjust(wspace=-0.1, hspace=0)

        # only the last page has subplots that need to be removed
        if i == int(np.ceil(N/Npage))-1:
            h,l = fig.axes[0].get_legend_handles_labels()
            ax = next(axes_iter)
            ax.axis('off')
            #ax.legend(h[::len(h)-1],l[::(len(l)-1)],fontsize=7,loc='center',frameon=False)
            #t = ax.text(0.06,0.87,'galaxy name: region ID', transform=ax.transAxes,color='black',fontsize=8)

            for i in range(nrows*ncols-len(sub_sample)-1):
                # remove the empty axes at the bottom
                ax = next(axes_iter)
                ax.axis('off')    

        pdf.savefig()  # saves the current figure into a pdf page
        plt.close()


use elipses for uncertainties

In [ ]:
from matplotlib.patches import Ellipse

ellipses = [Ellipse((row['age'],row['HA/FUV']),
                     row['age_err'],row['HA/FUV_err'],
                    alpha=0.5) for row in tmp]

fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

for e in ellipses:
    ax.add_artist(e)

ax.set(xlim=(0, 10),ylim=(0,80),xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))

ax.errorbar(tmp['age'],tmp['HA/FUV'],xerr=tmp['age_err'],yerr=tmp['HA/FUV_err'],fmt='o')

ax.set(xlim=(0, 10),ylim=(0,80),xlabel='age / Myr',ylabel=r'H$\alpha$ / FUV')

plt.show()

look for correlations by comparing all columns

In [ ]:
from itertools import combinations
from scipy.stats import spearmanr

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=8)
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#criteria &= (catalogue['neighbors']==0)

tmp = catalogue[criteria].copy()
with np.errstate(divide='ignore',invalid='ignore'):
    tmp['log_age'] = 6+np.log10(tmp['age'])
    tmp['log_eq_width'] = np.log10(tmp['eq_width'])
    tmp['log[SIII]/[SII]'] = np.log10(tmp['[SIII]/[SII]'])
print(len(tmp))
    
columns = ['HA/FUV','eq_width','Delta_met_scal','logq_D91','age','mass',
           'EBV_balmer','EBV_stars','GMC_sep','density','temperature']

correlation = []
pairs = []
for a,b in combinations(columns,2):
    y,x = tmp[a],tmp[b] 
    not_nan = ~np.isnan(x) & ~np.isnan(y)
    r,p = spearmanr(x[not_nan],y[not_nan])
    correlation.append(r)
    pairs.append(a+'+'+b)
correlation = Table([correlation,pairs],names=['r','pair'])
correlation.sort('r')

correlation

In [ ]:
limits['EBV_balmer'] = (0,1)
limits['EBV_stars'] = (0,1)
limits['GMC_sep'] = (0,10)

corner(tmp,['EBV_balmer','EBV_stars','HA/FUV','eq_width','GMC_sep'],function=plot_function,
       limits=limits,labels=labels,
       filename=None,
       figsize=two_column,aspect_ratio=1)

### Trends with escape fraction

In [ ]:
import ipywidgets as widgets
from cluster.plot import corner

mass_slider = widgets.FloatSlider(
    value=3.7,
    min=2,
    max=4.5,
    step=0.2,
    description='Cluster mass:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

age_slider = widgets.FloatLogSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Cluster age:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)
    
def plot_corner(mass,age):
    criteria  = (catalogue['mass']>mass) 
    criteria &= (catalogue['age']<age) 
    criteria &= (catalogue['overlap_neb']>0.1) 
    criteria &= (catalogue['overlap_asc']==1) 

    tmp = catalogue[criteria].copy()
    print(f'sample contains {len(tmp)} objects')


    filename = None #basedir/'reports'/f'all_galaxies_corner.png'
    columns  = ['age','HA/FUV','density','fesc','dig/hii','logq_D91','Delta_met_scal']
    limits   = {'HA/FUV':(0,100),'age':(0,10),'fesc':(0,1.1),'Delta_met_scal':(-0.1,0.1),
                'logq_D91':(6,8),'density':(0,250),
                'dig/hii':(0.2,0.9),'met_scal':(8.3,8.7)}

    corner(tmp,columns,limits,nbins=5,filename=None,vmin=1000,vmax=1e6,figsize=12,aspect_ratio=1)
    plt.show()
    
widgets.interact(plot_corner, mass=mass_slider,age=age_slider)

In [ ]:
from cluster.auxiliary import bin_stat

criteria  = (catalogue['mass']>5e3) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')


xlim = [0,1]
fig,ax=plt.subplots(figsize=(4,4/1.618))
sc = ax.scatter(tmp['fesc'],tmp['density'],s=10,alpha=0.8,vmin=0,vmax=1)

#x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
#ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='fesc',ylabel=r'[SII]6716 / [SII]6730',xlim=xlim)

#fig.colorbar(sc,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

#plt.savefig(basedir/'reports'/f'all_galaxies_HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:
from cluster.auxiliary import bin_stat

criteria  = (catalogue['mass']>5e3) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')


#xlim = [0,1]
fig,ax=plt.subplots(figsize=(4,4/1.618))
sc = ax.scatter(tmp['age'],tmp['mass'],s=10,alpha=0.8,vmin=0,vmax=1)

#x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
#ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='age',ylabel=r'mass')

plt.show()

### Abundance gradients

to help better understand the difference between abundances and local abundance offset

In [ ]:
from scipy.stats import binned_statistic
from astrotools.plot.utils import bin_stat
from scipy.stats import spearmanr


bins = np.logspace(-1.5,-0.2,10)
xlim = (1e-2,1e0)
cmap = mpl.cm.get_cmap('plasma')
norm = mpl.colors.Normalize(vmin=9.4,vmax=11)

fig,(ax1,ax2) = plt.subplots(ncols=2,figsize=(two_column,two_column/2))

table = nebulae[nebulae['[SIII]/[SII]']>3*nebulae['[SIII]/[SII]_err']]
bins = np.logspace(*np.log10(np.nanpercentile(table['[SIII]/[SII]'],(2,98))),10)

groups = table.group_by('gal_name')

ax1.scatter(table['[SIII]/[SII]'],table['met_scal'],color='gray',s=0.1)
ax2.scatter(table['[SIII]/[SII]'],table['Delta_met_scal'],color='gray',s=0.1)

rho1,rho2 = [], []
for group in groups.groups:
    gal_name = group[0]['gal_name']

    x,mean,std = bin_stat(group['[SIII]/[SII]'],group['met_scal'],[None,None],nbins=bins,statistic='median')
    ax1.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)

    x,mean,std = bin_stat(group['[SIII]/[SII]'],group['Delta_met_scal'],[None,None],nbins=bins,statistic='median')
    ax2.errorbar(x,mean,fmt='o-',ms=1.5,color=cmap(norm(sample_table.loc[gal_name]['mass'])),label=gal_name)
  
    r1,p1 = spearmanr(group['[SIII]/[SII]'],group['met_scal'],nan_policy='omit')
    r2,p2 = spearmanr(group['[SIII]/[SII]'],group['Delta_met_scal'],nan_policy='omit')
    #print(f'rho = {r1:.2f}, {r2:.2f}')
    rho1.append(r1)
    rho2.append(r2)
r,p = spearmanr(table['[SIII]/[SII]'],table['met_scal'],nan_policy='omit')
t = ax1.text(0.07,0.9,r'$\rho'+f'={r:.2f}$',transform=ax1.transAxes,fontsize=7)
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

r,p = spearmanr(table['[SIII]/[SII]'],table['Delta_met_scal'],nan_policy='omit')
t = ax2.text(0.07,0.9,r'$\rho'+f'={r:.2f}$',transform=ax2.transAxes,fontsize=7)
t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

ax1.set(xscale='log',xlim=[2e-2,1],ylim=[8,8.8],
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$12+\log (\mathrm{O}/\mathrm{H})$')
ax2.set(xscale='log',xlim=[2e-2,1],ylim=[-0.15,0.15],
        xlabel=r'$[\mathrm{S}\,\textsc{iii}]/[\mathrm{S}\,\textsc{ii}]$',ylabel=r'$\Delta$(O/H)')
plt.tight_layout()

plt.show()

In [ ]:
abundance_gradients = ascii.read(basedir/'data'/'external'/'radial_abundance_gradients.txt',
                                names=['name','R0','g_r25'])
abundance_gradients.add_index('name')

$$
\Delta = 12+\log (O/H) - g
$$

In [ ]:
gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
ax.scatter(tmp['r_R25'],tmp['met_scal'])
r = np.linspace(0,0.5)
ax.plot(r,R0+r*g_r25,color='black')

In [ ]:
for gal_name in np.unique(catalogue['gal_name']):
    tmp = catalogue[catalogue['gal_name']==gal_name]
    plt.scatter(tmp['met_scal'],tmp['Delta_met_scal'],c=tmp['r_R25'])
    #plt.scatter(tmp['galactic_radius'],tmp['Delta_met_scal'])
    break

In [ ]:

gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
sc=ax.scatter(tmp['met_scal'],tmp['Delta_met_scal'],c=tmp['r_R25'])
fig.colorbar(sc,label='r / R25')
ax.set(xlabel='12+logOH',ylabel=r'$\Delta$')
plt.show()
#r = np.linspace(0,0.5)
#ax.plot(r,R0+r*g_r25,color='black')

In [ ]:

gal_name = 'NGC0628'
tmp = catalogue[catalogue['gal_name']==gal_name]
R0 = abundance_gradients.loc[gal_name]['R0']
g_r25 = abundance_gradients.loc[gal_name]['g_r25']

fig,ax=plt.subplots(figsize=(6,4))
sc=ax.scatter(tmp['logq_D91'],tmp['met_scal'],c=tmp['r_R25'])
fig.colorbar(sc,label='r / R25')
ax.set(xlabel='log q',ylabel=r'$\Delta$(O/H)')
plt.show()
#r = np.linspace(0,0.5)
#ax.plot(r,R0+r*g_r25,color='black')

In [ ]:
from cluster.auxiliary import bin_stat

fig,ax=plt.subplots(figsize=(two_column,two_column/1.618))
for gal_name in np.unique(nebulae['gal_name']):
    tmp = nebulae[nebulae['gal_name']==gal_name]
    x,mean,std = bin_stat(tmp['logq_D91'],tmp['Delta_met_scal'],[5,8.5],nbins=10)
    ax.errorbar(x,mean,yerr=std,fmt='-',label=gal_name)
    #ax.scatter(tmp['logq_D91'],tmp['Delta_met_scal'],label=gal_name)
ax.set(xlim=[5,8.5],ylim=[-0.2,0.2],xscale='linear')
#ax.legend()
plt.show()

### Age and Ha/FUV

In [ ]:
from cluster.auxiliary import bin_stat

criteria  = (catalogue['mass']>1e4) 
criteria &= (catalogue['age']<=10) 
#criteria &= (catalogue['overlap_neb']>0.2) 
criteria &= (catalogue['overlap_asc']==1) 
#tmp = catalogue[criteria].copy()
print(f'sample contains {len(tmp)} objects')


xlim = [0.5,10.5]
fig,ax=plt.subplots(figsize=(4,4/1.618))
sc = ax.scatter(tmp['age'],tmp['HA/FUV'],s=10,alpha=0.8,vmin=0,vmax=1)

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel=r'H$\alpha$ / FUV',xlim=xlim,ylim=[0,100])

#fig.colorbar(sc,label=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$')
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

#plt.savefig(basedir/'reports'/f'all_galaxies_HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:

fig,(ax1,ax2)=plt.subplots(ncols=2,figsize=(8,3))

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='contained')]
ax1.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

tmp = catalogue[(catalogue['mass']>1e3) & (catalogue['overlap']=='partial')]
sc=ax2.scatter(tmp['dig/hii'],tmp['HA/FUV'],c=tmp['age'],vmin=0,vmax=20,s=2)

ax1.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')
ax2.set(xlim=[0,1],ylim=[0,100],xlabel=r'$f_\mathrm{dig}/f_\mathrm{H\,\textsc{ii}}$',ylabel=r'H$\alpha$/FUV')

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.93, 0.11, 0.02, 0.84])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr')

plt.show()

seperate subplot for each galaxy

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky
from scipy.stats import binned_statistic, pearsonr, spearmanr

# '[SIII]/[SII]' , 'HA/FUV', 'AGE_MINCHISQ', 'AGE_BAYES'
x_name, y_name = 'age', 'HA/FUV'
xlim = [0.5,10.5]
bins = 10
max_sep = 1*u.arcsec

#sample = set(np.unique(nebulae['gal_name'])) & hst_sample
sample = np.unique(catalogue['gal_name'])[:-1]

filename = basedir/'reports'/'all_objects_age_over_SII.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 3
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    criteria = (catalogue['gal_name']==name) 
    criteria &= (catalogue['mass']>1e3) 
    #criteria &= ~np.isnan(catalogue['HA/FUV'])
    criteria &= (catalogue['overlap'] == 'contained')
    #criteria &= (catalogue['neighbors'] == 0)
    
    tmp = catalogue[criteria]
    print(f'{name}: {len(tmp)} objects')
    
    mean, bin_edges, binnumber = binned_statistic(tmp[x_name],
                                                  tmp[y_name],
                                                  statistic='mean',
                                                  bins=bins,
                                                  range=xlim)
    std, _, _ = binned_statistic(tmp[x_name],
                                  tmp[y_name],
                                  statistic='std',
                                  bins=bins,
                                  range=xlim)

    ax.scatter(tmp[x_name],tmp[y_name],color='tab:blue',s=1)
    # plot the standard divation with yerr=std
    ax.errorbar((bin_edges[1:]+bin_edges[:-1])/2,mean,fmt='-')
    ax.text(0.65,0.85,f'{name}', transform=ax.transAxes,fontsize=7)

    ax.set(xlim=xlim)
    if i==nrows-1:
        ax.set_xlabel(f'{x_name.replace("_"," ")}')
    if j==0:
        ax.set_ylabel(f'{y_name.replace("_"," ")}')

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


#plt.savefig(filename,dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky

# '[SIII]/[SII]' , 'HA/FUV'
x_name, y_name, z_name = '[SIII]/[SII]', 'HA/FUV', 'AGE_MINCHISQ'
xlim = [0.5,10.5]
bins = 10
max_sep = 2*u.arcsec

sample = muse_sample & hst_sample & astrosat_sample

filename = basedir/'reports'/'all_objects_FUV_over_SII_with_age.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 2
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # it makes a different if we match the clusters to the nebulae or the other way around
    catalogcoord = clusters[clusters['gal_name']==name].copy()
    matchcoord   = nebulae[nebulae['gal_name']==name].copy()

    idx, sep, _ = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])

    catalogue = matchcoord.copy()

    for col in catalogcoord.columns:
        if col in catalogue.columns:
            catalogue[f'{col}2'] = catalogcoord[idx][col]
        else:
            catalogue[col] = catalogcoord[idx][col]

    catalogue['sep'] = sep
    catalogue = catalogue[sep.__lt__(max_sep)]
    catalogue = catalogue[~np.isnan(catalogue[x_name]) & ~np.isnan(catalogue[y_name]) & (catalogue['AGE_MINCHISQ']<10)]
    print(f'{name}: {len(catalogue)} objects in joined catalogue')

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    #catalogue = catalogue[catalogue['FUV_FLUX_CORR']>3*catalogue['FUV_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SII6716_FLUX_CORR']>3*catalogue['SII6716_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SIII9068_FLUX_CORR']>3*catalogue['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(catalogue)} objects')

    sc = ax.scatter(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'],
                    c=catalogue[z_name],vmin=0, vmax=10,cmap=plt.cm.RdBu_r,
                    s=3,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.7,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.55,0.05,f'{len(catalogue):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,2e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel('[SIII]/[SII]')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

fig.colorbar(sc,ax=axes.ravel().tolist(),label=f'{z_name.replace("_"," ")}')
        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()


## Age trend or escape fraction

\begin{equation}
f_{\mathrm{esc}} = \frac{Q_p-Q_o}{Q_p} 
\end{equation}

first we assume a constant escape fraction of ~70%

In [ ]:
age=cluster.ewidth['Time']
HaFUV_pred=(cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV']).value

HaFUV_obs = 0.3*HaFUV_pred

fesc = 1-HaFUV_obs/HaFUV_pred

fig,(ax1,ax2,ax3,ax4)=plt.subplots(ncols=4,figsize=(2*two_column,two_column/2))

ax1.scatter(age/1e6,HaFUV_pred)
ax1.set(xlim=[0,10],ylim=[0,80],ylabel='Ha/FUV',xlabel='age / Myr')

ax2.scatter(age/1e6,HaFUV_obs)
ax2.set(xlim=[0,10],ylim=[0,80],ylabel='Ha/FUV observed',xlabel='age / Myr')

ax3.scatter(age/1e6,fesc)
ax3.set(xlim=[0,10],ylim=[0,1],xlabel='age / Myr',ylabel='fesc')

ax4.scatter(HaFUV_obs,fesc)
ax4.set(xlim=[0,80],ylim=[0,1],xlabel='Ha/FUV observed',ylabel='fesc')

plt.tight_layout()

plt.show()

next we assume that we still observe the theoretical trend, but with the original flux being constant

In [ ]:
age=cluster.ewidth['Time'].value/1e6
HaFUV_obs=(cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV']).value

HaFUV_pred = 80

fesc = 1-HaFUV_obs/HaFUV_pred

fig,(ax1,ax2,ax3,ax4)=plt.subplots(ncols=4,figsize=(2*two_column,two_column/2))

ax1.scatter(age,0*age+HaFUV_pred)
ax1.set(xlim=[0,10],ylim=[0,90],ylabel='Ha/FUV',xlabel='age / Myr')

ax2.scatter(age,HaFUV_obs)
ax2.set(xlim=[0,10],ylim=[0,80],ylabel='Ha/FUV observed',xlabel='age / Myr')

ax3.scatter(age,fesc)
ax3.set(xlim=[0,10],ylim=[0,1],xlabel='age / Myr',ylabel='fesc')

ax4.scatter(HaFUV_obs,fesc)
ax4.set(xlim=[0,80],ylim=[0,1],xlabel='Ha/FUV',ylabel='fesc')

plt.tight_layout()

plt.show()

finally we assume an escape fraction that is increasing over time

In [ ]:
age=cluster.ewidth['Time'].value/1e6
HaFUV_pred=(cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV']).value

fesc = 0.7/(1+np.exp(-1*(age-3)))

HaFUV_obs = (1-fesc)*HaFUV_pred

fig,(ax1,ax2,ax3,ax4)=plt.subplots(ncols=4,figsize=(2*two_column,two_column/2))

ax1.scatter(age,HaFUV_pred)
ax1.set(xlim=[0,10],ylim=[0,80],xlabel='age / Myr',ylabel='Ha/FUV')

ax2.scatter(age,HaFUV_obs)
ax2.set(xlim=[0,10],ylim=[0,80],xlabel='age / Myr',ylabel='Ha/FUV observed')


ax3.scatter(age,fesc)
ax3.set(xlim=[0,10],ylim=[0,1],xlabel='age / Myr',ylabel='fesc')

ax4.scatter(HaFUV_obs,fesc)
ax4.set(xlim=[0,80],ylim=[0,1],xlabel='Ha/FUV',ylabel='fesc')

plt.tight_layout()

plt.show()

what if we don't observe any trends but there should be? How is the escape fraction behaving

In [ ]:
age=cluster.ewidth['Time'].value/1e6
HaFUV_pred=(cluster.ewidth['Luminosity_H_A']/cluster.FUV['FUV']).value

HaFUV_obs = 30
fesc = 1-HaFUV_obs/HaFUV_pred


fig,(ax1,ax2,ax3,ax4)=plt.subplots(ncols=4,figsize=(2*two_column,two_column/2))

ax1.scatter(age,HaFUV_pred)
ax1.set(xlim=[0,10],ylim=[0,80],xlabel='age / Myr',ylabel='Ha/FUV')

ax2.scatter(age,HaFUV_obs+0*age)
ax2.set(xlim=[0,10],ylim=[0,80],xlabel='age / Myr',ylabel='Ha/FUV observed')


ax3.scatter(age,fesc)
ax3.set(xlim=[0,10],ylim=[0,1],xlabel='age / Myr',ylabel='fesc')

ax4.scatter(HaFUV_pred,fesc)
ax4.set(xlim=[0,80],ylim=[0,1],xlabel='Ha/FUV',ylabel='fesc')

plt.tight_layout()

plt.show()

## Pretty pictures

In [ ]:
name = 'NGC0628'

# DAP linemaps (Halpha and OIII)
filename = next((data_ext/'MUSE'/'DR2.1'/'copt'/'MUSEDAP').glob(f'{name}*.fits'))
copt_res = float(filename.stem.split('-')[1].split('asec')[0])
with fits.open(filename) as hdul:
    Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                    mask=np.isnan(hdul['HA6562_FLUX'].data),
                    meta=hdul['HA6562_FLUX'].header,
                    wcs=WCS(hdul['HA6562_FLUX'].header))
    OIII = NDData(data=hdul['OIII5006_FLUX'].data,
                    uncertainty=StdDevUncertainty(hdul['OIII5006_FLUX_ERR'].data),
                    mask=np.isnan(hdul['OIII5006_FLUX'].data),
                    meta=hdul['OIII5006_FLUX'].header,
                    wcs=WCS(hdul['OIII5006_FLUX'].header)) 

filename = data_ext/'Products'/'Nebulae_catalogs'/'Nebulae_catalogue_v2'/'spatial_masks'/f'{name}_nebulae_mask.fits'
with fits.open(filename) as hdul:
    nebulae_mask = NDData(hdul[0].data.astype(float),mask=Halpha.mask,meta=hdul[0].header,wcs=WCS(hdul[0].header))
    nebulae_mask.data[nebulae_mask.data==-1] = np.nan

# WFI image (larger FOV)
filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
with fits.open(filename) as hdul:
    WFI = NDData(data=hdul[0].data,
                 meta=hdul[0].header,
                 wcs=WCS(hdul[0].header))

# and for HST
from cluster.io import read_associations, ReadHST

target  = name.lower()

associations, associations_mask = read_associations(folder=data_ext/'Products'/'stellar_associations',target=target,
                                                    HSTband=HSTband,scalepc=scalepc,version=version)

# read the compact cluster catalogues
filename = data_ext/'Products'/'compact_clusters'/f'PHANGS_IR3_{name.lower()}_phangs-hst_v1p1_ml_class12.fits'
compact_clusters = Table.read(filename)
compact_clusters['SkyCoord'] = SkyCoord(compact_clusters['PHANGS_RA']*u.deg,compact_clusters['PHANGS_DEC']*u.deg)

hst_images = ReadHST(name,data_ext / 'HST' / 'filterImages' )

In [ ]:
from reproject import reproject_interp
from skimage.measure import find_contours
from regions import PixCoord, RectangleSkyRegion

position = SkyCoord('03h33m36.36s','-36d08m25.45s')
position = SkyCoord(*list(sample_table.loc[name][['R.A.','Dec.']]))
position = nebulae[(nebulae['Nassoc']>2) & (nebulae['gal_name']==name) & (nebulae['neighbors']==0)][3]['SkyCoord_neb']
size  = (10*u.arcsec,10*u.arcsec)

f275w_cutout = Cutout2D(hst_images.f275w.data,position,size=size,wcs=hst_images.f275w.wcs)
f336w_cutout = reproject_interp(hst_images.f336w,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    
f438w_cutout = reproject_interp(hst_images.f438w,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    
f555w_cutout = reproject_interp(hst_images.f555w,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    
f814w_cutout = reproject_interp(hst_images.f814w,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    
halpha_cutout  = reproject_interp(Halpha,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    

print('search for mask contours')
cutout_nebulae, _  = reproject_interp(nebulae_mask,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,order='nearest-neighbor')    
region_ID = np.unique(cutout_nebulae[~np.isnan(cutout_nebulae)])

contours_nebulae = []
for i in region_ID:
    blank_mask = np.zeros_like(cutout_nebulae)
    blank_mask[cutout_nebulae==i] = 1
    contours_nebulae += find_contours(blank_mask, 0.5)

cutout_assoc, _  = reproject_interp(associations_mask,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,order='nearest-neighbor')    
assoc_ID = np.unique(cutout_assoc[~np.isnan(cutout_assoc)])

contours_assoc = []
for i in assoc_ID:
    blank_mask = np.zeros_like(cutout_assoc)
    blank_mask[cutout_assoc==i] = 1
    contours_assoc += find_contours(blank_mask, 0.5)    


In [ ]:
import astrotools.plot.multicolorfits as mcf


def create_multicolorimage(*args):
    '''combine multiple images to a three color image'''
    
    color_images = []
    for arg in args:
        color = arg.pop('color')
        greyscale = mcf.greyRGBize_image(**arg)
        color_images.append(mcf.colorize_image(greyscale,color, colorintype='hex',gammacorr_color=2.2))
    
    return mcf.combine_multicolor(color_images, gamma=2.2)

p1,p2 = np.nanpercentile(halpha_cutout.data,[0,100])
f275_white  = {'color':'#d5c5e3','datin':f275w_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[15,99.9]}
f336_purple = {'color':'#9C4FFF','datin':f336w_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[15,99.9]}
f438_blue   = {'color':'#61b3cf','datin':f438w_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[15,99.8]}
f555_green  = {'color':'#00e32d','datin':f555w_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[15,99.8]}
f814_yellow = {'color':'#e08e1b','datin':f814w_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[15,99.8]}
ha_red_peak = {'color':'#ed7f5a','datin':halpha_cutout.data,'rescalefn':'linear','scaletype':'perc','min_max':[70,99.8]}
ha_red_dif  = {'color':'#ed5a95','datin':halpha_cutout.data,'rescalefn':'log','scaletype':'abs','min_max':[4*p1,8*p2]}

rgb = create_multicolorimage(f275_white,f336_purple,f438_blue,f555_green,f814_yellow,ha_red_peak,ha_red_dif)


In [ ]:

fig,ax=plt.subplots(figsize=(two_column,two_column))
ax.imshow(rgb,origin='lower')

for coords in contours_nebulae:
    ax.plot(coords[:,1],coords[:,0],color='red',lw=0.5,label=r'H\textsc{ii} region')

mask = np.zeros((*cutout_nebulae.shape,4))
mask[~np.isnan(cutout_nebulae.data),:] = (0.84, 0.15, 0.16,0.1)
ax.imshow(mask,origin='lower')

for coords in contours_assoc:
    ax.plot(coords[:,1],coords[:,0],color='blue',lw=0.5,label='association')

mask = np.zeros((*cutout_assoc.shape,4))
mask[~np.isnan(cutout_assoc.data),:] = (0.12,0.47,0.71,0.1)
ax.imshow(mask,origin='lower')

region = RectangleSkyRegion(position,0.9*size[0],0.9*size[0])
in_frame = compact_clusters[region.contains(compact_clusters['SkyCoord'],f275w_cutout.wcs)]
for row in in_frame:
    x,y = row['SkyCoord'].to_pixel(f275w_cutout.wcs)
    if 5<x<f275w_cutout.data.shape[0]-5 and 5<y<f275w_cutout.data.shape[1]-5:
        ax.scatter(x,y,marker='o',facecolors='none',s=40,lw=0.8,color='green',label='DOLPHOT peaks')
        if 'label' in compact_clusters.columns:
            ax.annotate(row['label'], (x+4, y),color='green')
    

ax.axis('off')

#plt.savefig(basedir/'reports'/f'{name}_rgb.png',dpi=300)
plt.show()


plot the full image

In [ ]:
position = SkyCoord(*list(sample_table.loc[name][['R.A.','Dec.']]))
size  = (4*u.arcmin,4*u.arcmin)

f275w_cutout = Cutout2D(hst_images.f275w.data,position,size=size,wcs=hst_images.f275w.wcs)
halpha_cutout  = reproject_interp(Halpha,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,return_footprint=False)    

cutout_nebulae, _  = reproject_interp(nebulae_mask,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,order='nearest-neighbor')    

cutout_assoc, _  = reproject_interp(associations_mask,output_projection=f275w_cutout.wcs,shape_out=f275w_cutout.shape,order='nearest-neighbor')    


In [ ]:
fig,ax=plt.subplots()

norm = simple_norm(f275w_cutout.data,clip=False,percent=99)

ax.imshow(f275w_cutout.data,norm=norm,cmap=plt.cm.Greys,origin='lower')
ax.imshow(cutout_nebulae.data,cmap=plt.cm.Reds,alpha=0.5)
ax.imshow(cutout_assoc.data,cmap=plt.cm.Blues,alpha=0.5)

ax.axis('off')

plt.savefig(basedir/'reports'/name/f'{name}_nebulae_associations.pdf',dpi=300)
plt.show()

### BPT diagrams

In [ ]:
from astrotools.plot import BPT_diagram

tmp = nebulae[~HIIregion_mask]

fig = BPT_diagram(R3=np.log10(tmp['OIII5006_FLUX_CORR']/tmp['HB4861_FLUX_CORR']),
            N2=np.log10(tmp['NII6583_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),
            S2=np.log10((tmp['SII6716_FLUX_CORR']+tmp['SII6730_FLUX_CORR'])/tmp['HA6562_FLUX_CORR']),
            O1=np.log10(tmp['OI6300_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),s=0.1,
            #c=tmp['fesc'],vmin=0,vmax=1,cmap=plt.cm.get_cmap('plasma_r', 5),alpha=0.9,s=2,label=r'$f_\mathrm{esc}$',
            filename=None)

tmp = nebulae[HIIregion_mask]
BPT_diagram(R3=np.log10(tmp['OIII5006_FLUX_CORR']/tmp['HB4861_FLUX_CORR']),
            N2=np.log10(tmp['NII6583_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),
            S2=np.log10((tmp['SII6716_FLUX_CORR']+tmp['SII6730_FLUX_CORR'])/tmp['HA6562_FLUX_CORR']),
            O1=np.log10(tmp['OI6300_FLUX_CORR']/tmp['HA6562_FLUX_CORR']),s=0.1,
            #c=tmp['fesc'],vmin=0,vmax=1,cmap=plt.cm.get_cmap('plasma_r', 5),alpha=0.9,s=2,label=r'$f_\mathrm{esc}$',
            #fig=fig,filename=basedir/'reports'/'BPT_nebulae.jpg'
            )


## Playground

In [ ]:
from PIL import Image

In [ ]:
pil_img = Image.fromarray(img)


In [ ]:
def get_dominant_color(pil_img):
    img = pil_img.copy()
    img = img.convert("RGBA")
    img = img.resize((1, 1), resample=0)
    dominant_color = img.getpixel((0, 0))
    return mpl.colors.to_hex([x/255 for x in dominant_color])

get_dominant_color(pil_img)

In [ ]:
lst = []
for gal_name in sample_table_v1p6['gal_name']:
    folder = data_ext / 'Products'/'stellar_associations'/f'associations_{version}'/f'{gal_name.lower()}_{HSTband}'/f'{scalepc}pc'
    catalogue_file = folder / f'{gal_name.lower()}_phangshst_associations_{HSTband}_ws{scalepc}pc_{version}.fits'

    table = Table.read(catalogue_file)
    table.rename_columns(['reg_id'],['assoc_ID'])
    table.add_column(gal_name,index=0,name='gal_name')
    table.add_column(sample_table_v1p6.loc[gal_name]['dist'],index=1,name='distance')
    colnames = sum([[f'{filter}_dolmag_vega',f'{filter}_dolmag_vega_err'] for filter in ['NUV','U','B','V','I']],[])
    lst.append(table[['gal_name','assoc_ID','distance']+colnames])
table = vstack(lst)
table.write(basedir/'data'/f'associations_{HSTband}_{scalepc}pc_{version}.fits',overwrite=True)